In [1]:
!pip -q install neuroHarmonize neuroCombat
!pip install --upgrade imbalanced-learn 
!pip install torch-geometric lightgbm xgboost catboost
!pip install --upgrade imbalanced-learn lightgbm xgboost catboost
!pip install umap-learn

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 93.8 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.6.1 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 17.3 MB/s eta 0:00:00
  Att

In [2]:
!pip -q install neuroHarmonize neuroCombat scikit-learn 
!pip install --upgrade scikit-learn
!pip install --upgrade imbalanced-learn

In [3]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [4]:
import pandas as pd 
import numpy as np 
import re
from tqdm import tqdm
import optuna

import seaborn as sns
import matplotlib.pyplot as plt

from neuroHarmonize import harmonizationLearn

from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, cross_val_score, train_test_split, cross_validate, cross_val_predict
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve, make_scorer, brier_score_loss
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.combine import SMOTEENN
from imblearn.pipeline import Pipeline as ImbPipeline

from scipy.stats import kstest, ttest_ind, ks_2samp, mannwhitneyu, mode

In [5]:
import re
import copy
import numpy as np
import pandas as pd
import networkx as nx

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
from category_encoders import TargetEncoder
from lightgbm import LGBMClassifier

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch_geometric.loader import DataLoader


from torch_geometric.data import Data
from torch_geometric.nn import GATConv, global_mean_pool, BatchNorm

import numpy as np
import pandas as pd
import re
import networkx as nx
from joblib import Parallel, delayed

from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from umap import UMAP

from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.utils import class_weight
from sklearn.calibration import CalibratedClassifierCV
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier

import optuna
import igraph as ig


import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from torch_geometric.nn import GATConv, global_mean_pool, BatchNorm
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
import igraph as ig
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GATConv, global_mean_pool, GINEConv, global_max_pool, global_add_pool, JumpingKnowledge
from torch.utils.data.sampler import WeightedRandomSampler
from sklearn.preprocessing import RobustScaler, PolynomialFeatures
from sklearn.metrics import classification_report
import json
from torch_geometric.utils import dropout_edge, dropout_adj

2025-04-30 11:11:51.824679: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746011512.004285      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746011512.058139      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === 1. LOAD DATA ===
def load_data():
    train_cat  = pd.read_excel('/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAIN_CATEGORICAL_METADATA_new.xlsx')
    train_num  = pd.read_excel('/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAIN_QUANTITATIVE_METADATA_new.xlsx')
    train_conn = pd.read_csv('/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv')
    test_cat   = pd.read_excel('/kaggle/input/widsdatathon2025/TEST/TEST_CATEGORICAL.xlsx')
    test_num   = pd.read_excel('/kaggle/input/widsdatathon2025/TEST/TEST_QUANTITATIVE_METADATA.xlsx')
    test_conn  = pd.read_csv('/kaggle/input/widsdatathon2025/TEST/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv')
    y_train    = pd.read_excel('/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAINING_SOLUTIONS.xlsx')
    sample_sub = pd.read_excel('/kaggle/input/widsdatathon2025/SAMPLE_SUBMISSION.xlsx')
    y_train.set_index('participant_id', inplace=True)
    return train_cat, train_num, train_conn, test_cat, test_num, test_conn, y_train, sample_sub

train_cat, train_num, train_conn, test_cat, test_num, test_conn, y_train, sample_sub = load_data()

y_train['label4'] = 2*y_train['ADHD_Outcome'] + y_train['Sex_F']

# === 2. META-PREPROCESSING ===
# Low-cardinality: one-hot all categorical
low_card = [c for c in train_cat.columns if train_cat[c].nunique() < 10 and c!='participant_id']
high_card = [c for c in train_cat.columns if c not in low_card + ['participant_id']]

conn_mat = train_conn.drop('participant_id', axis=1).values  # [N, E]
pca = PCA(n_components=350, random_state=42).fit(conn_mat)
conn_pca = pca.transform(conn_mat)

# 2.1. Standard numeric + OHE pipeline
num_cols = [c for c in train_num.columns if c!='participant_id']
numeric_pipe = Pipeline([("imputer", IterativeImputer(random_state=42)), ('scale', StandardScaler())])
categorical_pipe = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])
preprocessor = ColumnTransformer([
    ('num', numeric_pipe, num_cols),
    ('cat', categorical_pipe, low_card)
])
# Fit on train
X_meta_train = pd.concat([
    train_num.set_index('participant_id'),
    train_cat.set_index('participant_id')[low_card]
], axis=1)
X_meta_trans = preprocessor.fit_transform(X_meta_train)

# 2.2. TARGET‑ENCODING cho high_card, 2 maps: ADHD & Sex
global_mean = {
    'adhd': y_train['ADHD_Outcome'].mean(),
    'sex':  y_train['Sex_F'].mean()
}
α = 5  # smoothing parameter
enc_maps = {}
for c in high_card:
    df_enc = pd.concat([
        train_cat.set_index('participant_id')[c],
        y_train[['ADHD_Outcome','Sex_F']]
    ], axis=1)
    stats = df_enc.groupby(c).agg(['mean','count'])
    # flatten MultiIndex
    stats.columns = ['_'.join(col) for col in stats.columns]
    # compute smoothed encodings
    stats['enc_adhd'] = (stats['ADHD_Outcome_mean'] * stats['ADHD_Outcome_count'] + global_mean['adhd']*α) \
                        / (stats['ADHD_Outcome_count'] + α)
    stats['enc_sex']  = (stats['Sex_F_mean']       * stats['Sex_F_count']       + global_mean['sex']*α)  \
                        / (stats['Sex_F_count'] + α)
    enc_maps[c] = stats[['enc_adhd','enc_sex']].to_dict(orient='index')

# build X_enc: for each row, concatenate all enc_adhd then all enc_sex
def encode_high(df_cat):
    X_he = []
    for pid, row in df_cat.set_index('participant_id')[high_card].iterrows():
        enc_adhd = []
        enc_sex  = []
        for c in high_card:
            val = row[c]
            if pd.isna(val) or val not in enc_maps[c]:
                enc_adhd.append(global_mean['adhd'])
                enc_sex.append(global_mean['sex'])
            else:
                enc_adhd.append(enc_maps[c][val]['enc_adhd'])
                enc_sex.append(enc_maps[c][val]['enc_sex'])
        X_he.append(enc_adhd + enc_sex)
    return np.array(X_he)
X_enc = encode_high(train_cat)
X_enc_test = encode_high(test_cat)

# === 3. GRAPH DATA PREBUILD ===
# Parse edge indices
edges   = train_conn.columns.drop('participant_id')
# parse các cặp (u,v) từ tên cột, chuyển sang 0‑based nếu cần
valid_col_idx = []
for c in edges:
    u, v = map(lambda x: int(x)-1, re.findall(r"\d+", c))
    if u >= 0 and v >= 0:
        valid_col_idx.append((u, v))
col_idx = valid_col_idx
n_nodes = max(max(u, v) for u, v in col_idx) + 1

def build_data_list(df_conn, is_train=True):
    data_list = []
    for i, row in enumerate(df_conn.set_index('participant_id').iterrows()):
        pid, vals = row
        vals = vals.values  # chỉ lấy mảng 1D của các giá trị kết nối
    
        # Tính degree vector và edge lists
        deg = np.zeros(n_nodes, dtype=np.float32)
        src, dst, wt = [], [], []
        for (u, v), w in zip(col_idx, vals):
            # u, v giờ là 0..n_nodes-1
            deg[u] += abs(w)
            deg[v] += abs(w)
            # thêm cả hai chiều cho GAT/GCN
            src += [u, v]
            dst += [v, u]
            wt  += [w, w]
    
        data = Data(
            x=torch.tensor(deg, dtype=torch.float).unsqueeze(1),
            edge_index=torch.tensor([src, dst], dtype=torch.long),
            edge_attr=torch.tensor(wt, dtype=torch.float).unsqueeze(1),
        )
        # gán index global để later dùng batch.idx
        data.idx = torch.tensor(i, dtype=torch.long)
        if is_train:
            label = y_train.loc[pid, ['ADHD_Outcome','Sex_F']].values.astype(float)
            data.y = torch.tensor(label, dtype=torch.float32).unsqueeze(0)
        data_list.append(data)
    return data_list
data_list = build_data_list(train_conn, is_train=True)
test_list = build_data_list(test_conn,  is_train=False)
# === 4. GNN MODEL ===
# --- Model Definition ---

class SexGNN(torch.nn.Module):
    def __init__(self,
                 in_channels,
                 hidden_channels,
                 meta_dim,
                 num_layers=3,
                 jk_mode='lstm',
                 dropout=0.3,
                 edge_dropout_p=0.2,    # thêm tham số này
                ):
        super().__init__()
        self.edge_dropout_p = edge_dropout_p

        self.convs = torch.nn.ModuleList()
        self.bns   = torch.nn.ModuleList()
        for i in range(num_layers):
            in_c = in_channels if i == 0 else hidden_channels
            self.convs.append(GATConv(in_c, hidden_channels, heads=4, concat=False))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))

        self.jk = JumpingKnowledge(mode=jk_mode,
                                   channels=hidden_channels,
                                   num_layers=num_layers)
        jk_dim = hidden_channels * num_layers if jk_mode=='cat' else hidden_channels

        self.shared  = torch.nn.Linear(jk_dim + meta_dim, 64)
        self.head    = torch.nn.Linear(64, 1)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, data, meta_feats):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

        # --- DropEdge (với weights) bằng dropout_adj ---
        if self.training and self.edge_dropout_p > 0:
            # edge_attr shape = [num_edges, 1] -> squeeze thành [num_edges]
            edge_weight = edge_attr.view(-1)
            edge_index, edge_weight = dropout_adj(
                edge_index,
                edge_weight,
                p=self.edge_dropout_p,
                force_undirected=True,
            )
            # dựng lại edge_attr thành [num_edges, 1]
            edge_attr = edge_weight.unsqueeze(-1)

        xs = []
        for conv, bn in zip(self.convs, self.bns):
            # truyền cả edge_attr vào GATConv
            x = F.elu(bn(conv(x, edge_index, edge_attr)))
            xs.append(x)

        x_jk   = self.jk(xs)
        x_pool = global_mean_pool(x_jk, data.batch)
        z      = torch.cat([x_pool, meta_feats], dim=1)
        z      = F.relu(self.shared(z))
        z      = self.dropout(z)
        return self.head(z).squeeze(-1)

# Focal loss for Sex head
def focal_loss(logits, targets, alpha=1, gamma=4):
    prob = torch.sigmoid(logits)
    ce = F.binary_cross_entropy_with_logits(logits, targets, reduction='none')
    p_t = prob * targets + (1 - prob) * (1 - targets)
    return (alpha * (1 - p_t)**gamma * ce).mean()

labels = y_train['Sex_F'].values.astype(int)
class_counts = np.bincount(labels)
pos_weight = torch.tensor(class_counts[0] / class_counts[1], device=device)

# BCEWithLogitsLoss nhưng giữ reduction='none' để sau đó áp focal term
bce_fn = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight, reduction='none')

def weighted_focal_loss(logits, targets, alpha=1.0, gamma=4.0):
    """
    logits: tensor 1D
    targets: cùng shape, giá trị 0/1
    alpha: trọng số cân bằng positive class
    gamma: tập trung vào hard example
    """
    # 1) tính BCE rời rạc cho mỗi sample
    bce_loss = bce_fn(logits, targets)

    # 2) xác suất positive
    p = torch.sigmoid(logits)
    # p_t = p       nếu target=1
    #     = 1 - p   nếu target=0
    p_t = p * targets + (1 - p) * (1 - targets)

    # 3) nhân thêm bản trọng focal
    focal_term = (1 - p_t) ** gamma

    # 4) kết hợp alpha (nếu muốn ưu tiên hơn nữa positive)
    loss = alpha * focal_term * bce_loss

    # 5) trả về mean
    return loss.mean()

# Threshold optimization
def optimize_threshold(y_true, y_prob):
    """Find the best threshold maximizing F1 score."""
    best_t, best_f1 = 0.5, 0
    # Use numpy float thresholds for compatibility
    for t in np.linspace(0.2, 0.8, 31):
        preds = (y_prob >= t).astype(int)
        f = f1_score(y_true, preds)
        if f > best_f1:
            best_f1, best_t = f, t
    return best_t

# Training and validation routines
def train_epoch(loader, model, optimizer, task='sex'):
    model.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)
        idxs = batch.idx.cpu().numpy()
        meta_feats = torch.tensor(
            np.hstack([X_meta_trans[idxs], X_enc[idxs], conn_pca[idxs]]),
            device=device, dtype=torch.float
        )
        logits = model(batch, meta_feats)
        y_true = batch.y.view(-1,2)[:,1]
        loss = weighted_focal_loss(logits, y_true, alpha=pos_weight.item(), gamma=2.0)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * y_true.size(0)
    return total_loss / len(loader.dataset)


def validate_epoch(loader, model, task='sex'):
    model.eval()
    probs, targs = [], []
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            idxs = batch.idx.cpu().numpy()
            meta_feats = torch.tensor(
                np.hstack([X_meta_trans[idxs], X_enc[idxs], conn_pca[idxs]]),
                device=device, dtype=torch.float
            )
            logits = model(batch, meta_feats)
            prob = torch.sigmoid(logits).cpu().numpy()
            true = batch.y.view(-1,2)[:,1].cpu().numpy()
            probs.extend(prob)
            targs.extend(true)
    probs = np.array(probs)
    targs = np.array(targs)
    thr = optimize_threshold(targs, probs)
    f1 = f1_score(targs, (probs >= thr).astype(int))
    class_rp = classification_report(targs, (probs >= thr).astype(int))
    return f1, thr, class_rp

# Cross-validation training
meta_dim = X_meta_trans.shape[1] + X_enc.shape[1] + conn_pca.shape[1]

for task in ['sex']:
    print(f"\n=== Training for {task.upper()} task ===")
    oof_probs = np.zeros(len(data_list))
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    labels = y_train['Sex_F'].values

    results = {}
    
    for fold, (tr_idx, va_idx) in enumerate(skf.split(X_meta_trans, labels)):
        train_subset = [data_list[i] for i in tr_idx]
        val_subset   = [data_list[i] for i in va_idx]

        # Sample weighting only for Sex task
        if task=='sex':
            class_counts = np.bincount(labels[tr_idx].astype(int))
            male_w, female_w = 5.0/class_counts[0], 5.0/class_counts[1]
            weights = np.where(labels[tr_idx]==1, female_w, male_w)
            sampler = WeightedRandomSampler(weights, len(tr_idx), replacement=True)
            train_loader = DataLoader(train_subset, batch_size=32, sampler=sampler)
        val_loader = DataLoader(val_subset, batch_size=32)
        
        # Initialize model
        ModelClass = SexGNN
        model = ModelClass(in_channels=1, hidden_channels=64, meta_dim=meta_dim, num_layers=3, jk_mode='lstm',dropout=0.3, edge_dropout_p=0.2).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3,weight_decay=1e-4)
        
        best_f1, best_t, best_epoch = 0, 0.5, 0
        best_f1_list = []
        # Train
        for epoch in range(1, 21):
            train_loss = train_epoch(train_loader, model, optimizer, task=task)
            f1, thr, class_rp = validate_epoch(val_loader, model, task=task)
            print(f"[Fold {fold}][{task.upper()}][Epoch {epoch:02d}] "
                  f"Loss: {train_loss:.4f} | F1: {f1:.4f}@{thr:.2f} \n {class_rp}")
            
            if f1>best_f1:
                best_f1, best_t, best_epoch = f1, thr, epoch
                torch.save(model.state_dict(), f"best_sex_fold{fold}{best_epoch}.pt")
        results[f"fold{fold}"] = {
            "f1": best_f1,
            "threshold": best_t,
            "model": f"best_sex_fold{fold}{best_epoch}.pt"
        }
    with open("best_sex_models.json","w") as f:
        json.dump(results, f, indent=2)
    print("CV done ==> ", results)

    best_fold = max(results.items(), key=lambda x: x[1]["f1"])[0]
    best_info = results[best_fold]
    print("Best fold:", best_fold, best_info)
    
#Test
X_meta_test = preprocessor.transform(
    pd.concat([test_num.set_index("participant_id"),
               test_cat.set_index("participant_id")[low_card]], axis=1)
)

conn_pca_test = pca.transform(test_conn.drop("participant_id",axis=1).values)

# === 7. LOAD BEST MODEL & INFERENCE ON TEST ===
model = SexGNN(in_channels=1, hidden_channels=64, meta_dim=meta_dim, num_layers=3, jk_mode='lstm',dropout=0.3).to(device)
model.load_state_dict(torch.load(best_info["model"]))
model.eval()

test_loader = DataLoader(test_list, batch_size=32, shuffle=False)
all_probs = []
with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        idxs  = batch.idx.cpu().numpy()
        meta  = torch.tensor(
            np.hstack([X_meta_test[idxs], X_enc_test[idxs], conn_pca_test[idxs]]),
            device=device, dtype=torch.float
        )
        logits= model(batch, meta)
        all_probs.append(torch.sigmoid(logits).cpu().numpy())
all_probs = np.concatenate(all_probs)

# Predict on test
# ... similarly build test meta, enc, graph -> DataLoader -> inference

sample_sub["Sex_F"]       = (all_probs >= 0.25).astype(int)
sample_sub.to_csv('submission.csv', index=False)


=== Training for SEX task ===


/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)


[Fold 0][SEX][Epoch 01] Loss: 0.4053 | F1: 0.5467@0.42 
               precision    recall  f1-score   support

         0.0       0.81      0.16      0.27       155
         1.0       0.39      0.93      0.55        88

    accuracy                           0.44       243
   macro avg       0.60      0.55      0.41       243
weighted avg       0.65      0.44      0.37       243



/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)


[Fold 0][SEX][Epoch 02] Loss: 0.2840 | F1: 0.5639@0.46 
               precision    recall  f1-score   support

         0.0       0.77      0.52      0.62       155
         1.0       0.46      0.73      0.56        88

    accuracy                           0.59       243
   macro avg       0.61      0.62      0.59       243
weighted avg       0.66      0.59      0.60       243



...

/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)


[Fold 4][SEX][Epoch 19] Loss: 0.0108 | F1: 0.5655@0.46 
               precision    recall  f1-score   support

         0.0       0.84      0.79      0.81       174
         1.0       0.53      0.60      0.57        68

    accuracy                           0.74       242
   macro avg       0.68      0.70      0.69       242
weighted avg       0.75      0.74      0.74       242



/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)


[Fold 4][SEX][Epoch 20] Loss: 0.0106 | F1: 0.5638@0.46 
               precision    recall  f1-score   support

         0.0       0.84      0.78      0.81       174
         1.0       0.52      0.62      0.56        68

    accuracy                           0.73       242
   macro avg       0.68      0.70      0.68       242
weighted avg       0.75      0.73      0.74       242

CV done ==>  {'fold0': {'f1': 0.6421052631578947, 'threshold': 0.4600000000000001, 'model': 'best_sex_fold020.pt'}, 'fold1': {'f1': 0.6763285024154589, 'threshold': 0.34, 'model': 'best_sex_fold113.pt'}, 'fold2': {'f1': 0.6739130434782609, 'threshold': 0.38000000000000006, 'model': 'best_sex_fold28.pt'}, 'fold3': {'f1': 0.6666666666666666, 'threshold': 0.32000000000000006, 'model': 'best_sex_fold317.pt'}, 'fold4': {'f1': 0.5875706214689266, 'threshold': 0.38000000000000006, 'model': 'best_sex_fold48.pt'}}
Best fold: fold1 {'f1': 0.6763285024154589, 'threshold': 0.34, 'model': 'best_sex_fold113.pt'}


/tmp/ipykernel_19/627033808.py:349: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_info["model"]))


In [7]:
# Deneme1 = ['EHQ_EHQ_Total', 'ColorVision_CV_Score', 'APQ_P_APQ_P_CP', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD', 'APQ_P_APQ_P_PP', 'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial', 'MRI_Track_Age_at_Scan', '0throw_5thcolumn', '0throw_20thcolumn', '0throw_61thcolumn', '0throw_70thcolumn', '0throw_86thcolumn', '0throw_95thcolumn', '0throw_102thcolumn', '0throw_104thcolumn', '0throw_105thcolumn', '0throw_113thcolumn', '0throw_132thcolumn', '0throw_134thcolumn', '0throw_142thcolumn', '0throw_161thcolumn', '1throw_25thcolumn', '1throw_26thcolumn', '1throw_52thcolumn', '1throw_53thcolumn', '1throw_56thcolumn', '1throw_61thcolumn', '1throw_73thcolumn', '1throw_78thcolumn', '1throw_101thcolumn', '1throw_117thcolumn', '1throw_119thcolumn', '1throw_161thcolumn', '1throw_182thcolumn', '2throw_49thcolumn', '2throw_56thcolumn', '2throw_60thcolumn', '2throw_95thcolumn', '2throw_126thcolumn', '2throw_175thcolumn', '3throw_9thcolumn', '3throw_11thcolumn', '3throw_50thcolumn', '3throw_121thcolumn', '3throw_139thcolumn', '4throw_78thcolumn', '4throw_79thcolumn', '4throw_80thcolumn', '4throw_83thcolumn', '4throw_91thcolumn', '4throw_135thcolumn', '4throw_185thcolumn', '5throw_7thcolumn', '5throw_30thcolumn', '5throw_33thcolumn', '5throw_96thcolumn', '5throw_99thcolumn', '5throw_131thcolumn', '6throw_30thcolumn', '6throw_138thcolumn', '7throw_33thcolumn', '7throw_39thcolumn', '7throw_57thcolumn', '7throw_74thcolumn', '7throw_75thcolumn', '7throw_106thcolumn', '7throw_107thcolumn', '7throw_141thcolumn', '7throw_157thcolumn', '7throw_167thcolumn', '8throw_51thcolumn', '8throw_124thcolumn', '9throw_63thcolumn', '9throw_68thcolumn', '9throw_73thcolumn', '9throw_106thcolumn', '9throw_107thcolumn', '9throw_108thcolumn', '9throw_141thcolumn', '9throw_151thcolumn', '10throw_12thcolumn', '10throw_86thcolumn', '10throw_108thcolumn', '10throw_135thcolumn', '10throw_143thcolumn', '10throw_177thcolumn', '11throw_93thcolumn', '11throw_105thcolumn', '11throw_110thcolumn', '11throw_111thcolumn', '11throw_180thcolumn', '11throw_184thcolumn', '11throw_193thcolumn', '12throw_47thcolumn', '12throw_95thcolumn', '12throw_121thcolumn', '12throw_192thcolumn', '13throw_20thcolumn', '13throw_54thcolumn', '14throw_113thcolumn', '14throw_123thcolumn', '14throw_157thcolumn', '15throw_26thcolumn', '15throw_72thcolumn', '15throw_86thcolumn', '15throw_114thcolumn', '16throw_37thcolumn', '17throw_63thcolumn', '17throw_96thcolumn', '17throw_123thcolumn', '17throw_125thcolumn', '17throw_160thcolumn', '17throw_199thcolumn', '18throw_21thcolumn', '18throw_61thcolumn', '18throw_81thcolumn', '18throw_85thcolumn', '18throw_188thcolumn', '18throw_195thcolumn', '19throw_23thcolumn', '19throw_48thcolumn', '19throw_69thcolumn', '19throw_86thcolumn', '19throw_116thcolumn', '19throw_127thcolumn', '19throw_129thcolumn', '19throw_134thcolumn', '19throw_145thcolumn', '20throw_70thcolumn', '20throw_79thcolumn', '20throw_83thcolumn', '20throw_93thcolumn', '20throw_96thcolumn', '20throw_125thcolumn', '20throw_126thcolumn', '20throw_159thcolumn', '20throw_196thcolumn', '21throw_47thcolumn', '21throw_51thcolumn', '21throw_61thcolumn', '21throw_94thcolumn', '21throw_125thcolumn', '21throw_178thcolumn', '22throw_40thcolumn', '22throw_60thcolumn', '22throw_75thcolumn', '22throw_92thcolumn', '22throw_110thcolumn', '22throw_141thcolumn', '22throw_158thcolumn', '22throw_162thcolumn', '23throw_42thcolumn', '23throw_87thcolumn', '23throw_88thcolumn', '23throw_89thcolumn', '23throw_92thcolumn', '23throw_112thcolumn', '23throw_123thcolumn', '24throw_36thcolumn', '24throw_72thcolumn', '24throw_124thcolumn', '24throw_127thcolumn', '24throw_181thcolumn', '24throw_198thcolumn', '24throw_199thcolumn', '25throw_64thcolumn', '25throw_70thcolumn', '25throw_78thcolumn', '25throw_89thcolumn', '25throw_123thcolumn', '25throw_127thcolumn', '26throw_35thcolumn', '26throw_69thcolumn', '26throw_92thcolumn', '26throw_116thcolumn', '26throw_171thcolumn', '26throw_190thcolumn', '27throw_68thcolumn', '27throw_97thcolumn', '27throw_114thcolumn', '27throw_158thcolumn', '27throw_174thcolumn', '28throw_56thcolumn', '28throw_80thcolumn', '28throw_83thcolumn', '28throw_92thcolumn', '28throw_132thcolumn', '28throw_190thcolumn', '28throw_194thcolumn', '28throw_199thcolumn', '29throw_57thcolumn', '29throw_73thcolumn', '29throw_115thcolumn', '29throw_118thcolumn', '29throw_134thcolumn', '30throw_48thcolumn', '30throw_54thcolumn', '30throw_67thcolumn', '30throw_88thcolumn', '30throw_124thcolumn', '30throw_130thcolumn', '30throw_170thcolumn', '30throw_188thcolumn', '31throw_65thcolumn', '31throw_112thcolumn', '31throw_132thcolumn', '32throw_49thcolumn', '32throw_60thcolumn', '32throw_94thcolumn', '32throw_165thcolumn', '33throw_47thcolumn', '33throw_53thcolumn', '33throw_135thcolumn', '33throw_169thcolumn', '34throw_68thcolumn', '34throw_73thcolumn', '34throw_77thcolumn', '34throw_125thcolumn', '34throw_140thcolumn', '34throw_174thcolumn', '35throw_96thcolumn', '35throw_118thcolumn', '35throw_125thcolumn', '36throw_55thcolumn', '36throw_59thcolumn', '36throw_163thcolumn', '37throw_61thcolumn', '37throw_108thcolumn', '37throw_163thcolumn', '37throw_190thcolumn', '38throw_48thcolumn', '38throw_68thcolumn', '38throw_82thcolumn', '38throw_85thcolumn', '38throw_153thcolumn', '38throw_156thcolumn', '38throw_169thcolumn', '39throw_73thcolumn', '39throw_116thcolumn', '39throw_123thcolumn', '39throw_134thcolumn', '39throw_136thcolumn', '39throw_182thcolumn', '40throw_46thcolumn', '40throw_63thcolumn', '40throw_124thcolumn', '40throw_149thcolumn', '40throw_172thcolumn', '40throw_188thcolumn', '41throw_50thcolumn', '41throw_62thcolumn', '41throw_75thcolumn', '41throw_77thcolumn', '41throw_97thcolumn', '41throw_112thcolumn', '41throw_186thcolumn', '42throw_85thcolumn', '42throw_102thcolumn', '42throw_152thcolumn', '42throw_181thcolumn', '43throw_44thcolumn', '43throw_50thcolumn', '43throw_99thcolumn', '43throw_142thcolumn', '44throw_50thcolumn', '44throw_54thcolumn', '44throw_87thcolumn', '44throw_93thcolumn', '44throw_130thcolumn', '44throw_132thcolumn', '44throw_150thcolumn', '44throw_157thcolumn', '44throw_162thcolumn', '44throw_165thcolumn', '44throw_184thcolumn', '45throw_70thcolumn', '45throw_148thcolumn', '45throw_150thcolumn', '45throw_155thcolumn', '45throw_173thcolumn', '46throw_182thcolumn', '46throw_184thcolumn', '47throw_80thcolumn', '47throw_94thcolumn', '47throw_118thcolumn', '47throw_160thcolumn', '48throw_123thcolumn', '48throw_163thcolumn', '48throw_166thcolumn', '48throw_185thcolumn', '49throw_56thcolumn', '49throw_133thcolumn', '49throw_153thcolumn', '49throw_184thcolumn', '49throw_191thcolumn', '50throw_53thcolumn', '50throw_66thcolumn', '50throw_72thcolumn', '50throw_129thcolumn', '50throw_157thcolumn', '50throw_158thcolumn', '50throw_187thcolumn', '51throw_61thcolumn', '51throw_80thcolumn', '51throw_92thcolumn', '51throw_123thcolumn', '51throw_177thcolumn', '52throw_61thcolumn', '52throw_64thcolumn', '52throw_106thcolumn', '52throw_179thcolumn', '52throw_182thcolumn', '53throw_70thcolumn', '53throw_75thcolumn', '53throw_86thcolumn', '53throw_89thcolumn', '53throw_94thcolumn', '53throw_98thcolumn', '53throw_103thcolumn', '53throw_131thcolumn', '53throw_191thcolumn', '54throw_57thcolumn', '54throw_60thcolumn', '54throw_69thcolumn', '54throw_80thcolumn', '54throw_87thcolumn', '54throw_97thcolumn', '54throw_162thcolumn', '54throw_182thcolumn', '55throw_86thcolumn', '55throw_100thcolumn', '55throw_135thcolumn', '55throw_154thcolumn', '55throw_156thcolumn', '55throw_159thcolumn', '56throw_62thcolumn', '56throw_72thcolumn', '56throw_79thcolumn', '56throw_140thcolumn', '56throw_146thcolumn', '56throw_148thcolumn', '56throw_168thcolumn', '56throw_188thcolumn', '57throw_90thcolumn', '57throw_96thcolumn', '57throw_97thcolumn', '57throw_140thcolumn', '57throw_146thcolumn', '57throw_151thcolumn', '57throw_167thcolumn', '57throw_173thcolumn', '57throw_179thcolumn', '57throw_197thcolumn', '58throw_68thcolumn', '58throw_73thcolumn', '58throw_106thcolumn', '58throw_137thcolumn', '58throw_181thcolumn', '59throw_88thcolumn', '59throw_91thcolumn', '59throw_168thcolumn', '59throw_196thcolumn', '60throw_93thcolumn', '60throw_100thcolumn', '61throw_79thcolumn', '61throw_118thcolumn', '61throw_123thcolumn', '61throw_124thcolumn', '61throw_134thcolumn', '61throw_173thcolumn', '62throw_104thcolumn', '62throw_123thcolumn', '62throw_128thcolumn', '62throw_132thcolumn', '62throw_135thcolumn', '62throw_164thcolumn', '63throw_93thcolumn', '63throw_174thcolumn', '63throw_188thcolumn', '63throw_194thcolumn', '64throw_81thcolumn', '64throw_144thcolumn', '64throw_150thcolumn', '64throw_151thcolumn', '64throw_190thcolumn', '65throw_135thcolumn', '65throw_170thcolumn', '65throw_178thcolumn', '65throw_189thcolumn', '65throw_192thcolumn', '66throw_85thcolumn', '66throw_131thcolumn', '66throw_144thcolumn', '66throw_157thcolumn', '67throw_69thcolumn', '67throw_70thcolumn', '67throw_89thcolumn', '67throw_116thcolumn', '67throw_131thcolumn', '67throw_149thcolumn', '67throw_168thcolumn', '68throw_78thcolumn', '68throw_96thcolumn', '68throw_114thcolumn', '68throw_183thcolumn', '68throw_192thcolumn', '69throw_111thcolumn', '69throw_137thcolumn', '69throw_158thcolumn', '69throw_165thcolumn', '69throw_185thcolumn', '69throw_191thcolumn', '70throw_79thcolumn', '70throw_97thcolumn', '70throw_99thcolumn', '70throw_114thcolumn', '70throw_123thcolumn', '70throw_125thcolumn', '70throw_126thcolumn', '70throw_144thcolumn', '70throw_161thcolumn', '70throw_170thcolumn', '70throw_180thcolumn', '70throw_183thcolumn', '71throw_78thcolumn', '71throw_99thcolumn', '71throw_171thcolumn', '72throw_78thcolumn', '72throw_174thcolumn', '72throw_177thcolumn', '73throw_82thcolumn', '73throw_86thcolumn', '73throw_182thcolumn', '74throw_83thcolumn', '74throw_120thcolumn', '74throw_140thcolumn', '75throw_88thcolumn', '75throw_97thcolumn', '75throw_120thcolumn', '75throw_125thcolumn', '75throw_129thcolumn', '75throw_160thcolumn', '76throw_123thcolumn', '76throw_156thcolumn', '76throw_185thcolumn', '77throw_99thcolumn', '77throw_105thcolumn', '77throw_106thcolumn', '77throw_145thcolumn', '77throw_151thcolumn', '77throw_170thcolumn', '77throw_198thcolumn', '78throw_133thcolumn', '78throw_163thcolumn', '78throw_167thcolumn', '78throw_195thcolumn', '79throw_89thcolumn', '79throw_150thcolumn', '79throw_155thcolumn', '79throw_166thcolumn', '79throw_176thcolumn', '80throw_92thcolumn', '80throw_125thcolumn', '80throw_160thcolumn', '80throw_161thcolumn', '81throw_101thcolumn', '81throw_115thcolumn', '81throw_129thcolumn', '81throw_133thcolumn', '81throw_154thcolumn', '81throw_163thcolumn', '82throw_124thcolumn', '82throw_191thcolumn', '83throw_90thcolumn', '83throw_177thcolumn', '84throw_87thcolumn', '84throw_137thcolumn', '84throw_141thcolumn', '84throw_146thcolumn', '84throw_147thcolumn', '84throw_194thcolumn', '85throw_89thcolumn', '85throw_111thcolumn', '85throw_117thcolumn', '85throw_192thcolumn', '85throw_196thcolumn', '85throw_197thcolumn', '86throw_90thcolumn', '86throw_153thcolumn', '86throw_168thcolumn', '86throw_183thcolumn', '86throw_198thcolumn', '87throw_122thcolumn', '87throw_139thcolumn', '88throw_114thcolumn', '88throw_163thcolumn', '88throw_165thcolumn', '89throw_96thcolumn', '89throw_106thcolumn', '89throw_115thcolumn', '89throw_179thcolumn', '89throw_191thcolumn', '90throw_180thcolumn', '91throw_93thcolumn', '91throw_105thcolumn', '91throw_112thcolumn', '91throw_133thcolumn', '91throw_134thcolumn', '92throw_135thcolumn', '92throw_159thcolumn', '92throw_175thcolumn', '92throw_189thcolumn', '93throw_96thcolumn', '93throw_100thcolumn', '93throw_120thcolumn', '93throw_135thcolumn', '93throw_172thcolumn', '93throw_173thcolumn', '93throw_174thcolumn', '94throw_107thcolumn', '94throw_166thcolumn', '95throw_102thcolumn', '95throw_155thcolumn', '96throw_104thcolumn', '96throw_106thcolumn', '96throw_112thcolumn', '96throw_138thcolumn', '96throw_184thcolumn', '96throw_194thcolumn', '97throw_100thcolumn', '97throw_143thcolumn', '97throw_154thcolumn', '98throw_167thcolumn', '98throw_183thcolumn', '99throw_147thcolumn', '99throw_162thcolumn', '100throw_135thcolumn', '100throw_171thcolumn', '101throw_152thcolumn', '101throw_196thcolumn', '102throw_111thcolumn', '102throw_120thcolumn', '102throw_132thcolumn', '102throw_177thcolumn', '102throw_194thcolumn', '104throw_130thcolumn', '104throw_178thcolumn', '106throw_110thcolumn', '106throw_192thcolumn', '107throw_126thcolumn', '108throw_110thcolumn', '108throw_114thcolumn', '108throw_119thcolumn', '108throw_141thcolumn', '108throw_151thcolumn', '108throw_155thcolumn', '108throw_181thcolumn', '109throw_157thcolumn', '109throw_177thcolumn', '109throw_179thcolumn', '110throw_159thcolumn', '110throw_166thcolumn', '110throw_177thcolumn', '110throw_185thcolumn', '110throw_186thcolumn', '111throw_121thcolumn', '111throw_135thcolumn', '111throw_137thcolumn', '111throw_151thcolumn', '112throw_116thcolumn', '112throw_118thcolumn', '113throw_142thcolumn', '113throw_189thcolumn', '113throw_199thcolumn', '114throw_148thcolumn', '114throw_176thcolumn', '114throw_182thcolumn', '114throw_184thcolumn', '114throw_188thcolumn', '114throw_192thcolumn', '115throw_171thcolumn', '115throw_173thcolumn', '115throw_180thcolumn', '116throw_118thcolumn', '116throw_136thcolumn', '116throw_137thcolumn', '117throw_130thcolumn', '117throw_135thcolumn', '118throw_162thcolumn', '119throw_173thcolumn', '120throw_129thcolumn', '120throw_138thcolumn', '120throw_179thcolumn', '120throw_194thcolumn', '121throw_122thcolumn', '122throw_176thcolumn', '122throw_189thcolumn', '122throw_199thcolumn', '123throw_127thcolumn', '123throw_165thcolumn', '123throw_175thcolumn', '123throw_188thcolumn', '123throw_190thcolumn', '124throw_158thcolumn', '124throw_175thcolumn', '125throw_157thcolumn', '125throw_159thcolumn', '125throw_193thcolumn', '125throw_197thcolumn', '126throw_153thcolumn', '126throw_175thcolumn', '127throw_135thcolumn', '127throw_141thcolumn', '127throw_158thcolumn', '127throw_159thcolumn', '127throw_172thcolumn', '127throw_184thcolumn', '127throw_191thcolumn', '128throw_178thcolumn', '129throw_168thcolumn', '129throw_188thcolumn', '129throw_190thcolumn', '130throw_140thcolumn', '130throw_144thcolumn', '130throw_155thcolumn', '130throw_174thcolumn', '130throw_183thcolumn', '130throw_187thcolumn', '131throw_148thcolumn', '133throw_151thcolumn', '135throw_141thcolumn', '135throw_183thcolumn', '136throw_169thcolumn', '137throw_150thcolumn', '137throw_169thcolumn', '137throw_173thcolumn', '137throw_177thcolumn', '138throw_148thcolumn', '138throw_171thcolumn', '138throw_194thcolumn', '139throw_177thcolumn', '140throw_153thcolumn', '140throw_163thcolumn', '140throw_176thcolumn', '141throw_145thcolumn', '143throw_170thcolumn', '144throw_156thcolumn', '144throw_176thcolumn', '144throw_194thcolumn', '144throw_199thcolumn', '145throw_167thcolumn', '146throw_165thcolumn', '146throw_171thcolumn', '147throw_193thcolumn', '148throw_160thcolumn', '148throw_166thcolumn', '149throw_163thcolumn', '149throw_165thcolumn', '149throw_173thcolumn', '149throw_178thcolumn', '149throw_192thcolumn', '150throw_170thcolumn', '150throw_173thcolumn', '151throw_164thcolumn', '151throw_173thcolumn', '152throw_160thcolumn', '152throw_164thcolumn', '154throw_156thcolumn', '154throw_166thcolumn', '155throw_170thcolumn', '155throw_185thcolumn', '155throw_196thcolumn', '156throw_183thcolumn', '157throw_159thcolumn', '158throw_187thcolumn', '159throw_160thcolumn', '159throw_190thcolumn', '160throw_171thcolumn', '160throw_195thcolumn', '161throw_174thcolumn', '161throw_184thcolumn', '161throw_195thcolumn', '162throw_194thcolumn', '163throw_166thcolumn', '163throw_178thcolumn', '163throw_181thcolumn', '164throw_179thcolumn', '165throw_173thcolumn', '165throw_180thcolumn', '165throw_186thcolumn', '166throw_174thcolumn', '166throw_194thcolumn', '169throw_170thcolumn', '169throw_184thcolumn', '170throw_173thcolumn', '173throw_177thcolumn', '174throw_190thcolumn', '174throw_192thcolumn', '174throw_197thcolumn', '175throw_180thcolumn', '176throw_197thcolumn', '177throw_180thcolumn', '180throw_193thcolumn', '181throw_184thcolumn', '181throw_193thcolumn', '181throw_195thcolumn', '182throw_192thcolumn', '184throw_193thcolumn', '186throw_187thcolumn', '190throw_191thcolumn', '194throw_196thcolumn', '197throw_199thcolumn']

# Deneme2=['Basic_Demos_Enroll_Year', '0throw_9thcolumn', '0throw_109thcolumn', '2throw_16thcolumn', '2throw_22thcolumn', '5throw_128thcolumn', '5throw_145thcolumn', '8throw_80thcolumn', '10throw_175thcolumn', '11throw_135thcolumn', '14throw_190thcolumn', '16throw_51thcolumn', '25throw_125thcolumn', '27throw_95thcolumn', '28throw_46thcolumn', '39throw_111thcolumn', '41throw_120thcolumn', '43throw_126thcolumn', '53throw_162thcolumn', '59throw_112thcolumn', '60throw_117thcolumn', '64throw_75thcolumn', '65throw_199thcolumn', '69throw_79thcolumn', '69throw_173thcolumn', '69throw_186thcolumn', '73throw_169thcolumn', '78throw_168thcolumn', '91throw_135thcolumn', '92throw_160thcolumn', '95throw_116thcolumn', '108throw_109thcolumn', '110throw_167thcolumn', '116throw_168thcolumn', '123throw_168thcolumn', '123throw_192thcolumn', '129throw_157thcolumn', '130throw_153thcolumn', '138throw_184thcolumn', '149throw_197thcolumn', '153throw_189thcolumn']

# Deneme3=['Basic_Demos_Study_Site', '35throw_186thcolumn', '132throw_162thcolumn', '106throw_124thcolumn', '5throw_156thcolumn', '26throw_38thcolumn', '37throw_71thcolumn', '55throw_182thcolumn', '73throw_138thcolumn', '3throw_38thcolumn', '32throw_34thcolumn', '0throw_136thcolumn', '26throw_132thcolumn', '73throw_76thcolumn', '39throw_90thcolumn', '84throw_177thcolumn', '128throw_189thcolumn', '187throw_188thcolumn', '67throw_81thcolumn', '107throw_121thcolumn', '86throw_147thcolumn', '31throw_60thcolumn', '21throw_176thcolumn', '32throw_111thcolumn', '43throw_76thcolumn', '60throw_165thcolumn', '115throw_197thcolumn', '85throw_195thcolumn', '26throw_52thcolumn', '26throw_63thcolumn', '123throw_163thcolumn', '67throw_193thcolumn', '139throw_154thcolumn', '42throw_98thcolumn', '9throw_157thcolumn', '139throw_171thcolumn', '137throw_148thcolumn', '10throw_70thcolumn', '0throw_78thcolumn', '0throw_42thcolumn', '26throw_36thcolumn', '2throw_101thcolumn', '8throw_189thcolumn', '64throw_131thcolumn', '95throw_101thcolumn', '160throw_174thcolumn', '63throw_142thcolumn', '131throw_165thcolumn', '98throw_151thcolumn', '3throw_181thcolumn', '22throw_176thcolumn', '37throw_78thcolumn', '2throw_42thcolumn', '23throw_93thcolumn', '12throw_72thcolumn', '19throw_57thcolumn', '53throw_197thcolumn', '106throw_168thcolumn', '134throw_148thcolumn', '53throw_137thcolumn', '19throw_75thcolumn', '28throw_175thcolumn', '76throw_194thcolumn', '70throw_175thcolumn', '10throw_96thcolumn', '86throw_164thcolumn', '78throw_143thcolumn', '84throw_138thcolumn', 'MRI_Track_Scan_Location', '23throw_59thcolumn', '73throw_141thcolumn', '71throw_111thcolumn', '64throw_160thcolumn', '41throw_130thcolumn', '105throw_192thcolumn', '113throw_127thcolumn', '33throw_89thcolumn', '57throw_191thcolumn', '33throw_183thcolumn', '17throw_51thcolumn', '81throw_159thcolumn', '128throw_169thcolumn', '86throw_96thcolumn', '102throw_152thcolumn', '147throw_187thcolumn', '10throw_36thcolumn', '37throw_173thcolumn', '11throw_173thcolumn', '85throw_188thcolumn', '3throw_167thcolumn', '117throw_197thcolumn', '26throw_87thcolumn', '104throw_194thcolumn', '113throw_156thcolumn', '52throw_152thcolumn', '106throw_150thcolumn', '104throw_121thcolumn', '15throw_38thcolumn', '11throw_170thcolumn', '111throw_153thcolumn', '96throw_172thcolumn', '38throw_133thcolumn', '40throw_119thcolumn', '113throw_169thcolumn', '64throw_114thcolumn', '94throw_189thcolumn', '25throw_60thcolumn', '23throw_195thcolumn', '33throw_40thcolumn', '4throw_124thcolumn', '126throw_152thcolumn', '115throw_122thcolumn', '46throw_80thcolumn', '15throw_104thcolumn', '169throw_182thcolumn', '36throw_195thcolumn', '0throw_41thcolumn', '51throw_152thcolumn', '22throw_72thcolumn', '56throw_155thcolumn', '33throw_70thcolumn', '40throw_169thcolumn', '186throw_189thcolumn', '76throw_143thcolumn', '58throw_103thcolumn', '56throw_137thcolumn', '43throw_120thcolumn', '20throw_110thcolumn', '5throw_162thcolumn', '4throw_156thcolumn', '114throw_117thcolumn', '56throw_133thcolumn', '114throw_177thcolumn', '17throw_161thcolumn', '154throw_164thcolumn', '40throw_180thcolumn', '32throw_63thcolumn', '171throw_196thcolumn', '125throw_127thcolumn', '48throw_57thcolumn', '146throw_153thcolumn', '16throw_57thcolumn', '13throw_161thcolumn', '26throw_94thcolumn', '65throw_119thcolumn', '1throw_31thcolumn', '2throw_146thcolumn', '94throw_170thcolumn', '156throw_180thcolumn', '86throw_190thcolumn', '39throw_171thcolumn', '91throw_156thcolumn', '67throw_143thcolumn', '10throw_27thcolumn', '188throw_198thcolumn', '2throw_66thcolumn', 'SDQ_SDQ_Emotional_Problems', '14throw_29thcolumn', '176throw_190thcolumn', '77throw_120thcolumn', '63throw_152thcolumn', '83throw_167thcolumn', '27throw_37thcolumn', '7throw_83thcolumn', '40throw_167thcolumn', '2throw_136thcolumn', '90throw_184thcolumn', '34throw_44thcolumn', '41throw_196thcolumn', '0throw_117thcolumn', '159throw_188thcolumn', '80throw_190thcolumn', '138throw_173thcolumn', '82throw_123thcolumn', '143throw_157thcolumn', '8throw_147thcolumn', '159throw_187thcolumn', '1throw_194thcolumn', '129throw_162thcolumn', '139throw_197thcolumn', '113throw_150thcolumn', '26throw_76thcolumn', '48throw_89thcolumn', '72throw_163thcolumn', '38throw_158thcolumn', '18throw_78thcolumn', '64throw_111thcolumn', '65throw_130thcolumn', '147throw_167thcolumn', '12throw_16thcolumn', '90throw_159thcolumn', '164throw_177thcolumn', '56throw_152thcolumn', '15throw_60thcolumn', '100throw_195thcolumn', '21throw_147thcolumn', '155throw_192thcolumn', '183throw_192thcolumn', '4throw_8thcolumn', '33throw_69thcolumn', '56throw_61thcolumn', '83throw_123thcolumn', '21throw_22thcolumn', '68throw_98thcolumn', '57throw_165thcolumn', '9throw_98thcolumn', '127throw_185thcolumn', '113throw_131thcolumn', '13throw_169thcolumn', '157throw_171thcolumn', '119throw_143thcolumn', '16throw_34thcolumn', '124throw_154thcolumn', '117throw_192thcolumn', '75throw_151thcolumn', '51throw_156thcolumn', '30throw_96thcolumn', '21throw_107thcolumn', '14throw_156thcolumn', '47throw_171thcolumn', '96throw_188thcolumn', '19throw_46thcolumn', '20throw_119thcolumn', '16throw_130thcolumn', '68throw_160thcolumn', '44throw_167thcolumn', '40throw_105thcolumn', '6throw_26thcolumn', '53throw_161thcolumn', '8throw_185thcolumn', '9throw_187thcolumn', '9throw_105thcolumn', '17throw_156thcolumn', '34throw_123thcolumn', '36throw_108thcolumn', '73throw_148thcolumn', '32throw_62thcolumn', '113throw_115thcolumn', '70throw_110thcolumn', '88throw_160thcolumn', '47throw_146thcolumn', '140throw_175thcolumn', '130throw_154thcolumn', '49throw_181thcolumn', '155throw_181thcolumn', '99throw_172thcolumn', '33throw_93thcolumn', '49throw_194thcolumn', '58throw_128thcolumn', '150throw_166thcolumn', '51throw_83thcolumn', '52throw_68thcolumn', '34throw_150thcolumn', '140throw_145thcolumn', '1throw_22thcolumn', '4throw_192thcolumn', '14throw_102thcolumn', '103throw_121thcolumn', '45throw_162thcolumn', '36throw_181thcolumn', '88throw_185thcolumn', '80throw_97thcolumn', '71throw_72thcolumn', '20throw_183thcolumn', '103throw_128thcolumn', '110throw_165thcolumn', '124throw_192thcolumn', '23throw_47thcolumn', '129throw_196thcolumn', '93throw_197thcolumn', '118throw_154thcolumn', '101throw_153thcolumn', '134throw_161thcolumn', '131throw_174thcolumn', '38throw_145thcolumn', '116throw_192thcolumn', '62throw_167thcolumn', '125throw_176thcolumn', '76throw_89thcolumn', '174throw_181thcolumn', '177throw_196thcolumn', '41throw_124thcolumn', '20throw_160thcolumn', '104throw_137thcolumn', '130throw_147thcolumn', '181throw_192thcolumn', '66throw_156thcolumn', '11throw_124thcolumn', '155throw_175thcolumn', '0throw_197thcolumn', '55throw_160thcolumn', '46throw_149thcolumn', '61throw_107thcolumn', '59throw_191thcolumn', '68throw_154thcolumn', '17throw_67thcolumn', '46throw_143thcolumn', '36throw_193thcolumn', '185throw_191thcolumn', '46throw_58thcolumn', '10throw_154thcolumn', '26throw_85thcolumn', '93throw_185thcolumn', '152throw_161thcolumn', '48throw_73thcolumn', '39throw_174thcolumn', '53throw_157thcolumn', '76throw_96thcolumn', '74throw_82thcolumn', '161throw_175thcolumn', '21throw_136thcolumn', '29throw_84thcolumn', '36throw_151thcolumn', '8throw_196thcolumn', '62throw_122thcolumn', '111throw_177thcolumn', '153throw_168thcolumn', '78throw_155thcolumn', '48throw_139thcolumn', '105throw_137thcolumn', '162throw_180thcolumn', '25throw_124thcolumn', '0throw_145thcolumn', '14throw_58thcolumn', '27throw_34thcolumn', '155throw_193thcolumn', '134throw_177thcolumn', '62throw_142thcolumn', '44throw_173thcolumn', '26throw_162thcolumn', '179throw_193thcolumn', '14throw_140thcolumn', '25throw_91thcolumn', '23throw_189thcolumn', '21throw_90thcolumn', '68throw_152thcolumn', '145throw_172thcolumn', '104throw_152thcolumn', '52throw_193thcolumn', '70throw_149thcolumn', '52throw_75thcolumn', '192throw_193thcolumn', '104throw_166thcolumn', '47throw_67thcolumn', '150throw_175thcolumn', '8throw_45thcolumn', '28throw_95thcolumn', '118throw_192thcolumn', '74throw_110thcolumn', '27throw_182thcolumn', '80throw_163thcolumn', '12throw_83thcolumn', '161throw_177thcolumn', '91throw_197thcolumn', '17throw_108thcolumn', '130throw_190thcolumn', '99throw_198thcolumn', '4throw_19thcolumn', '9throw_180thcolumn', '7throw_187thcolumn', '22throw_89thcolumn', '124throw_194thcolumn', '58throw_170thcolumn', '18throw_167thcolumn', '72throw_153thcolumn', '51throw_144thcolumn', '2throw_184thcolumn', '43throw_141thcolumn', '113throw_182thcolumn', '85throw_164thcolumn', '3throw_112thcolumn', '21throw_84thcolumn', '26throw_157thcolumn', '19throw_74thcolumn', '63throw_82thcolumn', '139throw_160thcolumn', '82throw_197thcolumn', '36throw_180thcolumn', '48throw_58thcolumn', '102throw_113thcolumn', '105throw_173thcolumn']

# Deneme4=['140throw_158thcolumn', '151throw_192thcolumn', '22throw_68thcolumn', '11throw_155thcolumn', '163throw_193thcolumn', '12throw_152thcolumn', 'SDQ_SDQ_Conduct_Problems', '180throw_187thcolumn', '25throw_93thcolumn', '69throw_106thcolumn', '120throw_168thcolumn', '87throw_172thcolumn', '143throw_153thcolumn', '17throw_55thcolumn', '171throw_189thcolumn', '96throw_165thcolumn', '26throw_129thcolumn', '135throw_181thcolumn', '132throw_138thcolumn', '28throw_122thcolumn', '15throw_19thcolumn', '152throw_167thcolumn', '107throw_182thcolumn', '72throw_115thcolumn', '113throw_130thcolumn', '49throw_120thcolumn', '29throw_55thcolumn', '97throw_157thcolumn', '59throw_90thcolumn', '90throw_130thcolumn', '9throw_34thcolumn', '2throw_166thcolumn', '58throw_71thcolumn', '29throw_70thcolumn', '123throw_178thcolumn', '19throw_141thcolumn', '109throw_166thcolumn', '27throw_60thcolumn', '173throw_197thcolumn', '14throw_67thcolumn', '63throw_105thcolumn', '75throw_81thcolumn', '29throw_66thcolumn', '63throw_69thcolumn', '54throw_197thcolumn', '138throw_177thcolumn', '1throw_113thcolumn', '15throw_132thcolumn', '14throw_112thcolumn', '55throw_60thcolumn', '31throw_91thcolumn', '18throw_187thcolumn', '77throw_78thcolumn', '146throw_192thcolumn', '146throw_173thcolumn', '71throw_193thcolumn', '137throw_176thcolumn', '113throw_168thcolumn', '170throw_171thcolumn', '65throw_183thcolumn', '121throw_179thcolumn', '70throw_142thcolumn', '104throw_155thcolumn', '62throw_192thcolumn', '61throw_97thcolumn', '14throw_21thcolumn', '48throw_67thcolumn', '16throw_71thcolumn', '48throw_115thcolumn', '69throw_94thcolumn', '111throw_163thcolumn', '110throw_151thcolumn', '171throw_183thcolumn', '52throw_124thcolumn', '49throw_101thcolumn', '161throw_178thcolumn', '74throw_88thcolumn', '88throw_142thcolumn', '40throw_185thcolumn', '65throw_103thcolumn', '135throw_148thcolumn', '47throw_114thcolumn', '88throw_118thcolumn', '0throw_34thcolumn', '140throw_155thcolumn', '13throw_67thcolumn', '63throw_184thcolumn', '75throw_138thcolumn', '10throw_136thcolumn', '29throw_77thcolumn', '132throw_180thcolumn', 'Barratt_Barratt_P2_Edu', '111throw_143thcolumn', '116throw_173thcolumn', '114throw_174thcolumn', '134throw_194thcolumn', '154throw_195thcolumn', '114throw_126thcolumn', '20throw_127thcolumn', '124throw_193thcolumn', '89throw_130thcolumn', '30throw_94thcolumn', '16throw_56thcolumn', '148throw_162thcolumn', '1throw_165thcolumn', '44throw_190thcolumn', '79throw_110thcolumn', '3throw_146thcolumn', '20throw_24thcolumn', '174throw_196thcolumn', '93throw_129thcolumn', '93throw_118thcolumn', '6throw_73thcolumn', '53throw_125thcolumn', '19throw_33thcolumn', '41throw_60thcolumn', '0throw_186thcolumn', '119throw_167thcolumn', '90throw_167thcolumn', '138throw_144thcolumn', '33throw_143thcolumn', '170throw_193thcolumn', '61throw_144thcolumn', '104throw_149thcolumn', '100throw_142thcolumn', '77throw_82thcolumn', '12throw_27thcolumn', '132throw_161thcolumn', '35throw_116thcolumn', '27throw_56thcolumn', '24throw_168thcolumn', '151throw_181thcolumn', '58throw_138thcolumn', '12throw_55thcolumn', '157throw_175thcolumn', '96throw_144thcolumn', '31throw_62thcolumn', '83throw_130thcolumn', '41throw_90thcolumn', '54throw_159thcolumn', '95throw_192thcolumn', '14throw_82thcolumn', '137throw_186thcolumn', '0throw_91thcolumn', 'Barratt_Barratt_P2_Occ', '11throw_168thcolumn', '69throw_99thcolumn', '11throw_37thcolumn', '5throw_38thcolumn', '28throw_111thcolumn', '92throw_129thcolumn', '17throw_163thcolumn', '161throw_167thcolumn', '22throw_175thcolumn', '20throw_55thcolumn', '61throw_114thcolumn', '155throw_179thcolumn', '144throw_152thcolumn', '51throw_151thcolumn', '81throw_132thcolumn', '22throw_61thcolumn', '101throw_187thcolumn', '66throw_178thcolumn', '16throw_120thcolumn', '60throw_119thcolumn', '117throw_121thcolumn', '61throw_84thcolumn', '57throw_60thcolumn', '89throw_99thcolumn', '170throw_178thcolumn', '36throw_71thcolumn', '88throw_134thcolumn', '136throw_150thcolumn', '70throw_164thcolumn', '139throw_179thcolumn', '51throw_166thcolumn', '28throw_137thcolumn', '128throw_160thcolumn', '30throw_142thcolumn', '7throw_38thcolumn', '22throw_70thcolumn', '54throw_167thcolumn', '24throw_178thcolumn', '17throw_85thcolumn', '87throw_114thcolumn', '5throw_107thcolumn', '57throw_150thcolumn', '19throw_59thcolumn', '84throw_89thcolumn', '24throw_167thcolumn', '61throw_178thcolumn', '8throw_10thcolumn', '6throw_147thcolumn', '124throw_178thcolumn', '70throw_194thcolumn', '143throw_146thcolumn', '2throw_71thcolumn', '114throw_152thcolumn', '51throw_110thcolumn', '99throw_159thcolumn', '70throw_163thcolumn', '70throw_122thcolumn', '37throw_120thcolumn', '57throw_59thcolumn', '174throw_182thcolumn', '29throw_58thcolumn', '32throw_143thcolumn', '86throw_102thcolumn', '28throw_183thcolumn', '2throw_193thcolumn', '64throw_104thcolumn', '27throw_110thcolumn', '87throw_97thcolumn', '106throw_161thcolumn', '35throw_165thcolumn', '63throw_107thcolumn', '45throw_192thcolumn', '184throw_194thcolumn', '76throw_77thcolumn', '48throw_72thcolumn', '2throw_96thcolumn', '21throw_185thcolumn', '121throw_166thcolumn', '89throw_192thcolumn', '129throw_194thcolumn', '24throw_187thcolumn', '38throw_118thcolumn', '10throw_151thcolumn', '29throw_54thcolumn', '24throw_143thcolumn', '53throw_155thcolumn', '6throw_35thcolumn', '157throw_178thcolumn', '111throw_139thcolumn', '122throw_128thcolumn', '83throw_107thcolumn', '128throw_196thcolumn', '159throw_165thcolumn', '13throw_118thcolumn', '111throw_183thcolumn', '109throw_192thcolumn', '65throw_165thcolumn', '57throw_152thcolumn', '58throw_60thcolumn', '47throw_172thcolumn', '83throw_137thcolumn', '0throw_31thcolumn', '51throw_178thcolumn', '43throw_119thcolumn', '88throw_187thcolumn', '12throw_50thcolumn', '10throw_43thcolumn', '53throw_186thcolumn', '36throw_167thcolumn', '2throw_129thcolumn', '66throw_191thcolumn', '27throw_42thcolumn', '8throw_103thcolumn', '98throw_117thcolumn', '136throw_146thcolumn', '111throw_189thcolumn', '29throw_184thcolumn', '169throw_198thcolumn', '69throw_150thcolumn', '124throw_127thcolumn', '0throw_54thcolumn', '115throw_183thcolumn', '159throw_181thcolumn', '124throw_142thcolumn', '65throw_114thcolumn', '49throw_97thcolumn', '88throw_130thcolumn', '79throw_140thcolumn', '52throw_162thcolumn', '127throw_197thcolumn', '5throw_100thcolumn', '80throw_115thcolumn', '112throw_120thcolumn', '41throw_192thcolumn', '167throw_175thcolumn', '3throw_17thcolumn', '10throw_66thcolumn', '29throw_154thcolumn', '1throw_159thcolumn', '4throw_37thcolumn', '6throw_137thcolumn', '47throw_89thcolumn', '30throw_89thcolumn', '21throw_127thcolumn', '109throw_113thcolumn', '59throw_105thcolumn', '69throw_92thcolumn', '70throw_187thcolumn', '11throw_55thcolumn', '54throw_123thcolumn', '65throw_122thcolumn', '164throw_198thcolumn', '129throw_178thcolumn', '7throw_66thcolumn', '49throw_173thcolumn', '28throw_134thcolumn', '54throw_73thcolumn', '146throw_149thcolumn', '29throw_38thcolumn', '162throw_173thcolumn', '6throw_109thcolumn', '26throw_88thcolumn', '56throw_197thcolumn', '97throw_160thcolumn', '84throw_157thcolumn', '69throw_72thcolumn', '68throw_97thcolumn', '70throw_158thcolumn', '5throw_45thcolumn', '40throw_181thcolumn', '17throw_106thcolumn', '8throw_117thcolumn', '131throw_182thcolumn', '118throw_143thcolumn', '147throw_160thcolumn', '152throw_158thcolumn', '21throw_75thcolumn', '66throw_193thcolumn', '84throw_144thcolumn', '21throw_59thcolumn', '26throw_48thcolumn', '111throw_175thcolumn', '110throw_145thcolumn', '92throw_109thcolumn', '43throw_125thcolumn', '116throw_193thcolumn', '54throw_89thcolumn', '34throw_49thcolumn', '130throw_161thcolumn', '30throw_91thcolumn', '88throw_100thcolumn', '16throw_160thcolumn', '95throw_184thcolumn', '35throw_123thcolumn', '51throw_69thcolumn', '40throw_44thcolumn', '37throw_98thcolumn', '45throw_48thcolumn', '170throw_188thcolumn', '57throw_71thcolumn', '38throw_164thcolumn', '143throw_166thcolumn', '116throw_135thcolumn', '53throw_178thcolumn', '20throw_88thcolumn', '110throw_163thcolumn', '47throw_194thcolumn', '104throw_126thcolumn', '55throw_127thcolumn', '74throw_191thcolumn', '135throw_172thcolumn', '30throw_103thcolumn', '146throw_169thcolumn', '39throw_149thcolumn', '85throw_199thcolumn', '1throw_4thcolumn', '77throw_185thcolumn', '80throw_127thcolumn', '172throw_189thcolumn', '58throw_65thcolumn', '16throw_77thcolumn', '90throw_150thcolumn', '75throw_170thcolumn', '88throw_127thcolumn', '4throw_194thcolumn', '44throw_72thcolumn', '50throw_162thcolumn', '65throw_84thcolumn', '38throw_72thcolumn', '60throw_85thcolumn', '56throw_104thcolumn', '71throw_170thcolumn', '4throw_76thcolumn', '16throw_119thcolumn', '69throw_145thcolumn', '59throw_118thcolumn', '70throw_174thcolumn', '94throw_177thcolumn', '160throw_176thcolumn', '6throw_160thcolumn', '106throw_162thcolumn', '143throw_180thcolumn', '9throw_194thcolumn', '134throw_156thcolumn', '82throw_179thcolumn', '7throw_28thcolumn', '160throw_163thcolumn', '63throw_157thcolumn', '122throw_146thcolumn', '8throw_158thcolumn', '89throw_148thcolumn', '116throw_140thcolumn', '27throw_170thcolumn', '7throw_171thcolumn', '50throw_159thcolumn', '73throw_97thcolumn', '126throw_170thcolumn', '4throw_53thcolumn', '23throw_50thcolumn', '117throw_155thcolumn', '123throw_141thcolumn', '123throw_130thcolumn', '59throw_96thcolumn', '66throw_192thcolumn', '67throw_115thcolumn', '46throw_158thcolumn', '162throw_196thcolumn', '109throw_143thcolumn', '60throw_161thcolumn', '148throw_186thcolumn', '9throw_78thcolumn', '85throw_170thcolumn', '80throw_152thcolumn', '19throw_41thcolumn', '22throw_103thcolumn', '33throw_55thcolumn', '98throw_199thcolumn', '67throw_144thcolumn', '115throw_165thcolumn', '111throw_166thcolumn', '105throw_155thcolumn', '70throw_71thcolumn', '11throw_70thcolumn', '95throw_128thcolumn', '51throw_179thcolumn', '33throw_121thcolumn', '89throw_194thcolumn', '3throw_187thcolumn', '34throw_152thcolumn', '9throw_17thcolumn', '63throw_163thcolumn', '42throw_143thcolumn', '46throw_177thcolumn', '53throw_87thcolumn', '145throw_159thcolumn', '123throw_136thcolumn', '3throw_104thcolumn', '128throw_142thcolumn', '83throw_113thcolumn', '74throw_113thcolumn', '81throw_193thcolumn', '64throw_170thcolumn', '98throw_188thcolumn', '1throw_7thcolumn', '101throw_171thcolumn', '120throw_134thcolumn', '143throw_187thcolumn', '13throw_175thcolumn', '15throw_100thcolumn', '123throw_196thcolumn', '54throw_131thcolumn', '97throw_151thcolumn', '50throw_78thcolumn', '17throw_104thcolumn', '144throw_151thcolumn', '192throw_196thcolumn', '125throw_158thcolumn', '75throw_137thcolumn', '136throw_161thcolumn', '43throw_143thcolumn', '7throw_179thcolumn', '14throw_166thcolumn', '133throw_197thcolumn', '89throw_123thcolumn', '32throw_175thcolumn', '24throw_103thcolumn', '0throw_179thcolumn', '96throw_155thcolumn', '28throw_75thcolumn', '34throw_118thcolumn', '65throw_94thcolumn', '71throw_176thcolumn', '21throw_129thcolumn', '16throw_152thcolumn', '84throw_165thcolumn', '127throw_146thcolumn', '45throw_93thcolumn', '33throw_175thcolumn', '79throw_195thcolumn', '7throw_110thcolumn', '3throw_161thcolumn', '61throw_172thcolumn', '27throw_127thcolumn', '95throw_160thcolumn', '107throw_169thcolumn', '10throw_100thcolumn', '61throw_125thcolumn', '47throw_170thcolumn', '69throw_100thcolumn', '75throw_123thcolumn', '68throw_167thcolumn', '77throw_190thcolumn', '55throw_166thcolumn', '76throw_137thcolumn', '53throw_158thcolumn', '35throw_138thcolumn', '1throw_12thcolumn', '69throw_179thcolumn', '113throw_191thcolumn', '22throw_152thcolumn', '41throw_155thcolumn', '33throw_187thcolumn', '81throw_128thcolumn', '155throw_198thcolumn', '87throw_120thcolumn', '161throw_196thcolumn', '79throw_173thcolumn', '118throw_198thcolumn', '149throw_161thcolumn', '19throw_51thcolumn', '52throw_101thcolumn', '29throw_164thcolumn', '21throw_57thcolumn', '16throw_17thcolumn', '26throw_37thcolumn', '90throw_179thcolumn', '23throw_144thcolumn', '113throw_175thcolumn', '31throw_138thcolumn', '23throw_83thcolumn', '69throw_73thcolumn', '50throw_61thcolumn', '36throw_60thcolumn', '46throw_122thcolumn', '83throw_175thcolumn', '96throw_105thcolumn', '79throw_192thcolumn', '34throw_103thcolumn', '0throw_13thcolumn', '5throw_122thcolumn', '14throw_98thcolumn', '134throw_187thcolumn', '5throw_76thcolumn', '114throw_155thcolumn', '112throw_181thcolumn', '140throw_192thcolumn', '136throw_194thcolumn', '7throw_109thcolumn', '47throw_161thcolumn', '33throw_199thcolumn', '139throw_155thcolumn', '132throw_163thcolumn', '55throw_82thcolumn', '124throw_138thcolumn', '57throw_161thcolumn', '10throw_67thcolumn', '154throw_184thcolumn', '73throw_174thcolumn', '51throw_192thcolumn', '57throw_65thcolumn', '77throw_179thcolumn', '23throw_94thcolumn', '77throw_166thcolumn', '34throw_100thcolumn', '32throw_73thcolumn', '2throw_52thcolumn', '85throw_179thcolumn', '135throw_180thcolumn', '27throw_52thcolumn', '58throw_97thcolumn', '53throw_188thcolumn', '87throw_176thcolumn', '72throw_155thcolumn', '19throw_113thcolumn', '135throw_151thcolumn', '110throw_158thcolumn', '12throw_190thcolumn', '77throw_153thcolumn', '26throw_198thcolumn', '72throw_80thcolumn', '61throw_67thcolumn', '149throw_150thcolumn', '71throw_80thcolumn', '80throw_157thcolumn', '41throw_57thcolumn', '17throw_28thcolumn', '57throw_137thcolumn', '53throw_148thcolumn', '50throw_155thcolumn', '40throw_65thcolumn', '173throw_188thcolumn', '63throw_92thcolumn', '66throw_84thcolumn', '80throw_191thcolumn', '107throw_180thcolumn', '68throw_105thcolumn', '8throw_106thcolumn', '26throw_156thcolumn', '44throw_187thcolumn', '20throw_53thcolumn', '29throw_142thcolumn', '57throw_82thcolumn', '41throw_165thcolumn', '38throw_55thcolumn', '125throw_174thcolumn', '10throw_93thcolumn', '6throw_53thcolumn', '102throw_186thcolumn', '8throw_105thcolumn', '27throw_163thcolumn', '3throw_109thcolumn', '146throw_170thcolumn', '121throw_167thcolumn', '23throw_148thcolumn', '125throw_171thcolumn', '78throw_90thcolumn', '33throw_181thcolumn', '46throw_144thcolumn', '72throw_199thcolumn', '99throw_128thcolumn', '30throw_134thcolumn', '40throw_45thcolumn', '113throw_163thcolumn', '33throw_42thcolumn', '79throw_186thcolumn', '71throw_152thcolumn', '66throw_186thcolumn', '17throw_141thcolumn', '70throw_152thcolumn', '45throw_58thcolumn', '97throw_161thcolumn', '9throw_191thcolumn', '1throw_57thcolumn', '27throw_165thcolumn', '86throw_127thcolumn', '4throw_69thcolumn', '11throw_141thcolumn', '70throw_162thcolumn', '88throw_133thcolumn', '0throw_135thcolumn', '124throw_181thcolumn', '19throw_159thcolumn', '90throw_154thcolumn', '5throw_67thcolumn', '19throw_67thcolumn', '54throw_160thcolumn', '84throw_156thcolumn', '29throw_59thcolumn', '92throw_176thcolumn', '120throw_184thcolumn', '131throw_155thcolumn', '11throw_91thcolumn', '43throw_147thcolumn', '50throw_62thcolumn', '117throw_125thcolumn', '26throw_47thcolumn', '48throw_61thcolumn', '112throw_191thcolumn', '167throw_179thcolumn', '171throw_185thcolumn', '2throw_123thcolumn', '121throw_197thcolumn', '10throw_117thcolumn', '123throw_152thcolumn', '57throw_154thcolumn', '45throw_167thcolumn', '7throw_68thcolumn', '26throw_154thcolumn', '91throw_137thcolumn', '77throw_132thcolumn', '3throw_36thcolumn', '111throw_133thcolumn', '112throw_177thcolumn', '149throw_195thcolumn', '11throw_174thcolumn', '171throw_177thcolumn', '26throw_123thcolumn', '142throw_181thcolumn', '61throw_165thcolumn', '80throw_180thcolumn', '43throw_133thcolumn', '35throw_36thcolumn', '143throw_164thcolumn', '64throw_166thcolumn', '52throw_166thcolumn', '74throw_98thcolumn', '5throw_109thcolumn', '72throw_171thcolumn', '47throw_181thcolumn', '72throw_151thcolumn', '171throw_197thcolumn', '176throw_198thcolumn', '82throw_86thcolumn', '21throw_141thcolumn', '120throw_167thcolumn', '131throw_134thcolumn', '6throw_51thcolumn', '36throw_185thcolumn', '49throw_71thcolumn', '4throw_198thcolumn', '37throw_167thcolumn', '44throw_56thcolumn', '163throw_191thcolumn', '7throw_27thcolumn', '82throw_140thcolumn', '54throw_149thcolumn', '150throw_161thcolumn', '164throw_181thcolumn', '159throw_183thcolumn', '43throw_85thcolumn', '98throw_187thcolumn', '51throw_159thcolumn', '74throw_172thcolumn', '115throw_176thcolumn', '58throw_59thcolumn', '97throw_165thcolumn', '51throw_165thcolumn', '12throw_63thcolumn', '65throw_162thcolumn', '45throw_46thcolumn', '0throw_56thcolumn', '46throw_133thcolumn', '29throw_165thcolumn', '17throw_59thcolumn', '71throw_155thcolumn', '93throw_149thcolumn', '76throw_155thcolumn', '124throw_151thcolumn', '83throw_100thcolumn', '58throw_196thcolumn', '34throw_127thcolumn', '20throw_148thcolumn', '55throw_84thcolumn', '25throw_47thcolumn', '104throw_170thcolumn', '32throw_119thcolumn', '58throw_66thcolumn', '58throw_174thcolumn', '175throw_184thcolumn', '155throw_191thcolumn', '39throw_40thcolumn', '78throw_97thcolumn', '3throw_40thcolumn', '17throw_53thcolumn', '111throw_126thcolumn', '41throw_96thcolumn', '41throw_81thcolumn', '49throw_193thcolumn', '51throw_147thcolumn', '116throw_119thcolumn', '145throw_147thcolumn', '138throw_163thcolumn', '93throw_163thcolumn', '137throw_161thcolumn', '120throw_145thcolumn', '106throw_184thcolumn', '113throw_129thcolumn', '21throw_197thcolumn', '48throw_138thcolumn', '60throw_84thcolumn', '25throw_27thcolumn', '164throw_170thcolumn', '106throw_107thcolumn', 'Barratt_Barratt_P1_Edu', '160throw_187thcolumn', '55throw_140thcolumn', '95throw_121thcolumn', '106throw_117thcolumn', '154throw_165thcolumn', '27throw_47thcolumn', '128throw_198thcolumn', '148throw_164thcolumn', '186throw_196thcolumn', '66throw_98thcolumn', '38throw_125thcolumn', '38throw_190thcolumn', '99throw_199thcolumn', '73throw_146thcolumn', '34throw_194thcolumn', '0throw_11thcolumn', '74throw_79thcolumn', '78throw_171thcolumn', '159throw_180thcolumn', '7throw_30thcolumn', '9throw_66thcolumn', '0throw_38thcolumn', '28throw_43thcolumn', '110throw_184thcolumn', '80throw_93thcolumn', '103throw_179thcolumn', '47throw_57thcolumn', '144throw_167thcolumn', '47throw_52thcolumn', '95throw_107thcolumn', '135throw_153thcolumn', '19throw_50thcolumn', '184throw_191thcolumn', '40throw_57thcolumn', '133throw_156thcolumn', '134throw_141thcolumn', '118throw_120thcolumn', '78throw_130thcolumn', '0throw_192thcolumn', '12throw_102thcolumn', '137throw_191thcolumn', '97throw_199thcolumn', '53throw_77thcolumn', '96throw_110thcolumn', '47throw_102thcolumn', '73throw_109thcolumn', '26throw_103thcolumn', '47throw_190thcolumn', '106throw_159thcolumn', '31throw_92thcolumn', '72throw_176thcolumn', '86throw_167thcolumn', '53throw_187thcolumn', '63throw_91thcolumn', '48throw_68thcolumn', '6throw_161thcolumn', '83throw_115thcolumn', '53throw_122thcolumn', '34throw_108thcolumn', '170throw_195thcolumn', '164throw_172thcolumn', '79throw_152thcolumn', '159throw_178thcolumn', '69throw_129thcolumn', '84throw_173thcolumn', '47throw_163thcolumn', '39throw_128thcolumn', '8throw_152thcolumn', '12throw_58thcolumn', '174throw_187thcolumn', '46throw_61thcolumn', '120throw_126thcolumn', '37throw_170thcolumn', '2throw_106thcolumn', '129throw_191thcolumn', '39throw_97thcolumn', '23throw_76thcolumn', '33throw_126thcolumn', '171throw_178thcolumn', '117throw_152thcolumn', '84throw_192thcolumn', '17throw_162thcolumn', '69throw_95thcolumn', '74throw_167thcolumn', '16throw_114thcolumn', '55throw_83thcolumn', '25throw_83thcolumn', '130throw_160thcolumn', '108throw_180thcolumn', '123throw_128thcolumn', '95throw_193thcolumn', '160throw_188thcolumn', '54throw_157thcolumn', '157throw_191thcolumn', '47throw_123thcolumn', '27throw_122thcolumn', '84throw_128thcolumn', '140throw_151thcolumn', '73throw_92thcolumn', '38throw_163thcolumn', '151throw_168thcolumn', '88throw_159thcolumn', '83throw_187thcolumn', '0throw_30thcolumn', '98throw_192thcolumn', '169throw_183thcolumn', '120throw_197thcolumn', '49throw_185thcolumn', '55throw_67thcolumn', '25throw_68thcolumn', '39throw_45thcolumn', '134throw_138thcolumn', '120throw_191thcolumn', '133throw_153thcolumn', '88throw_190thcolumn', '134throw_173thcolumn', '33throw_144thcolumn', '8throw_59thcolumn', '16throw_42thcolumn', '26throw_197thcolumn', '107throw_193thcolumn', '100throw_140thcolumn', '60throw_103thcolumn', '29throw_43thcolumn', '7throw_20thcolumn', '0throw_1thcolumn', '97throw_144thcolumn', '85throw_183thcolumn', '54throw_75thcolumn', '96throw_191thcolumn', '77throw_124thcolumn', '155throw_197thcolumn', '65throw_151thcolumn', '90throw_124thcolumn', 'SDQ_SDQ_Internalizing', '74throw_93thcolumn', '38throw_83thcolumn', '22throw_77thcolumn', '151throw_187thcolumn', '100throw_134thcolumn', '13throw_65thcolumn', '8throw_157thcolumn', '64throw_167thcolumn', '40throw_164thcolumn', '57throw_125thcolumn', '21throw_70thcolumn', '138throw_196thcolumn', '59throw_190thcolumn', '169throw_177thcolumn', '98throw_115thcolumn', '9throw_69thcolumn', '80throw_122thcolumn', '16throw_161thcolumn', '49throw_130thcolumn', '82throw_182thcolumn', '34throw_74thcolumn', '89throw_163thcolumn', '60throw_104thcolumn', '17throw_42thcolumn', '154throw_198thcolumn', '37throw_110thcolumn', '19throw_31thcolumn', '87throw_160thcolumn', '127throw_175thcolumn', '124throw_143thcolumn', '39throw_93thcolumn', '36throw_97thcolumn', '10throw_128thcolumn', '24throw_185thcolumn', '28throw_110thcolumn', '85throw_126thcolumn', '32throw_68thcolumn', '53throw_121thcolumn', '97throw_162thcolumn', '72throw_119thcolumn', '98throw_127thcolumn', '159throw_172thcolumn', '28throw_63thcolumn', '31throw_70thcolumn', '17throw_119thcolumn', '74throw_166thcolumn', '133throw_178thcolumn', '118throw_123thcolumn', '130throw_169thcolumn', '13throw_30thcolumn', '47throw_53thcolumn', '150throw_158thcolumn', '121throw_154thcolumn', '152throw_192thcolumn', '130throw_184thcolumn', '153throw_187thcolumn', '80throw_156thcolumn', '70throw_173thcolumn', '2throw_27thcolumn', '39throw_44thcolumn', '11throw_99thcolumn', '177throw_187thcolumn', '185throw_193thcolumn', '78throw_192thcolumn', '97throw_169thcolumn', '14throw_51thcolumn', '84throw_113thcolumn', '23throw_161thcolumn', '131throw_197thcolumn', '17throw_190thcolumn', '28throw_41thcolumn', '124throw_146thcolumn', '112throw_137thcolumn', '54throw_178thcolumn', '52throw_147thcolumn', '35throw_195thcolumn', '95throw_159thcolumn', '50throw_52thcolumn', '27throw_198thcolumn', '56throw_189thcolumn', '98throw_123thcolumn', '27throw_113thcolumn', '9throw_186thcolumn', '96throw_108thcolumn', '121throw_168thcolumn', '68throw_158thcolumn', '147throw_184thcolumn', '118throw_190thcolumn', '145throw_197thcolumn']

# Deneme5=['4throw_74thcolumn', '141throw_147thcolumn', '136throw_174thcolumn', '128throw_197thcolumn', '124throw_168thcolumn', '7throw_158thcolumn', '162throw_177thcolumn', '33throw_170thcolumn', '167throw_170thcolumn', '3throw_157thcolumn', '84throw_136thcolumn', '22throw_177thcolumn', '61throw_93thcolumn', '15throw_125thcolumn', '89throw_129thcolumn', '43throw_182thcolumn', '117throw_122thcolumn', '49throw_124thcolumn', 'SDQ_SDQ_Difficulties_Total', '68throw_109thcolumn', '38throw_62thcolumn', '165throw_181thcolumn', '47throw_63thcolumn', '58throw_69thcolumn', '26throw_51thcolumn', '69throw_82thcolumn', '32throw_66thcolumn', '30throw_159thcolumn', '1throw_93thcolumn', '27throw_137thcolumn', '1throw_95thcolumn', '106throw_179thcolumn', '67throw_156thcolumn', '36throw_44thcolumn', '178throw_184thcolumn', '134throw_181thcolumn', '23throw_156thcolumn', '131throw_147thcolumn', '73throw_168thcolumn', '87throw_127thcolumn', '69throw_160thcolumn', '142throw_186thcolumn', '93throw_114thcolumn', '5throw_86thcolumn', '80throw_143thcolumn', '156throw_158thcolumn', '3throw_130thcolumn', '55throw_132thcolumn', '128throw_175thcolumn', '27throw_195thcolumn', '33throw_100thcolumn', '72throw_108thcolumn', '26throw_75thcolumn', '26throw_79thcolumn', '9throw_37thcolumn', '74throw_164thcolumn', '38throw_51thcolumn', '24throw_48thcolumn', '40throw_142thcolumn', '140throw_196thcolumn', '160throw_175thcolumn', '121throw_184thcolumn', '170throw_177thcolumn', '104throw_184thcolumn', '66throw_139thcolumn', '76throw_168thcolumn', '35throw_137thcolumn', '55throw_198thcolumn', '96throw_143thcolumn', '20throw_115thcolumn', '112throw_196thcolumn', '90throw_183thcolumn', '28throw_105thcolumn', '14throw_33thcolumn', '50throw_88thcolumn', '122throw_166thcolumn', '39throw_86thcolumn', '95throw_182thcolumn', '41throw_190thcolumn', '63throw_150thcolumn', '50throw_90thcolumn', '170throw_186thcolumn', '21throw_188thcolumn', '77throw_186thcolumn', '46throw_192thcolumn', '17throw_189thcolumn', '12throw_37thcolumn', '57throw_124thcolumn', '141throw_175thcolumn', '175throw_199thcolumn', '106throw_156thcolumn', '56throw_170thcolumn', '19throw_132thcolumn', '6throw_97thcolumn', '107throw_197thcolumn', '46throw_56thcolumn', '33throw_165thcolumn', '37throw_171thcolumn', '129throw_150thcolumn', '53throw_82thcolumn', '14throw_138thcolumn', '80throw_141thcolumn', '161throw_176thcolumn', '39throw_157thcolumn', '96throw_109thcolumn', '10throw_88thcolumn', '43throw_153thcolumn', '28throw_129thcolumn', '54throw_56thcolumn', '135throw_158thcolumn', '118throw_193thcolumn', '131throw_167thcolumn', '15throw_48thcolumn', '8throw_107thcolumn', '18throw_50thcolumn', '169throw_189thcolumn', '106throw_109thcolumn', '86throw_88thcolumn', '35throw_104thcolumn', '75throw_176thcolumn', '25throw_88thcolumn', '178throw_199thcolumn', '3throw_111thcolumn', '78throw_80thcolumn', '54throw_174thcolumn', '45throw_52thcolumn', '24throw_80thcolumn', '27throw_112thcolumn', '126throw_156thcolumn', '72throw_189thcolumn', '161throw_182thcolumn', '20throw_178thcolumn', '52throw_73thcolumn', '36throw_192thcolumn', '106throw_175thcolumn', '38throw_138thcolumn', '129throw_192thcolumn', '132throw_134thcolumn', '148throw_150thcolumn', '57throw_181thcolumn', '3throw_89thcolumn', '108throw_164thcolumn', '72throw_101thcolumn', '20throw_140thcolumn', '66throw_167thcolumn', '67throw_138thcolumn', '143throw_190thcolumn', '92throw_122thcolumn', '67throw_162thcolumn', '15throw_27thcolumn', '26throw_42thcolumn', '1throw_96thcolumn', '29throw_95thcolumn', '79throw_85thcolumn', '172throw_174thcolumn', '137throw_185thcolumn', '56throw_57thcolumn', '12throw_189thcolumn', '31throw_44thcolumn', '53throw_67thcolumn', '56throw_102thcolumn', '168throw_170thcolumn', '31throw_52thcolumn', '47throw_162thcolumn', '106throw_157thcolumn', '73throw_129thcolumn', '48throw_178thcolumn', '188throw_195thcolumn', '0throw_181thcolumn', '22throw_80thcolumn', '54throw_176thcolumn', '36throw_177thcolumn', '149throw_189thcolumn', '57throw_182thcolumn', '1throw_104thcolumn', '57throw_148thcolumn', '106throw_193thcolumn', 'APQ_P_APQ_P_PM', '39throw_106thcolumn', '39throw_125thcolumn', '39throw_192thcolumn', '126throw_142thcolumn', '110throw_135thcolumn', '7throw_178thcolumn', '72throw_166thcolumn', '16throw_190thcolumn', '51throw_145thcolumn', '159throw_164thcolumn', '75throw_167thcolumn', '68throw_90thcolumn', '21throw_89thcolumn', '151throw_180thcolumn', '94throw_171thcolumn', '37throw_194thcolumn', '130throw_180thcolumn', '34throw_146thcolumn', '142throw_184thcolumn', '94throw_98thcolumn', '121throw_180thcolumn', 'Barratt_Barratt_P1_Occ', '82throw_88thcolumn', '0throw_183thcolumn', '168throw_185thcolumn', '18throw_66thcolumn', '120throw_164thcolumn', '3throw_90thcolumn', '25throw_61thcolumn', '112throw_150thcolumn', '78throw_112thcolumn', '93throw_169thcolumn', '66throw_81thcolumn', '65throw_174thcolumn', '43throw_150thcolumn', '172throw_173thcolumn', '70throw_195thcolumn', '33throw_38thcolumn', '81throw_158thcolumn', '13throw_94thcolumn', '156throw_159thcolumn', '81throw_92thcolumn', '22throw_148thcolumn', '65throw_81thcolumn', '6throw_170thcolumn', '60throw_187thcolumn', '115throw_174thcolumn', '44throw_124thcolumn', '31throw_102thcolumn', '5throw_134thcolumn', '1throw_48thcolumn', '29throw_190thcolumn', '47throw_195thcolumn', '38throw_84thcolumn', '13throw_113thcolumn', '162throw_169thcolumn', '137throw_187thcolumn', '38throw_89thcolumn', '31throw_57thcolumn', '142throw_199thcolumn', '120throw_125thcolumn', '24throw_135thcolumn', '71throw_77thcolumn', '76throw_86thcolumn', '78throw_193thcolumn', '28throw_39thcolumn', '60throw_66thcolumn', '91throw_96thcolumn', '7throw_19thcolumn', '171throw_193thcolumn', '53throw_71thcolumn', '21throw_30thcolumn', '38throw_98thcolumn', '48throw_132thcolumn', '58throw_141thcolumn', '32throw_56thcolumn', '37throw_182thcolumn', '19throw_180thcolumn', '30throw_45thcolumn', '152throw_195thcolumn', '18throw_38thcolumn', '22throw_62thcolumn', '34throw_46thcolumn', '115throw_142thcolumn', '99throw_178thcolumn', '161throw_168thcolumn', '135throw_177thcolumn', '94throw_129thcolumn', '193throw_199thcolumn', '70throw_134thcolumn', '37throw_101thcolumn', '0throw_114thcolumn', '13throw_86thcolumn', '24throw_51thcolumn', '136throw_143thcolumn', '46throw_70thcolumn', '115throw_121thcolumn', '114throw_125thcolumn', '124throw_174thcolumn', '96throw_136thcolumn', '53throw_93thcolumn', '120throw_161thcolumn', '21throw_66thcolumn', '108throw_187thcolumn', '32throw_78thcolumn', '148throw_177thcolumn', '16throw_92thcolumn', '93throw_97thcolumn', '16throw_84thcolumn', '163throw_174thcolumn', '138throw_166thcolumn', '78throw_174thcolumn', '17throw_131thcolumn', '91throw_182thcolumn', '7throw_51thcolumn', '66throw_111thcolumn', '18throw_41thcolumn', '79throw_112thcolumn', '99throw_130thcolumn', '77throw_150thcolumn', '85throw_131thcolumn', '130throw_162thcolumn', '47throw_191thcolumn', '69throw_97thcolumn', '12throw_162thcolumn', '3throw_126thcolumn', '124throw_125thcolumn', '7throw_140thcolumn', '33throw_101thcolumn', '11throw_177thcolumn', '75throw_156thcolumn', '100throw_110thcolumn', '0throw_7thcolumn', '8throw_63thcolumn', '153throw_176thcolumn', '46throw_108thcolumn', '151throw_195thcolumn', '59throw_130thcolumn', '58throw_102thcolumn', '1throw_42thcolumn', '122throw_169thcolumn', '71throw_124thcolumn', '58throw_107thcolumn', '36throw_51thcolumn', '44throw_139thcolumn', '68throw_186thcolumn', '12throw_65thcolumn', '41throw_145thcolumn', '11throw_166thcolumn', '125throw_147thcolumn', '192throw_195thcolumn', '160throw_162thcolumn', '45throw_154thcolumn', '149throw_183thcolumn', '31throw_158thcolumn', '68throw_88thcolumn', '8throw_37thcolumn', '24throw_175thcolumn', '54throw_92thcolumn', '52throw_100thcolumn', '70throw_160thcolumn', '56throw_73thcolumn', '130throw_178thcolumn', '153throw_197thcolumn', '38throw_99thcolumn', '112throw_114thcolumn', '3throw_22thcolumn', '22throw_196thcolumn', '6throw_70thcolumn', '140throw_191thcolumn', '1throw_126thcolumn', '1throw_168thcolumn', '115throw_186thcolumn', '48throw_164thcolumn', '131throw_194thcolumn', '111throw_161thcolumn', '4throw_95thcolumn', '49throw_170thcolumn', '127throw_156thcolumn', '30throw_46thcolumn', '15throw_70thcolumn', '64throw_127thcolumn', '45throw_145thcolumn', '34throw_78thcolumn', '131throw_172thcolumn', '145throw_181thcolumn', '80throw_139thcolumn', '122throw_194thcolumn', '48throw_172thcolumn', '92throw_190thcolumn', '103throw_156thcolumn', '103throw_188thcolumn', '70throw_181thcolumn', '3throw_61thcolumn', '29throw_104thcolumn', '50throw_87thcolumn', '18throw_161thcolumn', '8throw_92thcolumn', '12throw_175thcolumn', '30throw_197thcolumn', '68throw_79thcolumn', '0throw_137thcolumn', '24throw_177thcolumn', '15throw_78thcolumn', '1throw_120thcolumn', '79throw_91thcolumn', '23throw_128thcolumn', '27throw_192thcolumn', '75throw_168thcolumn', '116throw_149thcolumn', '76throw_127thcolumn', '27throw_45thcolumn', '5throw_193thcolumn', '70throw_85thcolumn', '56throw_87thcolumn', '65throw_83thcolumn', '26throw_68thcolumn', '180throw_199thcolumn', '96throw_179thcolumn', '7throw_46thcolumn', '15throw_39thcolumn', '113throw_117thcolumn', '69throw_109thcolumn', '13throw_56thcolumn', '75throw_190thcolumn', '119throw_151thcolumn', '41throw_74thcolumn', '18throw_115thcolumn', '37throw_111thcolumn', '76throw_78thcolumn', '13throw_119thcolumn', '89throw_150thcolumn', '31throw_46thcolumn', '14throw_118thcolumn', '50throw_184thcolumn', '83throw_186thcolumn', '67throw_77thcolumn', '80throw_168thcolumn', '161throw_183thcolumn', '21throw_117thcolumn', '35throw_160thcolumn', '123throw_159thcolumn', '97throw_155thcolumn', '11throw_63thcolumn', '38throw_140thcolumn', '7throw_47thcolumn', '5throw_191thcolumn', '108throw_157thcolumn', '50throw_156thcolumn', '51throw_131thcolumn', '97throw_121thcolumn', '79throw_109thcolumn', '13throw_137thcolumn', '110throw_142thcolumn', '52throw_80thcolumn', '5throw_183thcolumn', '80throw_101thcolumn', '24throw_182thcolumn', '18throw_152thcolumn', '62throw_87thcolumn', '139throw_167thcolumn', '68throw_75thcolumn', '89throw_142thcolumn', '12throw_75thcolumn', '84throw_170thcolumn', '84throw_160thcolumn', '39throw_71thcolumn', '123throw_176thcolumn', '167throw_197thcolumn', '91throw_104thcolumn', '21throw_81thcolumn', '63throw_114thcolumn', '72throw_110thcolumn', '166throw_178thcolumn', '70throw_168thcolumn', '13throw_69thcolumn', '90throw_195thcolumn', '66throw_150thcolumn', '98throw_184thcolumn', '4throw_75thcolumn', '104throw_146thcolumn', '104throw_136thcolumn', '55throw_183thcolumn', '53throw_133thcolumn', '39throw_87thcolumn', '101throw_157thcolumn', '6throw_121thcolumn', '97throw_148thcolumn', '171throw_180thcolumn', '107throw_159thcolumn', '43throw_73thcolumn', '51throw_70thcolumn', '101throw_107thcolumn', '50throw_154thcolumn', '168throw_197thcolumn', '69throw_190thcolumn', '57throw_117thcolumn', '44throw_120thcolumn', '110throw_179thcolumn', '100throw_112thcolumn', '44throw_137thcolumn', '163throw_177thcolumn', '134throw_186thcolumn', '84throw_163thcolumn', '44throw_169thcolumn', '76throw_84thcolumn', '61throw_158thcolumn', '44throw_151thcolumn', '140throw_188thcolumn', '119throw_129thcolumn', '176throw_182thcolumn', '83throw_157thcolumn', '68throw_189thcolumn', '82throw_193thcolumn', '3throw_102thcolumn', '22throw_28thcolumn', '104throw_138thcolumn', '109throw_176thcolumn', '122throw_180thcolumn', '65throw_66thcolumn', '4throw_31thcolumn', '104throw_127thcolumn', '93throw_119thcolumn', '111throw_185thcolumn', '35throw_174thcolumn', '18throw_157thcolumn', '70throw_140thcolumn', '4throw_50thcolumn', '6throw_100thcolumn', '75throw_133thcolumn', '92throw_157thcolumn', '15throw_42thcolumn', '66throw_82thcolumn', '4throw_163thcolumn', '5throw_13thcolumn', '28throw_131thcolumn', '88throw_96thcolumn', '57throw_101thcolumn', '2throw_113thcolumn', '24throw_151thcolumn', '61throw_150thcolumn', '10throw_64thcolumn', '94throw_173thcolumn', '53throw_179thcolumn', '90throw_158thcolumn', '57throw_78thcolumn', '8throw_71thcolumn', '89throw_177thcolumn', '40throw_72thcolumn', '4throw_72thcolumn', '166throw_176thcolumn', '108throw_150thcolumn', '177throw_179thcolumn', '27throw_38thcolumn', '87throw_163thcolumn', '125throw_175thcolumn', '90throw_191thcolumn', '30throw_184thcolumn', '22throw_69thcolumn', '7throw_101thcolumn', '90throw_185thcolumn', '2throw_62thcolumn', '49throw_53thcolumn', '93throw_102thcolumn', '53throw_106thcolumn', '32throw_54thcolumn', '9throw_179thcolumn', '15throw_143thcolumn', '97throw_170thcolumn', '72throw_84thcolumn', '96throw_175thcolumn', '90throw_186thcolumn', '95throw_177thcolumn', '21throw_62thcolumn', '21throw_50thcolumn', '59throw_146thcolumn', '41throw_194thcolumn', '43throw_51thcolumn', '60throw_182thcolumn', '53throw_184thcolumn', '129throw_195thcolumn', '13throw_165thcolumn', '32throw_112thcolumn', '18throw_95thcolumn', '91throw_165thcolumn', '52throw_191thcolumn', '37throw_124thcolumn', '4throw_16thcolumn', '167throw_169thcolumn', '28throw_196thcolumn', '10throw_63thcolumn', '16throw_162thcolumn', '167throw_190thcolumn', '16throw_74thcolumn', '22throw_179thcolumn', '5throw_28thcolumn', '53throw_127thcolumn', '47throw_164thcolumn', '181throw_191thcolumn', '41throw_152thcolumn', '91throw_187thcolumn', '183throw_187thcolumn', '33throw_48thcolumn', '12throw_114thcolumn', '50throw_83thcolumn', '19throw_97thcolumn', '76throw_100thcolumn', '11throw_51thcolumn', '62throw_174thcolumn', '165throw_185thcolumn', '90throw_199thcolumn', '26throw_139thcolumn', '101throw_158thcolumn', '34throw_164thcolumn', '34throw_149thcolumn', '49throw_91thcolumn', '52throw_125thcolumn', '26throw_89thcolumn', '27throw_132thcolumn', '29throw_173thcolumn', '181throw_187thcolumn', '69throw_142thcolumn', '5throw_197thcolumn', '32throw_110thcolumn', '103throw_127thcolumn', '73throw_164thcolumn', '48throw_130thcolumn', '115throw_119thcolumn', '136throw_148thcolumn', '92throw_99thcolumn', '85throw_138thcolumn', '116throw_131thcolumn', '144throw_172thcolumn', '14throw_132thcolumn', '1throw_2thcolumn', '6throw_47thcolumn', '1throw_83thcolumn', '96throw_131thcolumn', '44throw_192thcolumn', '75throw_159thcolumn', '12throw_69thcolumn', '17throw_44thcolumn', '75throw_115thcolumn', '97throw_196thcolumn', '51throw_87thcolumn', '79throw_86thcolumn', '28throw_100thcolumn', '124throw_147thcolumn', '67throw_192thcolumn', '9throw_134thcolumn', '118throw_137thcolumn', '40throw_96thcolumn', '17throw_64thcolumn', '41throw_119thcolumn', '70throw_177thcolumn', '165throw_170thcolumn', '77throw_87thcolumn', '71throw_189thcolumn', '28throw_140thcolumn', '2throw_105thcolumn', '124throw_161thcolumn', '18throw_124thcolumn', '86throw_196thcolumn', '88throw_129thcolumn', '158throw_189thcolumn', '26throw_93thcolumn', '102throw_105thcolumn', '60throw_192thcolumn', '126throw_136thcolumn', '53throw_130thcolumn', '148throw_161thcolumn', '167throw_174thcolumn', '65throw_172thcolumn', '121throw_192thcolumn', '15throw_58thcolumn', '41throw_198thcolumn', '76throw_165thcolumn', '105throw_184thcolumn', '27throw_40thcolumn', '84throw_92thcolumn', '80throw_123thcolumn', '19throw_96thcolumn', '7throw_177thcolumn', '44throw_53thcolumn', '46throw_94thcolumn', '134throw_178thcolumn', '47throw_136thcolumn', '84throw_167thcolumn', '100throw_150thcolumn', '40throw_190thcolumn', '124throw_198thcolumn', '158throw_164thcolumn', '134throw_196thcolumn', '53throw_64thcolumn', '13throw_25thcolumn', '27throw_76thcolumn', '9throw_182thcolumn', '40throw_77thcolumn', '93throw_157thcolumn', '157throw_166thcolumn', '119throw_156thcolumn', '46throw_115thcolumn', '70throw_102thcolumn', '65throw_129thcolumn', '22throw_98thcolumn', '127throw_192thcolumn', '67throw_160thcolumn', '49throw_182thcolumn', '13throw_63thcolumn', '1throw_108thcolumn', '126throw_171thcolumn', '37throw_144thcolumn', '44throw_133thcolumn', '42throw_43thcolumn', '45throw_195thcolumn', '110throw_120thcolumn', '130throw_191thcolumn', '7throw_10thcolumn', '56throw_112thcolumn', '36throw_58thcolumn', '60throw_140thcolumn', '45throw_119thcolumn', '56throw_134thcolumn', '107throw_138thcolumn', '11throw_73thcolumn', '17throw_167thcolumn', '93throw_99thcolumn', '1throw_187thcolumn', '165throw_178thcolumn', '36throw_101thcolumn', '64throw_194thcolumn', '5throw_102thcolumn', '38throw_119thcolumn', '114throw_154thcolumn', '129throw_171thcolumn', '35throw_199thcolumn', '78throw_165thcolumn', '19throw_27thcolumn', '14throw_160thcolumn', '50throw_122thcolumn', '24throw_101thcolumn', '106throw_160thcolumn', '91throw_188thcolumn', '92throw_198thcolumn', '4throw_62thcolumn', '88throw_150thcolumn', '22throw_120thcolumn', '40throw_165thcolumn', '92throw_114thcolumn', '17throw_194thcolumn', '97throw_192thcolumn', '162throw_170thcolumn', '60throw_130thcolumn', '1throw_121thcolumn', '8throw_164thcolumn', '35throw_102thcolumn', '51throw_140thcolumn', '5throw_161thcolumn', '78throw_164thcolumn', '48throw_97thcolumn', '57throw_166thcolumn', '37throw_196thcolumn', '53throw_177thcolumn', '143throw_173thcolumn', '64throw_163thcolumn', '119throw_132thcolumn', '27throw_67thcolumn', '2throw_141thcolumn', '48throw_155thcolumn', '68throw_153thcolumn', '52throw_160thcolumn', '62throw_186thcolumn', '143throw_148thcolumn', '155throw_184thcolumn', '2throw_151thcolumn', '97throw_179thcolumn', '35throw_48thcolumn', '21throw_171thcolumn', '35throw_75thcolumn', '110throw_141thcolumn', '1throw_103thcolumn', '82throw_130thcolumn', '16throw_81thcolumn', '33throw_111thcolumn', '84throw_117thcolumn', '125throw_152thcolumn', '74throw_192thcolumn', '37throw_157thcolumn', '101throw_123thcolumn', '54throw_170thcolumn', '21throw_116thcolumn', '15throw_156thcolumn', '41throw_82thcolumn', '47throw_147thcolumn', '54throw_65thcolumn', '19throw_44thcolumn', '188throw_191thcolumn', '66throw_80thcolumn', '15throw_28thcolumn', '12throw_73thcolumn', '91throw_157thcolumn', '24throw_142thcolumn', '21throw_56thcolumn', '136throw_137thcolumn', '4throw_193thcolumn', '68throw_179thcolumn', '55throw_133thcolumn', '53throw_88thcolumn', '145throw_155thcolumn', '125throw_151thcolumn', '158throw_186thcolumn', '135throw_165thcolumn', '93throw_152thcolumn', '71throw_182thcolumn', '21throw_55thcolumn', '139throw_158thcolumn', '84throw_118thcolumn', '124throw_155thcolumn', '48throw_64thcolumn', '111throw_179thcolumn', '10throw_110thcolumn', '118throw_169thcolumn', '0throw_89thcolumn', '33throw_117thcolumn', '93throw_192thcolumn', '64throw_103thcolumn', '78throw_102thcolumn', '7throw_175thcolumn', '43throw_60thcolumn', '34throw_116thcolumn', '142throw_174thcolumn', '90throw_136thcolumn', '160throw_182thcolumn', '63throw_162thcolumn', '55throw_106thcolumn', '46throw_127thcolumn', '60throw_62thcolumn', '67throw_188thcolumn', '153throw_177thcolumn', '28throw_170thcolumn', '127throw_188thcolumn', '138throw_162thcolumn', '33throw_180thcolumn', '145throw_168thcolumn', '113throw_183thcolumn', '11throw_163thcolumn', '19throw_121thcolumn', '96throw_185thcolumn', '0throw_50thcolumn', '10throw_44thcolumn', '28throw_30thcolumn', '101throw_127thcolumn', '66throw_125thcolumn', '28throw_191thcolumn', '95throw_119thcolumn', '102throw_188thcolumn', '21throw_34thcolumn', '121throw_132thcolumn', '47throw_71thcolumn', '44throw_168thcolumn', '65throw_77thcolumn', '109throw_159thcolumn', '75throw_110thcolumn', '2throw_79thcolumn', '84throw_88thcolumn', '91throw_117thcolumn', '49throw_102thcolumn', '71throw_181thcolumn', '53throw_142thcolumn', '147throw_183thcolumn', '57throw_73thcolumn', '52throw_57thcolumn', '115throw_159thcolumn', '163throw_189thcolumn', '81throw_120thcolumn', '67throw_170thcolumn', '11throw_52thcolumn', '87throw_153thcolumn', '39throw_62thcolumn', '55throw_72thcolumn', '66throw_78thcolumn', '58throw_75thcolumn', '80throw_181thcolumn', '53throw_81thcolumn', '34throw_193thcolumn', '86throw_148thcolumn', '5throw_55thcolumn', '45throw_56thcolumn', '162throw_168thcolumn', '23throw_33thcolumn', '86throw_150thcolumn', '16throw_172thcolumn', '103throw_129thcolumn', '18throw_143thcolumn', '42throw_147thcolumn', '108throw_177thcolumn', '34throw_185thcolumn', '54throw_119thcolumn', '88throw_161thcolumn', '45throw_77thcolumn', '118throw_127thcolumn', '145throw_189thcolumn', '155throw_187thcolumn', '35throw_80thcolumn', '89throw_117thcolumn', '134throw_180thcolumn', '120throw_199thcolumn', '23throw_141thcolumn', '34throw_137thcolumn', '14throw_164thcolumn', '19throw_80thcolumn', '9throw_71thcolumn', '57throw_80thcolumn', '5throw_54thcolumn', '155throw_163thcolumn', '130throw_192thcolumn', '18throw_99thcolumn', '33throw_91thcolumn', '81throw_102thcolumn', '22throw_159thcolumn', '59throw_139thcolumn', '161throw_191thcolumn', '129throw_182thcolumn', '177throw_183thcolumn', '63throw_180thcolumn', '74throw_160thcolumn', '3throw_34thcolumn', '152throw_162thcolumn', '35throw_109thcolumn', '127throw_138thcolumn', '53throw_152thcolumn', '152throw_180thcolumn', '18throw_191thcolumn', '11throw_58thcolumn', '107throw_110thcolumn', '89throw_196thcolumn', '123throw_171thcolumn', '54throw_93thcolumn', '61throw_135thcolumn', '133throw_177thcolumn', '46throw_171thcolumn', '36throw_118thcolumn', '22throw_73thcolumn', '165throw_172thcolumn', '13throw_173thcolumn', '121throw_134thcolumn', '49throw_150thcolumn', '82throw_154thcolumn', '143throw_163thcolumn', '119throw_128thcolumn', '146throw_189thcolumn', '141throw_192thcolumn', '6throw_56thcolumn', '15throw_50thcolumn', '76throw_81thcolumn', '129throw_142thcolumn', '97throw_103thcolumn', '69throw_193thcolumn', '25throw_102thcolumn', '101throw_134thcolumn', '84throw_183thcolumn', '97throw_134thcolumn', '12throw_70thcolumn', '34throw_97thcolumn', '40throw_125thcolumn', '77throw_111thcolumn', '47throw_60thcolumn', '17throw_175thcolumn', '30throw_161thcolumn', '114throw_160thcolumn', '57throw_136thcolumn', '42throw_146thcolumn', '90throw_141thcolumn', '20throw_142thcolumn', '115throw_193thcolumn', '33throw_103thcolumn', '92throw_150thcolumn', '41throw_182thcolumn', '3throw_101thcolumn', '119throw_142thcolumn', '3throw_115thcolumn', '68throw_194thcolumn', '101throw_112thcolumn', '13throw_50thcolumn', '54throw_61thcolumn', '26throw_186thcolumn', '123throw_191thcolumn', '2throw_3thcolumn', '82throw_96thcolumn', '123throw_135thcolumn', '70throw_101thcolumn', '46throw_199thcolumn', '55throw_150thcolumn', '58throw_130thcolumn', '55throw_153thcolumn', '39throw_101thcolumn', '153throw_182thcolumn', '22throw_35thcolumn']


In [8]:
import numpy as np 
import pandas as pd
from sklearn.metrics import f1_score, roc_auc_score
import lightgbm as lgb, xgboost as xgb, catboost as cb
from gc import collect
import os
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import scipy
import seaborn as sns
from sklearn.svm import SVC
from sklearn.base import clone
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split,KFold
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.decomposition import PCA
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import StandardScaler, FunctionTransformer, PolynomialFeatures, MinMaxScaler
from sklearn.kernel_approximation import Nystroem
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, roc_curve, make_scorer
from sklearn.feature_selection import RFECV, SequentialFeatureSelector, SelectKBest
from sklearn.linear_model import LogisticRegression, RidgeClassifier,RidgeClassifierCV
from sklearn.ensemble import ExtraTreesRegressor, RandomForestClassifier
from sklearn.metrics import r2_score
from sklearn.manifold import TSNE

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import StackingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.impute import SimpleImputer
from lightgbm import early_stopping
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import cv
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})
use_pca = True
fill_nan= True
train_single_model = True
n_components = 0.90
FOLDS = 5
pd.set_option('display.max_columns', 1500)
pd.set_option('display.max_rows', 30)
cat_fea=['participant_id', 'Basic_Demos_Enroll_Year', 'Basic_Demos_Study_Site',
       'PreInt_Demos_Fam_Child_Ethnicity', 'PreInt_Demos_Fam_Child_Race',
       'MRI_Track_Scan_Location', 'Barratt_Barratt_P1_Edu',
       'Barratt_Barratt_P1_Occ', 'Barratt_Barratt_P2_Edu',
       'Barratt_Barratt_P2_Occ']
cat_fea=['participant_id', 'Basic_Demos_Enroll_Year','PreInt_Demos_Fam_Child_Ethnicity', 'PreInt_Demos_Fam_Child_Race']
def get_feats(mode='train'):
    if mode=='TRAIN_NEW':
        feats=pd.read_excel(f"/kaggle/input/widsdatathon2025/{mode}/TRAIN_QUANTITATIVE_METADATA_new.xlsx")
    else:
        feats=pd.read_excel(f"/kaggle/input/widsdatathon2025/{mode}/{mode}_QUANTITATIVE_METADATA.xlsx")
    
    if mode=='TRAIN_NEW':
        cate=pd.read_excel(f"/kaggle/input/widsdatathon2025/{mode}/TRAIN_CATEGORICAL_METADATA_new.xlsx")
    else:
        cate=pd.read_excel(f"/kaggle/input/widsdatathon2025/{mode}/{mode}_CATEGORICAL.xlsx")
    feats=feats.merge(cate,on='participant_id',how='left')
    
    if mode=='TEST':
        func=pd.read_csv(f"/kaggle/input/widsdatathon2025/{mode}/{mode}_FUNCTIONAL_CONNECTOME_MATRICES.csv")
    else:
        func=pd.read_csv(f"/kaggle/input/widsdatathon2025/{mode}/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv")
    global func_fea
    func_fea = func.columns.to_list()
    func_fea.remove('participant_id')
    feats=feats.merge(func,on='participant_id',how='left')

    if mode=='TRAIN_NEW':
        solution=pd.read_excel("/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAINING_SOLUTIONS.xlsx")
        feats=feats.merge(solution,on='participant_id',how='left')
        
    return feats
    
train=get_feats(mode='TRAIN_NEW')
test=get_feats(mode='TEST')
sub = pd.read_excel('/kaggle/input/widsdatathon2025/SAMPLE_SUBMISSION.xlsx')
y = pd.read_excel(f"/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAINING_SOLUTIONS.xlsx")


Deneme1 = ['EHQ_EHQ_Total', 'ColorVision_CV_Score', 'APQ_P_APQ_P_CP', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD', 'APQ_P_APQ_P_PP', 'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial', 'MRI_Track_Age_at_Scan', '0throw_5thcolumn', '0throw_20thcolumn', '0throw_61thcolumn', '0throw_70thcolumn', '0throw_86thcolumn', '0throw_95thcolumn', '0throw_102thcolumn', '0throw_104thcolumn', '0throw_105thcolumn', '0throw_113thcolumn', '0throw_132thcolumn', '0throw_134thcolumn', '0throw_142thcolumn', '0throw_161thcolumn', '1throw_25thcolumn', '1throw_26thcolumn', '1throw_52thcolumn', '1throw_53thcolumn', '1throw_56thcolumn', '1throw_61thcolumn', '1throw_73thcolumn', '1throw_78thcolumn', '1throw_101thcolumn', '1throw_117thcolumn', '1throw_119thcolumn', '1throw_161thcolumn', '1throw_182thcolumn', '2throw_49thcolumn', '2throw_56thcolumn', '2throw_60thcolumn', '2throw_95thcolumn', '2throw_126thcolumn', '2throw_175thcolumn', '3throw_9thcolumn', '3throw_11thcolumn', '3throw_50thcolumn', '3throw_121thcolumn', '3throw_139thcolumn', '4throw_78thcolumn', '4throw_79thcolumn', '4throw_80thcolumn', '4throw_83thcolumn', '4throw_91thcolumn', '4throw_135thcolumn', '4throw_185thcolumn', '5throw_7thcolumn', '5throw_30thcolumn', '5throw_33thcolumn', '5throw_96thcolumn', '5throw_99thcolumn', '5throw_131thcolumn', '6throw_30thcolumn', '6throw_138thcolumn', '7throw_33thcolumn', '7throw_39thcolumn', '7throw_57thcolumn', '7throw_74thcolumn', '7throw_75thcolumn', '7throw_106thcolumn', '7throw_107thcolumn', '7throw_141thcolumn', '7throw_157thcolumn', '7throw_167thcolumn', '8throw_51thcolumn', '8throw_124thcolumn', '9throw_63thcolumn', '9throw_68thcolumn', '9throw_73thcolumn', '9throw_106thcolumn', '9throw_107thcolumn', '9throw_108thcolumn', '9throw_141thcolumn', '9throw_151thcolumn', '10throw_12thcolumn', '10throw_86thcolumn', '10throw_108thcolumn', '10throw_135thcolumn', '10throw_143thcolumn', '10throw_177thcolumn', '11throw_93thcolumn', '11throw_105thcolumn', '11throw_110thcolumn', '11throw_111thcolumn', '11throw_180thcolumn', '11throw_184thcolumn', '11throw_193thcolumn', '12throw_47thcolumn', '12throw_95thcolumn', '12throw_121thcolumn', '12throw_192thcolumn', '13throw_20thcolumn', '13throw_54thcolumn', '14throw_113thcolumn', '14throw_123thcolumn', '14throw_157thcolumn', '15throw_26thcolumn', '15throw_72thcolumn', '15throw_86thcolumn', '15throw_114thcolumn', '16throw_37thcolumn', '17throw_63thcolumn', '17throw_96thcolumn', '17throw_123thcolumn', '17throw_125thcolumn', '17throw_160thcolumn', '17throw_199thcolumn', '18throw_21thcolumn', '18throw_61thcolumn', '18throw_81thcolumn', '18throw_85thcolumn', '18throw_188thcolumn', '18throw_195thcolumn', '19throw_23thcolumn', '19throw_48thcolumn', '19throw_69thcolumn', '19throw_86thcolumn', '19throw_116thcolumn', '19throw_127thcolumn', '19throw_129thcolumn', '19throw_134thcolumn', '19throw_145thcolumn', '20throw_70thcolumn', '20throw_79thcolumn', '20throw_83thcolumn', '20throw_93thcolumn', '20throw_96thcolumn', '20throw_125thcolumn', '20throw_126thcolumn', '20throw_159thcolumn', '20throw_196thcolumn', '21throw_47thcolumn', '21throw_51thcolumn', '21throw_61thcolumn', '21throw_94thcolumn', '21throw_125thcolumn', '21throw_178thcolumn', '22throw_40thcolumn', '22throw_60thcolumn', '22throw_75thcolumn', '22throw_92thcolumn', '22throw_110thcolumn', '22throw_141thcolumn', '22throw_158thcolumn', '22throw_162thcolumn', '23throw_42thcolumn', '23throw_87thcolumn', '23throw_88thcolumn', '23throw_89thcolumn', '23throw_92thcolumn', '23throw_112thcolumn', '23throw_123thcolumn', '24throw_36thcolumn', '24throw_72thcolumn', '24throw_124thcolumn', '24throw_127thcolumn', '24throw_181thcolumn', '24throw_198thcolumn', '24throw_199thcolumn', '25throw_64thcolumn', '25throw_70thcolumn', '25throw_78thcolumn', '25throw_89thcolumn', '25throw_123thcolumn', '25throw_127thcolumn', '26throw_35thcolumn', '26throw_69thcolumn', '26throw_92thcolumn', '26throw_116thcolumn', '26throw_171thcolumn', '26throw_190thcolumn', '27throw_68thcolumn', '27throw_97thcolumn', '27throw_114thcolumn', '27throw_158thcolumn', '27throw_174thcolumn', '28throw_56thcolumn', '28throw_80thcolumn', '28throw_83thcolumn', '28throw_92thcolumn', '28throw_132thcolumn', '28throw_190thcolumn', '28throw_194thcolumn', '28throw_199thcolumn', '29throw_57thcolumn', '29throw_73thcolumn', '29throw_115thcolumn', '29throw_118thcolumn', '29throw_134thcolumn', '30throw_48thcolumn', '30throw_54thcolumn', '30throw_67thcolumn', '30throw_88thcolumn', '30throw_124thcolumn', '30throw_130thcolumn', '30throw_170thcolumn', '30throw_188thcolumn', '31throw_65thcolumn', '31throw_112thcolumn', '31throw_132thcolumn', '32throw_49thcolumn', '32throw_60thcolumn', '32throw_94thcolumn', '32throw_165thcolumn', '33throw_47thcolumn', '33throw_53thcolumn', '33throw_135thcolumn', '33throw_169thcolumn', '34throw_68thcolumn', '34throw_73thcolumn', '34throw_77thcolumn', '34throw_125thcolumn', '34throw_140thcolumn', '34throw_174thcolumn', '35throw_96thcolumn', '35throw_118thcolumn', '35throw_125thcolumn', '36throw_55thcolumn', '36throw_59thcolumn', '36throw_163thcolumn', '37throw_61thcolumn', '37throw_108thcolumn', '37throw_163thcolumn', '37throw_190thcolumn', '38throw_48thcolumn', '38throw_68thcolumn', '38throw_82thcolumn', '38throw_85thcolumn', '38throw_153thcolumn', '38throw_156thcolumn', '38throw_169thcolumn', '39throw_73thcolumn', '39throw_116thcolumn', '39throw_123thcolumn', '39throw_134thcolumn', '39throw_136thcolumn', '39throw_182thcolumn', '40throw_46thcolumn', '40throw_63thcolumn', '40throw_124thcolumn', '40throw_149thcolumn', '40throw_172thcolumn', '40throw_188thcolumn', '41throw_50thcolumn', '41throw_62thcolumn', '41throw_75thcolumn', '41throw_77thcolumn', '41throw_97thcolumn', '41throw_112thcolumn', '41throw_186thcolumn', '42throw_85thcolumn', '42throw_102thcolumn', '42throw_152thcolumn', '42throw_181thcolumn', '43throw_44thcolumn', '43throw_50thcolumn', '43throw_99thcolumn', '43throw_142thcolumn', '44throw_50thcolumn', '44throw_54thcolumn', '44throw_87thcolumn', '44throw_93thcolumn', '44throw_130thcolumn', '44throw_132thcolumn', '44throw_150thcolumn', '44throw_157thcolumn', '44throw_162thcolumn', '44throw_165thcolumn', '44throw_184thcolumn', '45throw_70thcolumn', '45throw_148thcolumn', '45throw_150thcolumn', '45throw_155thcolumn', '45throw_173thcolumn', '46throw_182thcolumn', '46throw_184thcolumn', '47throw_80thcolumn', '47throw_94thcolumn', '47throw_118thcolumn', '47throw_160thcolumn', '48throw_123thcolumn', '48throw_163thcolumn', '48throw_166thcolumn', '48throw_185thcolumn', '49throw_56thcolumn', '49throw_133thcolumn', '49throw_153thcolumn', '49throw_184thcolumn', '49throw_191thcolumn', '50throw_53thcolumn', '50throw_66thcolumn', '50throw_72thcolumn', '50throw_129thcolumn', '50throw_157thcolumn', '50throw_158thcolumn', '50throw_187thcolumn', '51throw_61thcolumn', '51throw_80thcolumn', '51throw_92thcolumn', '51throw_123thcolumn', '51throw_177thcolumn', '52throw_61thcolumn', '52throw_64thcolumn', '52throw_106thcolumn', '52throw_179thcolumn', '52throw_182thcolumn', '53throw_70thcolumn', '53throw_75thcolumn', '53throw_86thcolumn', '53throw_89thcolumn', '53throw_94thcolumn', '53throw_98thcolumn', '53throw_103thcolumn', '53throw_131thcolumn', '53throw_191thcolumn', '54throw_57thcolumn', '54throw_60thcolumn', '54throw_69thcolumn', '54throw_80thcolumn', '54throw_87thcolumn', '54throw_97thcolumn', '54throw_162thcolumn', '54throw_182thcolumn', '55throw_86thcolumn', '55throw_100thcolumn', '55throw_135thcolumn', '55throw_154thcolumn', '55throw_156thcolumn', '55throw_159thcolumn', '56throw_62thcolumn', '56throw_72thcolumn', '56throw_79thcolumn', '56throw_140thcolumn', '56throw_146thcolumn', '56throw_148thcolumn', '56throw_168thcolumn', '56throw_188thcolumn', '57throw_90thcolumn', '57throw_96thcolumn', '57throw_97thcolumn', '57throw_140thcolumn', '57throw_146thcolumn', '57throw_151thcolumn', '57throw_167thcolumn', '57throw_173thcolumn', '57throw_179thcolumn', '57throw_197thcolumn', '58throw_68thcolumn', '58throw_73thcolumn', '58throw_106thcolumn', '58throw_137thcolumn', '58throw_181thcolumn', '59throw_88thcolumn', '59throw_91thcolumn', '59throw_168thcolumn', '59throw_196thcolumn', '60throw_93thcolumn', '60throw_100thcolumn', '61throw_79thcolumn', '61throw_118thcolumn', '61throw_123thcolumn', '61throw_124thcolumn', '61throw_134thcolumn', '61throw_173thcolumn', '62throw_104thcolumn', '62throw_123thcolumn', '62throw_128thcolumn', '62throw_132thcolumn', '62throw_135thcolumn', '62throw_164thcolumn', '63throw_93thcolumn', '63throw_174thcolumn', '63throw_188thcolumn', '63throw_194thcolumn', '64throw_81thcolumn', '64throw_144thcolumn', '64throw_150thcolumn', '64throw_151thcolumn', '64throw_190thcolumn', '65throw_135thcolumn', '65throw_170thcolumn', '65throw_178thcolumn', '65throw_189thcolumn', '65throw_192thcolumn', '66throw_85thcolumn', '66throw_131thcolumn', '66throw_144thcolumn', '66throw_157thcolumn', '67throw_69thcolumn', '67throw_70thcolumn', '67throw_89thcolumn', '67throw_116thcolumn', '67throw_131thcolumn', '67throw_149thcolumn', '67throw_168thcolumn', '68throw_78thcolumn', '68throw_96thcolumn', '68throw_114thcolumn', '68throw_183thcolumn', '68throw_192thcolumn', '69throw_111thcolumn', '69throw_137thcolumn', '69throw_158thcolumn', '69throw_165thcolumn', '69throw_185thcolumn', '69throw_191thcolumn', '70throw_79thcolumn', '70throw_97thcolumn', '70throw_99thcolumn', '70throw_114thcolumn', '70throw_123thcolumn', '70throw_125thcolumn', '70throw_126thcolumn', '70throw_144thcolumn', '70throw_161thcolumn', '70throw_170thcolumn', '70throw_180thcolumn', '70throw_183thcolumn', '71throw_78thcolumn', '71throw_99thcolumn', '71throw_171thcolumn', '72throw_78thcolumn', '72throw_174thcolumn', '72throw_177thcolumn', '73throw_82thcolumn', '73throw_86thcolumn', '73throw_182thcolumn', '74throw_83thcolumn', '74throw_120thcolumn', '74throw_140thcolumn', '75throw_88thcolumn', '75throw_97thcolumn', '75throw_120thcolumn', '75throw_125thcolumn', '75throw_129thcolumn', '75throw_160thcolumn', '76throw_123thcolumn', '76throw_156thcolumn', '76throw_185thcolumn', '77throw_99thcolumn', '77throw_105thcolumn', '77throw_106thcolumn', '77throw_145thcolumn', '77throw_151thcolumn', '77throw_170thcolumn', '77throw_198thcolumn', '78throw_133thcolumn', '78throw_163thcolumn', '78throw_167thcolumn', '78throw_195thcolumn', '79throw_89thcolumn', '79throw_150thcolumn', '79throw_155thcolumn', '79throw_166thcolumn', '79throw_176thcolumn', '80throw_92thcolumn', '80throw_125thcolumn', '80throw_160thcolumn', '80throw_161thcolumn', '81throw_101thcolumn', '81throw_115thcolumn', '81throw_129thcolumn', '81throw_133thcolumn', '81throw_154thcolumn', '81throw_163thcolumn', '82throw_124thcolumn', '82throw_191thcolumn', '83throw_90thcolumn', '83throw_177thcolumn', '84throw_87thcolumn', '84throw_137thcolumn', '84throw_141thcolumn', '84throw_146thcolumn', '84throw_147thcolumn', '84throw_194thcolumn', '85throw_89thcolumn', '85throw_111thcolumn', '85throw_117thcolumn', '85throw_192thcolumn', '85throw_196thcolumn', '85throw_197thcolumn', '86throw_90thcolumn', '86throw_153thcolumn', '86throw_168thcolumn', '86throw_183thcolumn', '86throw_198thcolumn', '87throw_122thcolumn', '87throw_139thcolumn', '88throw_114thcolumn', '88throw_163thcolumn', '88throw_165thcolumn', '89throw_96thcolumn', '89throw_106thcolumn', '89throw_115thcolumn', '89throw_179thcolumn', '89throw_191thcolumn', '90throw_180thcolumn', '91throw_93thcolumn', '91throw_105thcolumn', '91throw_112thcolumn', '91throw_133thcolumn', '91throw_134thcolumn', '92throw_135thcolumn', '92throw_159thcolumn', '92throw_175thcolumn', '92throw_189thcolumn', '93throw_96thcolumn', '93throw_100thcolumn', '93throw_120thcolumn', '93throw_135thcolumn', '93throw_172thcolumn', '93throw_173thcolumn', '93throw_174thcolumn', '94throw_107thcolumn', '94throw_166thcolumn', '95throw_102thcolumn', '95throw_155thcolumn', '96throw_104thcolumn', '96throw_106thcolumn', '96throw_112thcolumn', '96throw_138thcolumn', '96throw_184thcolumn', '96throw_194thcolumn', '97throw_100thcolumn', '97throw_143thcolumn', '97throw_154thcolumn', '98throw_167thcolumn', '98throw_183thcolumn', '99throw_147thcolumn', '99throw_162thcolumn', '100throw_135thcolumn', '100throw_171thcolumn', '101throw_152thcolumn', '101throw_196thcolumn', '102throw_111thcolumn', '102throw_120thcolumn', '102throw_132thcolumn', '102throw_177thcolumn', '102throw_194thcolumn', '104throw_130thcolumn', '104throw_178thcolumn', '106throw_110thcolumn', '106throw_192thcolumn', '107throw_126thcolumn', '108throw_110thcolumn', '108throw_114thcolumn', '108throw_119thcolumn', '108throw_141thcolumn', '108throw_151thcolumn', '108throw_155thcolumn', '108throw_181thcolumn', '109throw_157thcolumn', '109throw_177thcolumn', '109throw_179thcolumn', '110throw_159thcolumn', '110throw_166thcolumn', '110throw_177thcolumn', '110throw_185thcolumn', '110throw_186thcolumn', '111throw_121thcolumn', '111throw_135thcolumn', '111throw_137thcolumn', '111throw_151thcolumn', '112throw_116thcolumn', '112throw_118thcolumn', '113throw_142thcolumn', '113throw_189thcolumn', '113throw_199thcolumn', '114throw_148thcolumn', '114throw_176thcolumn', '114throw_182thcolumn', '114throw_184thcolumn', '114throw_188thcolumn', '114throw_192thcolumn', '115throw_171thcolumn', '115throw_173thcolumn', '115throw_180thcolumn', '116throw_118thcolumn', '116throw_136thcolumn', '116throw_137thcolumn', '117throw_130thcolumn', '117throw_135thcolumn', '118throw_162thcolumn', '119throw_173thcolumn', '120throw_129thcolumn', '120throw_138thcolumn', '120throw_179thcolumn', '120throw_194thcolumn', '121throw_122thcolumn', '122throw_176thcolumn', '122throw_189thcolumn', '122throw_199thcolumn', '123throw_127thcolumn', '123throw_165thcolumn', '123throw_175thcolumn', '123throw_188thcolumn', '123throw_190thcolumn', '124throw_158thcolumn', '124throw_175thcolumn', '125throw_157thcolumn', '125throw_159thcolumn', '125throw_193thcolumn', '125throw_197thcolumn', '126throw_153thcolumn', '126throw_175thcolumn', '127throw_135thcolumn', '127throw_141thcolumn', '127throw_158thcolumn', '127throw_159thcolumn', '127throw_172thcolumn', '127throw_184thcolumn', '127throw_191thcolumn', '128throw_178thcolumn', '129throw_168thcolumn', '129throw_188thcolumn', '129throw_190thcolumn', '130throw_140thcolumn', '130throw_144thcolumn', '130throw_155thcolumn', '130throw_174thcolumn', '130throw_183thcolumn', '130throw_187thcolumn', '131throw_148thcolumn', '133throw_151thcolumn', '135throw_141thcolumn', '135throw_183thcolumn', '136throw_169thcolumn', '137throw_150thcolumn', '137throw_169thcolumn', '137throw_173thcolumn', '137throw_177thcolumn', '138throw_148thcolumn', '138throw_171thcolumn', '138throw_194thcolumn', '139throw_177thcolumn', '140throw_153thcolumn', '140throw_163thcolumn', '140throw_176thcolumn', '141throw_145thcolumn', '143throw_170thcolumn', '144throw_156thcolumn', '144throw_176thcolumn', '144throw_194thcolumn', '144throw_199thcolumn', '145throw_167thcolumn', '146throw_165thcolumn', '146throw_171thcolumn', '147throw_193thcolumn', '148throw_160thcolumn', '148throw_166thcolumn', '149throw_163thcolumn', '149throw_165thcolumn', '149throw_173thcolumn', '149throw_178thcolumn', '149throw_192thcolumn', '150throw_170thcolumn', '150throw_173thcolumn', '151throw_164thcolumn', '151throw_173thcolumn', '152throw_160thcolumn', '152throw_164thcolumn', '154throw_156thcolumn', '154throw_166thcolumn', '155throw_170thcolumn', '155throw_185thcolumn', '155throw_196thcolumn', '156throw_183thcolumn', '157throw_159thcolumn', '158throw_187thcolumn', '159throw_160thcolumn', '159throw_190thcolumn', '160throw_171thcolumn', '160throw_195thcolumn', '161throw_174thcolumn', '161throw_184thcolumn', '161throw_195thcolumn', '162throw_194thcolumn', '163throw_166thcolumn', '163throw_178thcolumn', '163throw_181thcolumn', '164throw_179thcolumn', '165throw_173thcolumn', '165throw_180thcolumn', '165throw_186thcolumn', '166throw_174thcolumn', '166throw_194thcolumn', '169throw_170thcolumn', '169throw_184thcolumn', '170throw_173thcolumn', '173throw_177thcolumn', '174throw_190thcolumn', '174throw_192thcolumn', '174throw_197thcolumn', '175throw_180thcolumn', '176throw_197thcolumn', '177throw_180thcolumn', '180throw_193thcolumn', '181throw_184thcolumn', '181throw_193thcolumn', '181throw_195thcolumn', '182throw_192thcolumn', '184throw_193thcolumn', '186throw_187thcolumn', '190throw_191thcolumn', '194throw_196thcolumn', '197throw_199thcolumn']
#2.deneme
Deneme2=['Basic_Demos_Enroll_Year', '0throw_9thcolumn', '0throw_109thcolumn', '2throw_16thcolumn', '2throw_22thcolumn', '5throw_128thcolumn', '5throw_145thcolumn', '8throw_80thcolumn', '10throw_175thcolumn', '11throw_135thcolumn', '14throw_190thcolumn', '16throw_51thcolumn', '25throw_125thcolumn', '27throw_95thcolumn', '28throw_46thcolumn', '39throw_111thcolumn', '41throw_120thcolumn', '43throw_126thcolumn', '53throw_162thcolumn', '59throw_112thcolumn', '60throw_117thcolumn', '64throw_75thcolumn', '65throw_199thcolumn', '69throw_79thcolumn', '69throw_173thcolumn', '69throw_186thcolumn', '73throw_169thcolumn', '78throw_168thcolumn', '91throw_135thcolumn', '92throw_160thcolumn', '95throw_116thcolumn', '108throw_109thcolumn', '110throw_167thcolumn', '116throw_168thcolumn', '123throw_168thcolumn', '123throw_192thcolumn', '129throw_157thcolumn', '130throw_153thcolumn', '138throw_184thcolumn', '149throw_197thcolumn', '153throw_189thcolumn']

Deneme3=['Basic_Demos_Study_Site', '35throw_186thcolumn', '132throw_162thcolumn', '106throw_124thcolumn', '5throw_156thcolumn', '26throw_38thcolumn', '37throw_71thcolumn', '55throw_182thcolumn', '73throw_138thcolumn', '3throw_38thcolumn', '32throw_34thcolumn', '0throw_136thcolumn', '26throw_132thcolumn', '73throw_76thcolumn', '39throw_90thcolumn', '84throw_177thcolumn', '128throw_189thcolumn', '187throw_188thcolumn', '67throw_81thcolumn', '107throw_121thcolumn', '86throw_147thcolumn', '31throw_60thcolumn', '21throw_176thcolumn', '32throw_111thcolumn', '43throw_76thcolumn', '60throw_165thcolumn', '115throw_197thcolumn', '85throw_195thcolumn', '26throw_52thcolumn', '26throw_63thcolumn', '123throw_163thcolumn', '67throw_193thcolumn', '139throw_154thcolumn', '42throw_98thcolumn', '9throw_157thcolumn', '139throw_171thcolumn', '137throw_148thcolumn', '10throw_70thcolumn', '0throw_78thcolumn', '0throw_42thcolumn', '26throw_36thcolumn', '2throw_101thcolumn', '8throw_189thcolumn', '64throw_131thcolumn', '95throw_101thcolumn', '160throw_174thcolumn', '63throw_142thcolumn', '131throw_165thcolumn', '98throw_151thcolumn', '3throw_181thcolumn', '22throw_176thcolumn', '37throw_78thcolumn', '2throw_42thcolumn', '23throw_93thcolumn', '12throw_72thcolumn', '19throw_57thcolumn', '53throw_197thcolumn', '106throw_168thcolumn', '134throw_148thcolumn', '53throw_137thcolumn', '19throw_75thcolumn', '28throw_175thcolumn', '76throw_194thcolumn', '70throw_175thcolumn', '10throw_96thcolumn', '86throw_164thcolumn', '78throw_143thcolumn', '84throw_138thcolumn', 'MRI_Track_Scan_Location', '23throw_59thcolumn', '73throw_141thcolumn', '71throw_111thcolumn', '64throw_160thcolumn', '41throw_130thcolumn', '105throw_192thcolumn', '113throw_127thcolumn', '33throw_89thcolumn', '57throw_191thcolumn', '33throw_183thcolumn', '17throw_51thcolumn', '81throw_159thcolumn', '128throw_169thcolumn', '86throw_96thcolumn', '102throw_152thcolumn', '147throw_187thcolumn', '10throw_36thcolumn', '37throw_173thcolumn', '11throw_173thcolumn', '85throw_188thcolumn', '3throw_167thcolumn', '117throw_197thcolumn', '26throw_87thcolumn', '104throw_194thcolumn', '113throw_156thcolumn', '52throw_152thcolumn', '106throw_150thcolumn', '104throw_121thcolumn', '15throw_38thcolumn', '11throw_170thcolumn', '111throw_153thcolumn', '96throw_172thcolumn', '38throw_133thcolumn', '40throw_119thcolumn', '113throw_169thcolumn', '64throw_114thcolumn', '94throw_189thcolumn', '25throw_60thcolumn', '23throw_195thcolumn', '33throw_40thcolumn', '4throw_124thcolumn', '126throw_152thcolumn', '115throw_122thcolumn', '46throw_80thcolumn', '15throw_104thcolumn', '169throw_182thcolumn', '36throw_195thcolumn', '0throw_41thcolumn', '51throw_152thcolumn', '22throw_72thcolumn', '56throw_155thcolumn', '33throw_70thcolumn', '40throw_169thcolumn', '186throw_189thcolumn', '76throw_143thcolumn', '58throw_103thcolumn', '56throw_137thcolumn', '43throw_120thcolumn', '20throw_110thcolumn', '5throw_162thcolumn', '4throw_156thcolumn', '114throw_117thcolumn', '56throw_133thcolumn', '114throw_177thcolumn', '17throw_161thcolumn', '154throw_164thcolumn', '40throw_180thcolumn', '32throw_63thcolumn', '171throw_196thcolumn', '125throw_127thcolumn', '48throw_57thcolumn', '146throw_153thcolumn', '16throw_57thcolumn', '13throw_161thcolumn', '26throw_94thcolumn', '65throw_119thcolumn', '1throw_31thcolumn', '2throw_146thcolumn', '94throw_170thcolumn', '156throw_180thcolumn', '86throw_190thcolumn', '39throw_171thcolumn', '91throw_156thcolumn', '67throw_143thcolumn', '10throw_27thcolumn', '188throw_198thcolumn', '2throw_66thcolumn', 'SDQ_SDQ_Emotional_Problems', '14throw_29thcolumn', '176throw_190thcolumn', '77throw_120thcolumn', '63throw_152thcolumn', '83throw_167thcolumn', '27throw_37thcolumn', '7throw_83thcolumn', '40throw_167thcolumn', '2throw_136thcolumn', '90throw_184thcolumn', '34throw_44thcolumn', '41throw_196thcolumn', '0throw_117thcolumn', '159throw_188thcolumn', '80throw_190thcolumn', '138throw_173thcolumn', '82throw_123thcolumn', '143throw_157thcolumn', '8throw_147thcolumn', '159throw_187thcolumn', '1throw_194thcolumn', '129throw_162thcolumn', '139throw_197thcolumn', '113throw_150thcolumn', '26throw_76thcolumn', '48throw_89thcolumn', '72throw_163thcolumn', '38throw_158thcolumn', '18throw_78thcolumn', '64throw_111thcolumn', '65throw_130thcolumn', '147throw_167thcolumn', '12throw_16thcolumn', '90throw_159thcolumn', '164throw_177thcolumn', '56throw_152thcolumn', '15throw_60thcolumn', '100throw_195thcolumn', '21throw_147thcolumn', '155throw_192thcolumn', '183throw_192thcolumn', '4throw_8thcolumn', '33throw_69thcolumn', '56throw_61thcolumn', '83throw_123thcolumn', '21throw_22thcolumn', '68throw_98thcolumn', '57throw_165thcolumn', '9throw_98thcolumn', '127throw_185thcolumn', '113throw_131thcolumn', '13throw_169thcolumn', '157throw_171thcolumn', '119throw_143thcolumn', '16throw_34thcolumn', '124throw_154thcolumn', '117throw_192thcolumn', '75throw_151thcolumn', '51throw_156thcolumn', '30throw_96thcolumn', '21throw_107thcolumn', '14throw_156thcolumn', '47throw_171thcolumn', '96throw_188thcolumn', '19throw_46thcolumn', '20throw_119thcolumn', '16throw_130thcolumn', '68throw_160thcolumn', '44throw_167thcolumn', '40throw_105thcolumn', '6throw_26thcolumn', '53throw_161thcolumn', '8throw_185thcolumn', '9throw_187thcolumn', '9throw_105thcolumn', '17throw_156thcolumn', '34throw_123thcolumn', '36throw_108thcolumn', '73throw_148thcolumn', '32throw_62thcolumn', '113throw_115thcolumn', '70throw_110thcolumn', '88throw_160thcolumn', '47throw_146thcolumn', '140throw_175thcolumn', '130throw_154thcolumn', '49throw_181thcolumn', '155throw_181thcolumn', '99throw_172thcolumn', '33throw_93thcolumn', '49throw_194thcolumn', '58throw_128thcolumn', '150throw_166thcolumn', '51throw_83thcolumn', '52throw_68thcolumn', '34throw_150thcolumn', '140throw_145thcolumn', '1throw_22thcolumn', '4throw_192thcolumn', '14throw_102thcolumn', '103throw_121thcolumn', '45throw_162thcolumn', '36throw_181thcolumn', '88throw_185thcolumn', '80throw_97thcolumn', '71throw_72thcolumn', '20throw_183thcolumn', '103throw_128thcolumn', '110throw_165thcolumn', '124throw_192thcolumn', '23throw_47thcolumn', '129throw_196thcolumn', '93throw_197thcolumn', '118throw_154thcolumn', '101throw_153thcolumn', '134throw_161thcolumn', '131throw_174thcolumn', '38throw_145thcolumn', '116throw_192thcolumn', '62throw_167thcolumn', '125throw_176thcolumn', '76throw_89thcolumn', '174throw_181thcolumn', '177throw_196thcolumn', '41throw_124thcolumn', '20throw_160thcolumn', '104throw_137thcolumn', '130throw_147thcolumn', '181throw_192thcolumn', '66throw_156thcolumn', '11throw_124thcolumn', '155throw_175thcolumn', '0throw_197thcolumn', '55throw_160thcolumn', '46throw_149thcolumn', '61throw_107thcolumn', '59throw_191thcolumn', '68throw_154thcolumn', '17throw_67thcolumn', '46throw_143thcolumn', '36throw_193thcolumn', '185throw_191thcolumn', '46throw_58thcolumn', '10throw_154thcolumn', '26throw_85thcolumn', '93throw_185thcolumn', '152throw_161thcolumn', '48throw_73thcolumn', '39throw_174thcolumn', '53throw_157thcolumn', '76throw_96thcolumn', '74throw_82thcolumn', '161throw_175thcolumn', '21throw_136thcolumn', '29throw_84thcolumn', '36throw_151thcolumn', '8throw_196thcolumn', '62throw_122thcolumn', '111throw_177thcolumn', '153throw_168thcolumn', '78throw_155thcolumn', '48throw_139thcolumn', '105throw_137thcolumn', '162throw_180thcolumn', '25throw_124thcolumn', '0throw_145thcolumn', '14throw_58thcolumn', '27throw_34thcolumn', '155throw_193thcolumn', '134throw_177thcolumn', '62throw_142thcolumn', '44throw_173thcolumn', '26throw_162thcolumn', '179throw_193thcolumn', '14throw_140thcolumn', '25throw_91thcolumn', '23throw_189thcolumn', '21throw_90thcolumn', '68throw_152thcolumn', '145throw_172thcolumn', '104throw_152thcolumn', '52throw_193thcolumn', '70throw_149thcolumn', '52throw_75thcolumn', '192throw_193thcolumn', '104throw_166thcolumn', '47throw_67thcolumn', '150throw_175thcolumn', '8throw_45thcolumn', '28throw_95thcolumn', '118throw_192thcolumn', '74throw_110thcolumn', '27throw_182thcolumn', '80throw_163thcolumn', '12throw_83thcolumn', '161throw_177thcolumn', '91throw_197thcolumn', '17throw_108thcolumn', '130throw_190thcolumn', '99throw_198thcolumn', '4throw_19thcolumn', '9throw_180thcolumn', '7throw_187thcolumn', '22throw_89thcolumn', '124throw_194thcolumn', '58throw_170thcolumn', '18throw_167thcolumn', '72throw_153thcolumn', '51throw_144thcolumn', '2throw_184thcolumn', '43throw_141thcolumn', '113throw_182thcolumn', '85throw_164thcolumn', '3throw_112thcolumn', '21throw_84thcolumn', '26throw_157thcolumn', '19throw_74thcolumn', '63throw_82thcolumn', '139throw_160thcolumn', '82throw_197thcolumn', '36throw_180thcolumn', '48throw_58thcolumn', '102throw_113thcolumn', '105throw_173thcolumn']

Deneme4=['140throw_158thcolumn', '151throw_192thcolumn', '22throw_68thcolumn', '11throw_155thcolumn', '163throw_193thcolumn', '12throw_152thcolumn', 'SDQ_SDQ_Conduct_Problems', '180throw_187thcolumn', '25throw_93thcolumn', '69throw_106thcolumn', '120throw_168thcolumn', '87throw_172thcolumn', '143throw_153thcolumn', '17throw_55thcolumn', '171throw_189thcolumn', '96throw_165thcolumn', '26throw_129thcolumn', '135throw_181thcolumn', '132throw_138thcolumn', '28throw_122thcolumn', '15throw_19thcolumn', '152throw_167thcolumn', '107throw_182thcolumn', '72throw_115thcolumn', '113throw_130thcolumn', '49throw_120thcolumn', '29throw_55thcolumn', '97throw_157thcolumn', '59throw_90thcolumn', '90throw_130thcolumn', '9throw_34thcolumn', '2throw_166thcolumn', '58throw_71thcolumn', '29throw_70thcolumn', '123throw_178thcolumn', '19throw_141thcolumn', '109throw_166thcolumn', '27throw_60thcolumn', '173throw_197thcolumn', '14throw_67thcolumn', '63throw_105thcolumn', '75throw_81thcolumn', '29throw_66thcolumn', '63throw_69thcolumn', '54throw_197thcolumn', '138throw_177thcolumn', '1throw_113thcolumn', '15throw_132thcolumn', '14throw_112thcolumn', '55throw_60thcolumn', '31throw_91thcolumn', '18throw_187thcolumn', '77throw_78thcolumn', '146throw_192thcolumn', '146throw_173thcolumn', '71throw_193thcolumn', '137throw_176thcolumn', '113throw_168thcolumn', '170throw_171thcolumn', '65throw_183thcolumn', '121throw_179thcolumn', '70throw_142thcolumn', '104throw_155thcolumn', '62throw_192thcolumn', '61throw_97thcolumn', '14throw_21thcolumn', '48throw_67thcolumn', '16throw_71thcolumn', '48throw_115thcolumn', '69throw_94thcolumn', '111throw_163thcolumn', '110throw_151thcolumn', '171throw_183thcolumn', '52throw_124thcolumn', '49throw_101thcolumn', '161throw_178thcolumn', '74throw_88thcolumn', '88throw_142thcolumn', '40throw_185thcolumn', '65throw_103thcolumn', '135throw_148thcolumn', '47throw_114thcolumn', '88throw_118thcolumn', '0throw_34thcolumn', '140throw_155thcolumn', '13throw_67thcolumn', '63throw_184thcolumn', '75throw_138thcolumn', '10throw_136thcolumn', '29throw_77thcolumn', '132throw_180thcolumn', 'Barratt_Barratt_P2_Edu', '111throw_143thcolumn', '116throw_173thcolumn', '114throw_174thcolumn', '134throw_194thcolumn', '154throw_195thcolumn', '114throw_126thcolumn', '20throw_127thcolumn', '124throw_193thcolumn', '89throw_130thcolumn', '30throw_94thcolumn', '16throw_56thcolumn', '148throw_162thcolumn', '1throw_165thcolumn', '44throw_190thcolumn', '79throw_110thcolumn', '3throw_146thcolumn', '20throw_24thcolumn', '174throw_196thcolumn', '93throw_129thcolumn', '93throw_118thcolumn', '6throw_73thcolumn', '53throw_125thcolumn', '19throw_33thcolumn', '41throw_60thcolumn', '0throw_186thcolumn', '119throw_167thcolumn', '90throw_167thcolumn', '138throw_144thcolumn', '33throw_143thcolumn', '170throw_193thcolumn', '61throw_144thcolumn', '104throw_149thcolumn', '100throw_142thcolumn', '77throw_82thcolumn', '12throw_27thcolumn', '132throw_161thcolumn', '35throw_116thcolumn', '27throw_56thcolumn', '24throw_168thcolumn', '151throw_181thcolumn', '58throw_138thcolumn', '12throw_55thcolumn', '157throw_175thcolumn', '96throw_144thcolumn', '31throw_62thcolumn', '83throw_130thcolumn', '41throw_90thcolumn', '54throw_159thcolumn', '95throw_192thcolumn', '14throw_82thcolumn', '137throw_186thcolumn', '0throw_91thcolumn', 'Barratt_Barratt_P2_Occ', '11throw_168thcolumn', '69throw_99thcolumn', '11throw_37thcolumn', '5throw_38thcolumn', '28throw_111thcolumn', '92throw_129thcolumn', '17throw_163thcolumn', '161throw_167thcolumn', '22throw_175thcolumn', '20throw_55thcolumn', '61throw_114thcolumn', '155throw_179thcolumn', '144throw_152thcolumn', '51throw_151thcolumn', '81throw_132thcolumn', '22throw_61thcolumn', '101throw_187thcolumn', '66throw_178thcolumn', '16throw_120thcolumn', '60throw_119thcolumn', '117throw_121thcolumn', '61throw_84thcolumn', '57throw_60thcolumn', '89throw_99thcolumn', '170throw_178thcolumn', '36throw_71thcolumn', '88throw_134thcolumn', '136throw_150thcolumn', '70throw_164thcolumn', '139throw_179thcolumn', '51throw_166thcolumn', '28throw_137thcolumn', '128throw_160thcolumn', '30throw_142thcolumn', '7throw_38thcolumn', '22throw_70thcolumn', '54throw_167thcolumn', '24throw_178thcolumn', '17throw_85thcolumn', '87throw_114thcolumn', '5throw_107thcolumn', '57throw_150thcolumn', '19throw_59thcolumn', '84throw_89thcolumn', '24throw_167thcolumn', '61throw_178thcolumn', '8throw_10thcolumn', '6throw_147thcolumn', '124throw_178thcolumn', '70throw_194thcolumn', '143throw_146thcolumn', '2throw_71thcolumn', '114throw_152thcolumn', '51throw_110thcolumn', '99throw_159thcolumn', '70throw_163thcolumn', '70throw_122thcolumn', '37throw_120thcolumn', '57throw_59thcolumn', '174throw_182thcolumn', '29throw_58thcolumn', '32throw_143thcolumn', '86throw_102thcolumn', '28throw_183thcolumn', '2throw_193thcolumn', '64throw_104thcolumn', '27throw_110thcolumn', '87throw_97thcolumn', '106throw_161thcolumn', '35throw_165thcolumn', '63throw_107thcolumn', '45throw_192thcolumn', '184throw_194thcolumn', '76throw_77thcolumn', '48throw_72thcolumn', '2throw_96thcolumn', '21throw_185thcolumn', '121throw_166thcolumn', '89throw_192thcolumn', '129throw_194thcolumn', '24throw_187thcolumn', '38throw_118thcolumn', '10throw_151thcolumn', '29throw_54thcolumn', '24throw_143thcolumn', '53throw_155thcolumn', '6throw_35thcolumn', '157throw_178thcolumn', '111throw_139thcolumn', '122throw_128thcolumn', '83throw_107thcolumn', '128throw_196thcolumn', '159throw_165thcolumn', '13throw_118thcolumn', '111throw_183thcolumn', '109throw_192thcolumn', '65throw_165thcolumn', '57throw_152thcolumn', '58throw_60thcolumn', '47throw_172thcolumn', '83throw_137thcolumn', '0throw_31thcolumn', '51throw_178thcolumn', '43throw_119thcolumn', '88throw_187thcolumn', '12throw_50thcolumn', '10throw_43thcolumn', '53throw_186thcolumn', '36throw_167thcolumn', '2throw_129thcolumn', '66throw_191thcolumn', '27throw_42thcolumn', '8throw_103thcolumn', '98throw_117thcolumn', '136throw_146thcolumn', '111throw_189thcolumn', '29throw_184thcolumn', '169throw_198thcolumn', '69throw_150thcolumn', '124throw_127thcolumn', '0throw_54thcolumn', '115throw_183thcolumn', '159throw_181thcolumn', '124throw_142thcolumn', '65throw_114thcolumn', '49throw_97thcolumn', '88throw_130thcolumn', '79throw_140thcolumn', '52throw_162thcolumn', '127throw_197thcolumn', '5throw_100thcolumn', '80throw_115thcolumn', '112throw_120thcolumn', '41throw_192thcolumn', '167throw_175thcolumn', '3throw_17thcolumn', '10throw_66thcolumn', '29throw_154thcolumn', '1throw_159thcolumn', '4throw_37thcolumn', '6throw_137thcolumn', '47throw_89thcolumn', '30throw_89thcolumn', '21throw_127thcolumn', '109throw_113thcolumn', '59throw_105thcolumn', '69throw_92thcolumn', '70throw_187thcolumn', '11throw_55thcolumn', '54throw_123thcolumn', '65throw_122thcolumn', '164throw_198thcolumn', '129throw_178thcolumn', '7throw_66thcolumn', '49throw_173thcolumn', '28throw_134thcolumn', '54throw_73thcolumn', '146throw_149thcolumn', '29throw_38thcolumn', '162throw_173thcolumn', '6throw_109thcolumn', '26throw_88thcolumn', '56throw_197thcolumn', '97throw_160thcolumn', '84throw_157thcolumn', '69throw_72thcolumn', '68throw_97thcolumn', '70throw_158thcolumn', '5throw_45thcolumn', '40throw_181thcolumn', '17throw_106thcolumn', '8throw_117thcolumn', '131throw_182thcolumn', '118throw_143thcolumn', '147throw_160thcolumn', '152throw_158thcolumn', '21throw_75thcolumn', '66throw_193thcolumn', '84throw_144thcolumn', '21throw_59thcolumn', '26throw_48thcolumn', '111throw_175thcolumn', '110throw_145thcolumn', '92throw_109thcolumn', '43throw_125thcolumn', '116throw_193thcolumn', '54throw_89thcolumn', '34throw_49thcolumn', '130throw_161thcolumn', '30throw_91thcolumn', '88throw_100thcolumn', '16throw_160thcolumn', '95throw_184thcolumn', '35throw_123thcolumn', '51throw_69thcolumn', '40throw_44thcolumn', '37throw_98thcolumn', '45throw_48thcolumn', '170throw_188thcolumn', '57throw_71thcolumn', '38throw_164thcolumn', '143throw_166thcolumn', '116throw_135thcolumn', '53throw_178thcolumn', '20throw_88thcolumn', '110throw_163thcolumn', '47throw_194thcolumn', '104throw_126thcolumn', '55throw_127thcolumn', '74throw_191thcolumn', '135throw_172thcolumn', '30throw_103thcolumn', '146throw_169thcolumn', '39throw_149thcolumn', '85throw_199thcolumn', '1throw_4thcolumn', '77throw_185thcolumn', '80throw_127thcolumn', '172throw_189thcolumn', '58throw_65thcolumn', '16throw_77thcolumn', '90throw_150thcolumn', '75throw_170thcolumn', '88throw_127thcolumn', '4throw_194thcolumn', '44throw_72thcolumn', '50throw_162thcolumn', '65throw_84thcolumn', '38throw_72thcolumn', '60throw_85thcolumn', '56throw_104thcolumn', '71throw_170thcolumn', '4throw_76thcolumn', '16throw_119thcolumn', '69throw_145thcolumn', '59throw_118thcolumn', '70throw_174thcolumn', '94throw_177thcolumn', '160throw_176thcolumn', '6throw_160thcolumn', '106throw_162thcolumn', '143throw_180thcolumn', '9throw_194thcolumn', '134throw_156thcolumn', '82throw_179thcolumn', '7throw_28thcolumn', '160throw_163thcolumn', '63throw_157thcolumn', '122throw_146thcolumn', '8throw_158thcolumn', '89throw_148thcolumn', '116throw_140thcolumn', '27throw_170thcolumn', '7throw_171thcolumn', '50throw_159thcolumn', '73throw_97thcolumn', '126throw_170thcolumn', '4throw_53thcolumn', '23throw_50thcolumn', '117throw_155thcolumn', '123throw_141thcolumn', '123throw_130thcolumn', '59throw_96thcolumn', '66throw_192thcolumn', '67throw_115thcolumn', '46throw_158thcolumn', '162throw_196thcolumn', '109throw_143thcolumn', '60throw_161thcolumn', '148throw_186thcolumn', '9throw_78thcolumn', '85throw_170thcolumn', '80throw_152thcolumn', '19throw_41thcolumn', '22throw_103thcolumn', '33throw_55thcolumn', '98throw_199thcolumn', '67throw_144thcolumn', '115throw_165thcolumn', '111throw_166thcolumn', '105throw_155thcolumn', '70throw_71thcolumn', '11throw_70thcolumn', '95throw_128thcolumn', '51throw_179thcolumn', '33throw_121thcolumn', '89throw_194thcolumn', '3throw_187thcolumn', '34throw_152thcolumn', '9throw_17thcolumn', '63throw_163thcolumn', '42throw_143thcolumn', '46throw_177thcolumn', '53throw_87thcolumn', '145throw_159thcolumn', '123throw_136thcolumn', '3throw_104thcolumn', '128throw_142thcolumn', '83throw_113thcolumn', '74throw_113thcolumn', '81throw_193thcolumn', '64throw_170thcolumn', '98throw_188thcolumn', '1throw_7thcolumn', '101throw_171thcolumn', '120throw_134thcolumn', '143throw_187thcolumn', '13throw_175thcolumn', '15throw_100thcolumn', '123throw_196thcolumn', '54throw_131thcolumn', '97throw_151thcolumn', '50throw_78thcolumn', '17throw_104thcolumn', '144throw_151thcolumn', '192throw_196thcolumn', '125throw_158thcolumn', '75throw_137thcolumn', '136throw_161thcolumn', '43throw_143thcolumn', '7throw_179thcolumn', '14throw_166thcolumn', '133throw_197thcolumn', '89throw_123thcolumn', '32throw_175thcolumn', '24throw_103thcolumn', '0throw_179thcolumn', '96throw_155thcolumn', '28throw_75thcolumn', '34throw_118thcolumn', '65throw_94thcolumn', '71throw_176thcolumn', '21throw_129thcolumn', '16throw_152thcolumn', '84throw_165thcolumn', '127throw_146thcolumn', '45throw_93thcolumn', '33throw_175thcolumn', '79throw_195thcolumn', '7throw_110thcolumn', '3throw_161thcolumn', '61throw_172thcolumn', '27throw_127thcolumn', '95throw_160thcolumn', '107throw_169thcolumn', '10throw_100thcolumn', '61throw_125thcolumn', '47throw_170thcolumn', '69throw_100thcolumn', '75throw_123thcolumn', '68throw_167thcolumn', '77throw_190thcolumn', '55throw_166thcolumn', '76throw_137thcolumn', '53throw_158thcolumn', '35throw_138thcolumn', '1throw_12thcolumn', '69throw_179thcolumn', '113throw_191thcolumn', '22throw_152thcolumn', '41throw_155thcolumn', '33throw_187thcolumn', '81throw_128thcolumn', '155throw_198thcolumn', '87throw_120thcolumn', '161throw_196thcolumn', '79throw_173thcolumn', '118throw_198thcolumn', '149throw_161thcolumn', '19throw_51thcolumn', '52throw_101thcolumn', '29throw_164thcolumn', '21throw_57thcolumn', '16throw_17thcolumn', '26throw_37thcolumn', '90throw_179thcolumn', '23throw_144thcolumn', '113throw_175thcolumn', '31throw_138thcolumn', '23throw_83thcolumn', '69throw_73thcolumn', '50throw_61thcolumn', '36throw_60thcolumn', '46throw_122thcolumn', '83throw_175thcolumn', '96throw_105thcolumn', '79throw_192thcolumn', '34throw_103thcolumn', '0throw_13thcolumn', '5throw_122thcolumn', '14throw_98thcolumn', '134throw_187thcolumn', '5throw_76thcolumn', '114throw_155thcolumn', '112throw_181thcolumn', '140throw_192thcolumn', '136throw_194thcolumn', '7throw_109thcolumn', '47throw_161thcolumn', '33throw_199thcolumn', '139throw_155thcolumn', '132throw_163thcolumn', '55throw_82thcolumn', '124throw_138thcolumn', '57throw_161thcolumn', '10throw_67thcolumn', '154throw_184thcolumn', '73throw_174thcolumn', '51throw_192thcolumn', '57throw_65thcolumn', '77throw_179thcolumn', '23throw_94thcolumn', '77throw_166thcolumn', '34throw_100thcolumn', '32throw_73thcolumn', '2throw_52thcolumn', '85throw_179thcolumn', '135throw_180thcolumn', '27throw_52thcolumn', '58throw_97thcolumn', '53throw_188thcolumn', '87throw_176thcolumn', '72throw_155thcolumn', '19throw_113thcolumn', '135throw_151thcolumn', '110throw_158thcolumn', '12throw_190thcolumn', '77throw_153thcolumn', '26throw_198thcolumn', '72throw_80thcolumn', '61throw_67thcolumn', '149throw_150thcolumn', '71throw_80thcolumn', '80throw_157thcolumn', '41throw_57thcolumn', '17throw_28thcolumn', '57throw_137thcolumn', '53throw_148thcolumn', '50throw_155thcolumn', '40throw_65thcolumn', '173throw_188thcolumn', '63throw_92thcolumn', '66throw_84thcolumn', '80throw_191thcolumn', '107throw_180thcolumn', '68throw_105thcolumn', '8throw_106thcolumn', '26throw_156thcolumn', '44throw_187thcolumn', '20throw_53thcolumn', '29throw_142thcolumn', '57throw_82thcolumn', '41throw_165thcolumn', '38throw_55thcolumn', '125throw_174thcolumn', '10throw_93thcolumn', '6throw_53thcolumn', '102throw_186thcolumn', '8throw_105thcolumn', '27throw_163thcolumn', '3throw_109thcolumn', '146throw_170thcolumn', '121throw_167thcolumn', '23throw_148thcolumn', '125throw_171thcolumn', '78throw_90thcolumn', '33throw_181thcolumn', '46throw_144thcolumn', '72throw_199thcolumn', '99throw_128thcolumn', '30throw_134thcolumn', '40throw_45thcolumn', '113throw_163thcolumn', '33throw_42thcolumn', '79throw_186thcolumn', '71throw_152thcolumn', '66throw_186thcolumn', '17throw_141thcolumn', '70throw_152thcolumn', '45throw_58thcolumn', '97throw_161thcolumn', '9throw_191thcolumn', '1throw_57thcolumn', '27throw_165thcolumn', '86throw_127thcolumn', '4throw_69thcolumn', '11throw_141thcolumn', '70throw_162thcolumn', '88throw_133thcolumn', '0throw_135thcolumn', '124throw_181thcolumn', '19throw_159thcolumn', '90throw_154thcolumn', '5throw_67thcolumn', '19throw_67thcolumn', '54throw_160thcolumn', '84throw_156thcolumn', '29throw_59thcolumn', '92throw_176thcolumn', '120throw_184thcolumn', '131throw_155thcolumn', '11throw_91thcolumn', '43throw_147thcolumn', '50throw_62thcolumn', '117throw_125thcolumn', '26throw_47thcolumn', '48throw_61thcolumn', '112throw_191thcolumn', '167throw_179thcolumn', '171throw_185thcolumn', '2throw_123thcolumn', '121throw_197thcolumn', '10throw_117thcolumn', '123throw_152thcolumn', '57throw_154thcolumn', '45throw_167thcolumn', '7throw_68thcolumn', '26throw_154thcolumn', '91throw_137thcolumn', '77throw_132thcolumn', '3throw_36thcolumn', '111throw_133thcolumn', '112throw_177thcolumn', '149throw_195thcolumn', '11throw_174thcolumn', '171throw_177thcolumn', '26throw_123thcolumn', '142throw_181thcolumn', '61throw_165thcolumn', '80throw_180thcolumn', '43throw_133thcolumn', '35throw_36thcolumn', '143throw_164thcolumn', '64throw_166thcolumn', '52throw_166thcolumn', '74throw_98thcolumn', '5throw_109thcolumn', '72throw_171thcolumn', '47throw_181thcolumn', '72throw_151thcolumn', '171throw_197thcolumn', '176throw_198thcolumn', '82throw_86thcolumn', '21throw_141thcolumn', '120throw_167thcolumn', '131throw_134thcolumn', '6throw_51thcolumn', '36throw_185thcolumn', '49throw_71thcolumn', '4throw_198thcolumn', '37throw_167thcolumn', '44throw_56thcolumn', '163throw_191thcolumn', '7throw_27thcolumn', '82throw_140thcolumn', '54throw_149thcolumn', '150throw_161thcolumn', '164throw_181thcolumn', '159throw_183thcolumn', '43throw_85thcolumn', '98throw_187thcolumn', '51throw_159thcolumn', '74throw_172thcolumn', '115throw_176thcolumn', '58throw_59thcolumn', '97throw_165thcolumn', '51throw_165thcolumn', '12throw_63thcolumn', '65throw_162thcolumn', '45throw_46thcolumn', '0throw_56thcolumn', '46throw_133thcolumn', '29throw_165thcolumn', '17throw_59thcolumn', '71throw_155thcolumn', '93throw_149thcolumn', '76throw_155thcolumn', '124throw_151thcolumn', '83throw_100thcolumn', '58throw_196thcolumn', '34throw_127thcolumn', '20throw_148thcolumn', '55throw_84thcolumn', '25throw_47thcolumn', '104throw_170thcolumn', '32throw_119thcolumn', '58throw_66thcolumn', '58throw_174thcolumn', '175throw_184thcolumn', '155throw_191thcolumn', '39throw_40thcolumn', '78throw_97thcolumn', '3throw_40thcolumn', '17throw_53thcolumn', '111throw_126thcolumn', '41throw_96thcolumn', '41throw_81thcolumn', '49throw_193thcolumn', '51throw_147thcolumn', '116throw_119thcolumn', '145throw_147thcolumn', '138throw_163thcolumn', '93throw_163thcolumn', '137throw_161thcolumn', '120throw_145thcolumn', '106throw_184thcolumn', '113throw_129thcolumn', '21throw_197thcolumn', '48throw_138thcolumn', '60throw_84thcolumn', '25throw_27thcolumn', '164throw_170thcolumn', '106throw_107thcolumn', 'Barratt_Barratt_P1_Edu', '160throw_187thcolumn', '55throw_140thcolumn', '95throw_121thcolumn', '106throw_117thcolumn', '154throw_165thcolumn', '27throw_47thcolumn', '128throw_198thcolumn', '148throw_164thcolumn', '186throw_196thcolumn', '66throw_98thcolumn', '38throw_125thcolumn', '38throw_190thcolumn', '99throw_199thcolumn', '73throw_146thcolumn', '34throw_194thcolumn', '0throw_11thcolumn', '74throw_79thcolumn', '78throw_171thcolumn', '159throw_180thcolumn', '7throw_30thcolumn', '9throw_66thcolumn', '0throw_38thcolumn', '28throw_43thcolumn', '110throw_184thcolumn', '80throw_93thcolumn', '103throw_179thcolumn', '47throw_57thcolumn', '144throw_167thcolumn', '47throw_52thcolumn', '95throw_107thcolumn', '135throw_153thcolumn', '19throw_50thcolumn', '184throw_191thcolumn', '40throw_57thcolumn', '133throw_156thcolumn', '134throw_141thcolumn', '118throw_120thcolumn', '78throw_130thcolumn', '0throw_192thcolumn', '12throw_102thcolumn', '137throw_191thcolumn', '97throw_199thcolumn', '53throw_77thcolumn', '96throw_110thcolumn', '47throw_102thcolumn', '73throw_109thcolumn', '26throw_103thcolumn', '47throw_190thcolumn', '106throw_159thcolumn', '31throw_92thcolumn', '72throw_176thcolumn', '86throw_167thcolumn', '53throw_187thcolumn', '63throw_91thcolumn', '48throw_68thcolumn', '6throw_161thcolumn', '83throw_115thcolumn', '53throw_122thcolumn', '34throw_108thcolumn', '170throw_195thcolumn', '164throw_172thcolumn', '79throw_152thcolumn', '159throw_178thcolumn', '69throw_129thcolumn', '84throw_173thcolumn', '47throw_163thcolumn', '39throw_128thcolumn', '8throw_152thcolumn', '12throw_58thcolumn', '174throw_187thcolumn', '46throw_61thcolumn', '120throw_126thcolumn', '37throw_170thcolumn', '2throw_106thcolumn', '129throw_191thcolumn', '39throw_97thcolumn', '23throw_76thcolumn', '33throw_126thcolumn', '171throw_178thcolumn', '117throw_152thcolumn', '84throw_192thcolumn', '17throw_162thcolumn', '69throw_95thcolumn', '74throw_167thcolumn', '16throw_114thcolumn', '55throw_83thcolumn', '25throw_83thcolumn', '130throw_160thcolumn', '108throw_180thcolumn', '123throw_128thcolumn', '95throw_193thcolumn', '160throw_188thcolumn', '54throw_157thcolumn', '157throw_191thcolumn', '47throw_123thcolumn', '27throw_122thcolumn', '84throw_128thcolumn', '140throw_151thcolumn', '73throw_92thcolumn', '38throw_163thcolumn', '151throw_168thcolumn', '88throw_159thcolumn', '83throw_187thcolumn', '0throw_30thcolumn', '98throw_192thcolumn', '169throw_183thcolumn', '120throw_197thcolumn', '49throw_185thcolumn', '55throw_67thcolumn', '25throw_68thcolumn', '39throw_45thcolumn', '134throw_138thcolumn', '120throw_191thcolumn', '133throw_153thcolumn', '88throw_190thcolumn', '134throw_173thcolumn', '33throw_144thcolumn', '8throw_59thcolumn', '16throw_42thcolumn', '26throw_197thcolumn', '107throw_193thcolumn', '100throw_140thcolumn', '60throw_103thcolumn', '29throw_43thcolumn', '7throw_20thcolumn', '0throw_1thcolumn', '97throw_144thcolumn', '85throw_183thcolumn', '54throw_75thcolumn', '96throw_191thcolumn', '77throw_124thcolumn', '155throw_197thcolumn', '65throw_151thcolumn', '90throw_124thcolumn', 'SDQ_SDQ_Internalizing', '74throw_93thcolumn', '38throw_83thcolumn', '22throw_77thcolumn', '151throw_187thcolumn', '100throw_134thcolumn', '13throw_65thcolumn', '8throw_157thcolumn', '64throw_167thcolumn', '40throw_164thcolumn', '57throw_125thcolumn', '21throw_70thcolumn', '138throw_196thcolumn', '59throw_190thcolumn', '169throw_177thcolumn', '98throw_115thcolumn', '9throw_69thcolumn', '80throw_122thcolumn', '16throw_161thcolumn', '49throw_130thcolumn', '82throw_182thcolumn', '34throw_74thcolumn', '89throw_163thcolumn', '60throw_104thcolumn', '17throw_42thcolumn', '154throw_198thcolumn', '37throw_110thcolumn', '19throw_31thcolumn', '87throw_160thcolumn', '127throw_175thcolumn', '124throw_143thcolumn', '39throw_93thcolumn', '36throw_97thcolumn', '10throw_128thcolumn', '24throw_185thcolumn', '28throw_110thcolumn', '85throw_126thcolumn', '32throw_68thcolumn', '53throw_121thcolumn', '97throw_162thcolumn', '72throw_119thcolumn', '98throw_127thcolumn', '159throw_172thcolumn', '28throw_63thcolumn', '31throw_70thcolumn', '17throw_119thcolumn', '74throw_166thcolumn', '133throw_178thcolumn', '118throw_123thcolumn', '130throw_169thcolumn', '13throw_30thcolumn', '47throw_53thcolumn', '150throw_158thcolumn', '121throw_154thcolumn', '152throw_192thcolumn', '130throw_184thcolumn', '153throw_187thcolumn', '80throw_156thcolumn', '70throw_173thcolumn', '2throw_27thcolumn', '39throw_44thcolumn', '11throw_99thcolumn', '177throw_187thcolumn', '185throw_193thcolumn', '78throw_192thcolumn', '97throw_169thcolumn', '14throw_51thcolumn', '84throw_113thcolumn', '23throw_161thcolumn', '131throw_197thcolumn', '17throw_190thcolumn', '28throw_41thcolumn', '124throw_146thcolumn', '112throw_137thcolumn', '54throw_178thcolumn', '52throw_147thcolumn', '35throw_195thcolumn', '95throw_159thcolumn', '50throw_52thcolumn', '27throw_198thcolumn', '56throw_189thcolumn', '98throw_123thcolumn', '27throw_113thcolumn', '9throw_186thcolumn', '96throw_108thcolumn', '121throw_168thcolumn', '68throw_158thcolumn', '147throw_184thcolumn', '118throw_190thcolumn', '145throw_197thcolumn']

Deneme5=['4throw_74thcolumn', '141throw_147thcolumn', '136throw_174thcolumn', '128throw_197thcolumn', '124throw_168thcolumn', '7throw_158thcolumn', '162throw_177thcolumn', '33throw_170thcolumn', '167throw_170thcolumn', '3throw_157thcolumn', '84throw_136thcolumn', '22throw_177thcolumn', '61throw_93thcolumn', '15throw_125thcolumn', '89throw_129thcolumn', '43throw_182thcolumn', '117throw_122thcolumn', '49throw_124thcolumn', 'SDQ_SDQ_Difficulties_Total', '68throw_109thcolumn', '38throw_62thcolumn', '165throw_181thcolumn', '47throw_63thcolumn', '58throw_69thcolumn', '26throw_51thcolumn', '69throw_82thcolumn', '32throw_66thcolumn', '30throw_159thcolumn', '1throw_93thcolumn', '27throw_137thcolumn', '1throw_95thcolumn', '106throw_179thcolumn', '67throw_156thcolumn', '36throw_44thcolumn', '178throw_184thcolumn', '134throw_181thcolumn', '23throw_156thcolumn', '131throw_147thcolumn', '73throw_168thcolumn', '87throw_127thcolumn', '69throw_160thcolumn', '142throw_186thcolumn', '93throw_114thcolumn', '5throw_86thcolumn', '80throw_143thcolumn', '156throw_158thcolumn', '3throw_130thcolumn', '55throw_132thcolumn', '128throw_175thcolumn', '27throw_195thcolumn', '33throw_100thcolumn', '72throw_108thcolumn', '26throw_75thcolumn', '26throw_79thcolumn', '9throw_37thcolumn', '74throw_164thcolumn', '38throw_51thcolumn', '24throw_48thcolumn', '40throw_142thcolumn', '140throw_196thcolumn', '160throw_175thcolumn', '121throw_184thcolumn', '170throw_177thcolumn', '104throw_184thcolumn', '66throw_139thcolumn', '76throw_168thcolumn', '35throw_137thcolumn', '55throw_198thcolumn', '96throw_143thcolumn', '20throw_115thcolumn', '112throw_196thcolumn', '90throw_183thcolumn', '28throw_105thcolumn', '14throw_33thcolumn', '50throw_88thcolumn', '122throw_166thcolumn', '39throw_86thcolumn', '95throw_182thcolumn', '41throw_190thcolumn', '63throw_150thcolumn', '50throw_90thcolumn', '170throw_186thcolumn', '21throw_188thcolumn', '77throw_186thcolumn', '46throw_192thcolumn', '17throw_189thcolumn', '12throw_37thcolumn', '57throw_124thcolumn', '141throw_175thcolumn', '175throw_199thcolumn', '106throw_156thcolumn', '56throw_170thcolumn', '19throw_132thcolumn', '6throw_97thcolumn', '107throw_197thcolumn', '46throw_56thcolumn', '33throw_165thcolumn', '37throw_171thcolumn', '129throw_150thcolumn', '53throw_82thcolumn', '14throw_138thcolumn', '80throw_141thcolumn', '161throw_176thcolumn', '39throw_157thcolumn', '96throw_109thcolumn', '10throw_88thcolumn', '43throw_153thcolumn', '28throw_129thcolumn', '54throw_56thcolumn', '135throw_158thcolumn', '118throw_193thcolumn', '131throw_167thcolumn', '15throw_48thcolumn', '8throw_107thcolumn', '18throw_50thcolumn', '169throw_189thcolumn', '106throw_109thcolumn', '86throw_88thcolumn', '35throw_104thcolumn', '75throw_176thcolumn', '25throw_88thcolumn', '178throw_199thcolumn', '3throw_111thcolumn', '78throw_80thcolumn', '54throw_174thcolumn', '45throw_52thcolumn', '24throw_80thcolumn', '27throw_112thcolumn', '126throw_156thcolumn', '72throw_189thcolumn', '161throw_182thcolumn', '20throw_178thcolumn', '52throw_73thcolumn', '36throw_192thcolumn', '106throw_175thcolumn', '38throw_138thcolumn', '129throw_192thcolumn', '132throw_134thcolumn', '148throw_150thcolumn', '57throw_181thcolumn', '3throw_89thcolumn', '108throw_164thcolumn', '72throw_101thcolumn', '20throw_140thcolumn', '66throw_167thcolumn', '67throw_138thcolumn', '143throw_190thcolumn', '92throw_122thcolumn', '67throw_162thcolumn', '15throw_27thcolumn', '26throw_42thcolumn', '1throw_96thcolumn', '29throw_95thcolumn', '79throw_85thcolumn', '172throw_174thcolumn', '137throw_185thcolumn', '56throw_57thcolumn', '12throw_189thcolumn', '31throw_44thcolumn', '53throw_67thcolumn', '56throw_102thcolumn', '168throw_170thcolumn', '31throw_52thcolumn', '47throw_162thcolumn', '106throw_157thcolumn', '73throw_129thcolumn', '48throw_178thcolumn', '188throw_195thcolumn', '0throw_181thcolumn', '22throw_80thcolumn', '54throw_176thcolumn', '36throw_177thcolumn', '149throw_189thcolumn', '57throw_182thcolumn', '1throw_104thcolumn', '57throw_148thcolumn', '106throw_193thcolumn', 'APQ_P_APQ_P_PM', '39throw_106thcolumn', '39throw_125thcolumn', '39throw_192thcolumn', '126throw_142thcolumn', '110throw_135thcolumn', '7throw_178thcolumn', '72throw_166thcolumn', '16throw_190thcolumn', '51throw_145thcolumn', '159throw_164thcolumn', '75throw_167thcolumn', '68throw_90thcolumn', '21throw_89thcolumn', '151throw_180thcolumn', '94throw_171thcolumn', '37throw_194thcolumn', '130throw_180thcolumn', '34throw_146thcolumn', '142throw_184thcolumn', '94throw_98thcolumn', '121throw_180thcolumn', 'Barratt_Barratt_P1_Occ', '82throw_88thcolumn', '0throw_183thcolumn', '168throw_185thcolumn', '18throw_66thcolumn', '120throw_164thcolumn', '3throw_90thcolumn', '25throw_61thcolumn', '112throw_150thcolumn', '78throw_112thcolumn', '93throw_169thcolumn', '66throw_81thcolumn', '65throw_174thcolumn', '43throw_150thcolumn', '172throw_173thcolumn', '70throw_195thcolumn', '33throw_38thcolumn', '81throw_158thcolumn', '13throw_94thcolumn', '156throw_159thcolumn', '81throw_92thcolumn', '22throw_148thcolumn', '65throw_81thcolumn', '6throw_170thcolumn', '60throw_187thcolumn', '115throw_174thcolumn', '44throw_124thcolumn', '31throw_102thcolumn', '5throw_134thcolumn', '1throw_48thcolumn', '29throw_190thcolumn', '47throw_195thcolumn', '38throw_84thcolumn', '13throw_113thcolumn', '162throw_169thcolumn', '137throw_187thcolumn', '38throw_89thcolumn', '31throw_57thcolumn', '142throw_199thcolumn', '120throw_125thcolumn', '24throw_135thcolumn', '71throw_77thcolumn', '76throw_86thcolumn', '78throw_193thcolumn', '28throw_39thcolumn', '60throw_66thcolumn', '91throw_96thcolumn', '7throw_19thcolumn', '171throw_193thcolumn', '53throw_71thcolumn', '21throw_30thcolumn', '38throw_98thcolumn', '48throw_132thcolumn', '58throw_141thcolumn', '32throw_56thcolumn', '37throw_182thcolumn', '19throw_180thcolumn', '30throw_45thcolumn', '152throw_195thcolumn', '18throw_38thcolumn', '22throw_62thcolumn', '34throw_46thcolumn', '115throw_142thcolumn', '99throw_178thcolumn', '161throw_168thcolumn', '135throw_177thcolumn', '94throw_129thcolumn', '193throw_199thcolumn', '70throw_134thcolumn', '37throw_101thcolumn', '0throw_114thcolumn', '13throw_86thcolumn', '24throw_51thcolumn', '136throw_143thcolumn', '46throw_70thcolumn', '115throw_121thcolumn', '114throw_125thcolumn', '124throw_174thcolumn', '96throw_136thcolumn', '53throw_93thcolumn', '120throw_161thcolumn', '21throw_66thcolumn', '108throw_187thcolumn', '32throw_78thcolumn', '148throw_177thcolumn', '16throw_92thcolumn', '93throw_97thcolumn', '16throw_84thcolumn', '163throw_174thcolumn', '138throw_166thcolumn', '78throw_174thcolumn', '17throw_131thcolumn', '91throw_182thcolumn', '7throw_51thcolumn', '66throw_111thcolumn', '18throw_41thcolumn', '79throw_112thcolumn', '99throw_130thcolumn', '77throw_150thcolumn', '85throw_131thcolumn', '130throw_162thcolumn', '47throw_191thcolumn', '69throw_97thcolumn', '12throw_162thcolumn', '3throw_126thcolumn', '124throw_125thcolumn', '7throw_140thcolumn', '33throw_101thcolumn', '11throw_177thcolumn', '75throw_156thcolumn', '100throw_110thcolumn', '0throw_7thcolumn', '8throw_63thcolumn', '153throw_176thcolumn', '46throw_108thcolumn', '151throw_195thcolumn', '59throw_130thcolumn', '58throw_102thcolumn', '1throw_42thcolumn', '122throw_169thcolumn', '71throw_124thcolumn', '58throw_107thcolumn', '36throw_51thcolumn', '44throw_139thcolumn', '68throw_186thcolumn', '12throw_65thcolumn', '41throw_145thcolumn', '11throw_166thcolumn', '125throw_147thcolumn', '192throw_195thcolumn', '160throw_162thcolumn', '45throw_154thcolumn', '149throw_183thcolumn', '31throw_158thcolumn', '68throw_88thcolumn', '8throw_37thcolumn', '24throw_175thcolumn', '54throw_92thcolumn', '52throw_100thcolumn', '70throw_160thcolumn', '56throw_73thcolumn', '130throw_178thcolumn', '153throw_197thcolumn', '38throw_99thcolumn', '112throw_114thcolumn', '3throw_22thcolumn', '22throw_196thcolumn', '6throw_70thcolumn', '140throw_191thcolumn', '1throw_126thcolumn', '1throw_168thcolumn', '115throw_186thcolumn', '48throw_164thcolumn', '131throw_194thcolumn', '111throw_161thcolumn', '4throw_95thcolumn', '49throw_170thcolumn', '127throw_156thcolumn', '30throw_46thcolumn', '15throw_70thcolumn', '64throw_127thcolumn', '45throw_145thcolumn', '34throw_78thcolumn', '131throw_172thcolumn', '145throw_181thcolumn', '80throw_139thcolumn', '122throw_194thcolumn', '48throw_172thcolumn', '92throw_190thcolumn', '103throw_156thcolumn', '103throw_188thcolumn', '70throw_181thcolumn', '3throw_61thcolumn', '29throw_104thcolumn', '50throw_87thcolumn', '18throw_161thcolumn', '8throw_92thcolumn', '12throw_175thcolumn', '30throw_197thcolumn', '68throw_79thcolumn', '0throw_137thcolumn', '24throw_177thcolumn', '15throw_78thcolumn', '1throw_120thcolumn', '79throw_91thcolumn', '23throw_128thcolumn', '27throw_192thcolumn', '75throw_168thcolumn', '116throw_149thcolumn', '76throw_127thcolumn', '27throw_45thcolumn', '5throw_193thcolumn', '70throw_85thcolumn', '56throw_87thcolumn', '65throw_83thcolumn', '26throw_68thcolumn', '180throw_199thcolumn', '96throw_179thcolumn', '7throw_46thcolumn', '15throw_39thcolumn', '113throw_117thcolumn', '69throw_109thcolumn', '13throw_56thcolumn', '75throw_190thcolumn', '119throw_151thcolumn', '41throw_74thcolumn', '18throw_115thcolumn', '37throw_111thcolumn', '76throw_78thcolumn', '13throw_119thcolumn', '89throw_150thcolumn', '31throw_46thcolumn', '14throw_118thcolumn', '50throw_184thcolumn', '83throw_186thcolumn', '67throw_77thcolumn', '80throw_168thcolumn', '161throw_183thcolumn', '21throw_117thcolumn', '35throw_160thcolumn', '123throw_159thcolumn', '97throw_155thcolumn', '11throw_63thcolumn', '38throw_140thcolumn', '7throw_47thcolumn', '5throw_191thcolumn', '108throw_157thcolumn', '50throw_156thcolumn', '51throw_131thcolumn', '97throw_121thcolumn', '79throw_109thcolumn', '13throw_137thcolumn', '110throw_142thcolumn', '52throw_80thcolumn', '5throw_183thcolumn', '80throw_101thcolumn', '24throw_182thcolumn', '18throw_152thcolumn', '62throw_87thcolumn', '139throw_167thcolumn', '68throw_75thcolumn', '89throw_142thcolumn', '12throw_75thcolumn', '84throw_170thcolumn', '84throw_160thcolumn', '39throw_71thcolumn', '123throw_176thcolumn', '167throw_197thcolumn', '91throw_104thcolumn', '21throw_81thcolumn', '63throw_114thcolumn', '72throw_110thcolumn', '166throw_178thcolumn', '70throw_168thcolumn', '13throw_69thcolumn', '90throw_195thcolumn', '66throw_150thcolumn', '98throw_184thcolumn', '4throw_75thcolumn', '104throw_146thcolumn', '104throw_136thcolumn', '55throw_183thcolumn', '53throw_133thcolumn', '39throw_87thcolumn', '101throw_157thcolumn', '6throw_121thcolumn', '97throw_148thcolumn', '171throw_180thcolumn', '107throw_159thcolumn', '43throw_73thcolumn', '51throw_70thcolumn', '101throw_107thcolumn', '50throw_154thcolumn', '168throw_197thcolumn', '69throw_190thcolumn', '57throw_117thcolumn', '44throw_120thcolumn', '110throw_179thcolumn', '100throw_112thcolumn', '44throw_137thcolumn', '163throw_177thcolumn', '134throw_186thcolumn', '84throw_163thcolumn', '44throw_169thcolumn', '76throw_84thcolumn', '61throw_158thcolumn', '44throw_151thcolumn', '140throw_188thcolumn', '119throw_129thcolumn', '176throw_182thcolumn', '83throw_157thcolumn', '68throw_189thcolumn', '82throw_193thcolumn', '3throw_102thcolumn', '22throw_28thcolumn', '104throw_138thcolumn', '109throw_176thcolumn', '122throw_180thcolumn', '65throw_66thcolumn', '4throw_31thcolumn', '104throw_127thcolumn', '93throw_119thcolumn', '111throw_185thcolumn', '35throw_174thcolumn', '18throw_157thcolumn', '70throw_140thcolumn', '4throw_50thcolumn', '6throw_100thcolumn', '75throw_133thcolumn', '92throw_157thcolumn', '15throw_42thcolumn', '66throw_82thcolumn', '4throw_163thcolumn', '5throw_13thcolumn', '28throw_131thcolumn', '88throw_96thcolumn', '57throw_101thcolumn', '2throw_113thcolumn', '24throw_151thcolumn', '61throw_150thcolumn', '10throw_64thcolumn', '94throw_173thcolumn', '53throw_179thcolumn', '90throw_158thcolumn', '57throw_78thcolumn', '8throw_71thcolumn', '89throw_177thcolumn', '40throw_72thcolumn', '4throw_72thcolumn', '166throw_176thcolumn', '108throw_150thcolumn', '177throw_179thcolumn', '27throw_38thcolumn', '87throw_163thcolumn', '125throw_175thcolumn', '90throw_191thcolumn', '30throw_184thcolumn', '22throw_69thcolumn', '7throw_101thcolumn', '90throw_185thcolumn', '2throw_62thcolumn', '49throw_53thcolumn', '93throw_102thcolumn', '53throw_106thcolumn', '32throw_54thcolumn', '9throw_179thcolumn', '15throw_143thcolumn', '97throw_170thcolumn', '72throw_84thcolumn', '96throw_175thcolumn', '90throw_186thcolumn', '95throw_177thcolumn', '21throw_62thcolumn', '21throw_50thcolumn', '59throw_146thcolumn', '41throw_194thcolumn', '43throw_51thcolumn', '60throw_182thcolumn', '53throw_184thcolumn', '129throw_195thcolumn', '13throw_165thcolumn', '32throw_112thcolumn', '18throw_95thcolumn', '91throw_165thcolumn', '52throw_191thcolumn', '37throw_124thcolumn', '4throw_16thcolumn', '167throw_169thcolumn', '28throw_196thcolumn', '10throw_63thcolumn', '16throw_162thcolumn', '167throw_190thcolumn', '16throw_74thcolumn', '22throw_179thcolumn', '5throw_28thcolumn', '53throw_127thcolumn', '47throw_164thcolumn', '181throw_191thcolumn', '41throw_152thcolumn', '91throw_187thcolumn', '183throw_187thcolumn', '33throw_48thcolumn', '12throw_114thcolumn', '50throw_83thcolumn', '19throw_97thcolumn', '76throw_100thcolumn', '11throw_51thcolumn', '62throw_174thcolumn', '165throw_185thcolumn', '90throw_199thcolumn', '26throw_139thcolumn', '101throw_158thcolumn', '34throw_164thcolumn', '34throw_149thcolumn', '49throw_91thcolumn', '52throw_125thcolumn', '26throw_89thcolumn', '27throw_132thcolumn', '29throw_173thcolumn', '181throw_187thcolumn', '69throw_142thcolumn', '5throw_197thcolumn', '32throw_110thcolumn', '103throw_127thcolumn', '73throw_164thcolumn', '48throw_130thcolumn', '115throw_119thcolumn', '136throw_148thcolumn', '92throw_99thcolumn', '85throw_138thcolumn', '116throw_131thcolumn', '144throw_172thcolumn', '14throw_132thcolumn', '1throw_2thcolumn', '6throw_47thcolumn', '1throw_83thcolumn', '96throw_131thcolumn', '44throw_192thcolumn', '75throw_159thcolumn', '12throw_69thcolumn', '17throw_44thcolumn', '75throw_115thcolumn', '97throw_196thcolumn', '51throw_87thcolumn', '79throw_86thcolumn', '28throw_100thcolumn', '124throw_147thcolumn', '67throw_192thcolumn', '9throw_134thcolumn', '118throw_137thcolumn', '40throw_96thcolumn', '17throw_64thcolumn', '41throw_119thcolumn', '70throw_177thcolumn', '165throw_170thcolumn', '77throw_87thcolumn', '71throw_189thcolumn', '28throw_140thcolumn', '2throw_105thcolumn', '124throw_161thcolumn', '18throw_124thcolumn', '86throw_196thcolumn', '88throw_129thcolumn', '158throw_189thcolumn', '26throw_93thcolumn', '102throw_105thcolumn', '60throw_192thcolumn', '126throw_136thcolumn', '53throw_130thcolumn', '148throw_161thcolumn', '167throw_174thcolumn', '65throw_172thcolumn', '121throw_192thcolumn', '15throw_58thcolumn', '41throw_198thcolumn', '76throw_165thcolumn', '105throw_184thcolumn', '27throw_40thcolumn', '84throw_92thcolumn', '80throw_123thcolumn', '19throw_96thcolumn', '7throw_177thcolumn', '44throw_53thcolumn', '46throw_94thcolumn', '134throw_178thcolumn', '47throw_136thcolumn', '84throw_167thcolumn', '100throw_150thcolumn', '40throw_190thcolumn', '124throw_198thcolumn', '158throw_164thcolumn', '134throw_196thcolumn', '53throw_64thcolumn', '13throw_25thcolumn', '27throw_76thcolumn', '9throw_182thcolumn', '40throw_77thcolumn', '93throw_157thcolumn', '157throw_166thcolumn', '119throw_156thcolumn', '46throw_115thcolumn', '70throw_102thcolumn', '65throw_129thcolumn', '22throw_98thcolumn', '127throw_192thcolumn', '67throw_160thcolumn', '49throw_182thcolumn', '13throw_63thcolumn', '1throw_108thcolumn', '126throw_171thcolumn', '37throw_144thcolumn', '44throw_133thcolumn', '42throw_43thcolumn', '45throw_195thcolumn', '110throw_120thcolumn', '130throw_191thcolumn', '7throw_10thcolumn', '56throw_112thcolumn', '36throw_58thcolumn', '60throw_140thcolumn', '45throw_119thcolumn', '56throw_134thcolumn', '107throw_138thcolumn', '11throw_73thcolumn', '17throw_167thcolumn', '93throw_99thcolumn', '1throw_187thcolumn', '165throw_178thcolumn', '36throw_101thcolumn', '64throw_194thcolumn', '5throw_102thcolumn', '38throw_119thcolumn', '114throw_154thcolumn', '129throw_171thcolumn', '35throw_199thcolumn', '78throw_165thcolumn', '19throw_27thcolumn', '14throw_160thcolumn', '50throw_122thcolumn', '24throw_101thcolumn', '106throw_160thcolumn', '91throw_188thcolumn', '92throw_198thcolumn', '4throw_62thcolumn', '88throw_150thcolumn', '22throw_120thcolumn', '40throw_165thcolumn', '92throw_114thcolumn', '17throw_194thcolumn', '97throw_192thcolumn', '162throw_170thcolumn', '60throw_130thcolumn', '1throw_121thcolumn', '8throw_164thcolumn', '35throw_102thcolumn', '51throw_140thcolumn', '5throw_161thcolumn', '78throw_164thcolumn', '48throw_97thcolumn', '57throw_166thcolumn', '37throw_196thcolumn', '53throw_177thcolumn', '143throw_173thcolumn', '64throw_163thcolumn', '119throw_132thcolumn', '27throw_67thcolumn', '2throw_141thcolumn', '48throw_155thcolumn', '68throw_153thcolumn', '52throw_160thcolumn', '62throw_186thcolumn', '143throw_148thcolumn', '155throw_184thcolumn', '2throw_151thcolumn', '97throw_179thcolumn', '35throw_48thcolumn', '21throw_171thcolumn', '35throw_75thcolumn', '110throw_141thcolumn', '1throw_103thcolumn', '82throw_130thcolumn', '16throw_81thcolumn', '33throw_111thcolumn', '84throw_117thcolumn', '125throw_152thcolumn', '74throw_192thcolumn', '37throw_157thcolumn', '101throw_123thcolumn', '54throw_170thcolumn', '21throw_116thcolumn', '15throw_156thcolumn', '41throw_82thcolumn', '47throw_147thcolumn', '54throw_65thcolumn', '19throw_44thcolumn', '188throw_191thcolumn', '66throw_80thcolumn', '15throw_28thcolumn', '12throw_73thcolumn', '91throw_157thcolumn', '24throw_142thcolumn', '21throw_56thcolumn', '136throw_137thcolumn', '4throw_193thcolumn', '68throw_179thcolumn', '55throw_133thcolumn', '53throw_88thcolumn', '145throw_155thcolumn', '125throw_151thcolumn', '158throw_186thcolumn', '135throw_165thcolumn', '93throw_152thcolumn', '71throw_182thcolumn', '21throw_55thcolumn', '139throw_158thcolumn', '84throw_118thcolumn', '124throw_155thcolumn', '48throw_64thcolumn', '111throw_179thcolumn', '10throw_110thcolumn', '118throw_169thcolumn', '0throw_89thcolumn', '33throw_117thcolumn', '93throw_192thcolumn', '64throw_103thcolumn', '78throw_102thcolumn', '7throw_175thcolumn', '43throw_60thcolumn', '34throw_116thcolumn', '142throw_174thcolumn', '90throw_136thcolumn', '160throw_182thcolumn', '63throw_162thcolumn', '55throw_106thcolumn', '46throw_127thcolumn', '60throw_62thcolumn', '67throw_188thcolumn', '153throw_177thcolumn', '28throw_170thcolumn', '127throw_188thcolumn', '138throw_162thcolumn', '33throw_180thcolumn', '145throw_168thcolumn', '113throw_183thcolumn', '11throw_163thcolumn', '19throw_121thcolumn', '96throw_185thcolumn', '0throw_50thcolumn', '10throw_44thcolumn', '28throw_30thcolumn', '101throw_127thcolumn', '66throw_125thcolumn', '28throw_191thcolumn', '95throw_119thcolumn', '102throw_188thcolumn', '21throw_34thcolumn', '121throw_132thcolumn', '47throw_71thcolumn', '44throw_168thcolumn', '65throw_77thcolumn', '109throw_159thcolumn', '75throw_110thcolumn', '2throw_79thcolumn', '84throw_88thcolumn', '91throw_117thcolumn', '49throw_102thcolumn', '71throw_181thcolumn', '53throw_142thcolumn', '147throw_183thcolumn', '57throw_73thcolumn', '52throw_57thcolumn', '115throw_159thcolumn', '163throw_189thcolumn', '81throw_120thcolumn', '67throw_170thcolumn', '11throw_52thcolumn', '87throw_153thcolumn', '39throw_62thcolumn', '55throw_72thcolumn', '66throw_78thcolumn', '58throw_75thcolumn', '80throw_181thcolumn', '53throw_81thcolumn', '34throw_193thcolumn', '86throw_148thcolumn', '5throw_55thcolumn', '45throw_56thcolumn', '162throw_168thcolumn', '23throw_33thcolumn', '86throw_150thcolumn', '16throw_172thcolumn', '103throw_129thcolumn', '18throw_143thcolumn', '42throw_147thcolumn', '108throw_177thcolumn', '34throw_185thcolumn', '54throw_119thcolumn', '88throw_161thcolumn', '45throw_77thcolumn', '118throw_127thcolumn', '145throw_189thcolumn', '155throw_187thcolumn', '35throw_80thcolumn', '89throw_117thcolumn', '134throw_180thcolumn', '120throw_199thcolumn', '23throw_141thcolumn', '34throw_137thcolumn', '14throw_164thcolumn', '19throw_80thcolumn', '9throw_71thcolumn', '57throw_80thcolumn', '5throw_54thcolumn', '155throw_163thcolumn', '130throw_192thcolumn', '18throw_99thcolumn', '33throw_91thcolumn', '81throw_102thcolumn', '22throw_159thcolumn', '59throw_139thcolumn', '161throw_191thcolumn', '129throw_182thcolumn', '177throw_183thcolumn', '63throw_180thcolumn', '74throw_160thcolumn', '3throw_34thcolumn', '152throw_162thcolumn', '35throw_109thcolumn', '127throw_138thcolumn', '53throw_152thcolumn', '152throw_180thcolumn', '18throw_191thcolumn', '11throw_58thcolumn', '107throw_110thcolumn', '89throw_196thcolumn', '123throw_171thcolumn', '54throw_93thcolumn', '61throw_135thcolumn', '133throw_177thcolumn', '46throw_171thcolumn', '36throw_118thcolumn', '22throw_73thcolumn', '165throw_172thcolumn', '13throw_173thcolumn', '121throw_134thcolumn', '49throw_150thcolumn', '82throw_154thcolumn', '143throw_163thcolumn', '119throw_128thcolumn', '146throw_189thcolumn', '141throw_192thcolumn', '6throw_56thcolumn', '15throw_50thcolumn', '76throw_81thcolumn', '129throw_142thcolumn', '97throw_103thcolumn', '69throw_193thcolumn', '25throw_102thcolumn', '101throw_134thcolumn', '84throw_183thcolumn', '97throw_134thcolumn', '12throw_70thcolumn', '34throw_97thcolumn', '40throw_125thcolumn', '77throw_111thcolumn', '47throw_60thcolumn', '17throw_175thcolumn', '30throw_161thcolumn', '114throw_160thcolumn', '57throw_136thcolumn', '42throw_146thcolumn', '90throw_141thcolumn', '20throw_142thcolumn', '115throw_193thcolumn', '33throw_103thcolumn', '92throw_150thcolumn', '41throw_182thcolumn', '3throw_101thcolumn', '119throw_142thcolumn', '3throw_115thcolumn', '68throw_194thcolumn', '101throw_112thcolumn', '13throw_50thcolumn', '54throw_61thcolumn', '26throw_186thcolumn', '123throw_191thcolumn', '2throw_3thcolumn', '82throw_96thcolumn', '123throw_135thcolumn', '70throw_101thcolumn', '46throw_199thcolumn', '55throw_150thcolumn', '58throw_130thcolumn', '55throw_153thcolumn', '39throw_101thcolumn', '153throw_182thcolumn', '22throw_35thcolumn']

Cikarilacak_sutunlar=Deneme1+Deneme2+Deneme3+Deneme4+Deneme5

test = test.drop(Cikarilacak_sutunlar, axis=1)
train = train.drop(Cikarilacak_sutunlar, axis=1)

print(test.isna().sum()[test.isna().sum() > 0])

print(train.isna().sum()[train.isna().sum() > 0])

def doldur_fonksiyon(sutun):
    if sutun.dtype == 'object':  # Eğer sütun tipi 'object' ise
        return sutun.fillna('unknown')  # 'unknown' ile doldur
    else:  # Diğer durumlarda (sayısal türler için)
        #return sutun.fillna(sutun.mean())  # Ortalama ile doldur
        return sutun.fillna(-1)  
if fill_nan:
    train = train.apply(doldur_fonksiyon)
    test = test.apply(doldur_fonksiyon)

total_nan_count = train.ADHD_Outcome.isna().sum()
print("Toplam NaN Sayısı:", total_nan_count)
y1 = y['ADHD_Outcome'].copy()
y2 = y['Sex_F'].copy()
y2.head()
yc=y['ADHD_Outcome']+y['Sex_F']*2
train.set_index('participant_id',inplace=True)
test.set_index('participant_id',inplace=True)
targets = ['ADHD_Outcome','Sex_F']
features = test.columns
def check_for_nulls(df):
  """
  Checks for null values in a pandas DataFrame and prints a message.

  Args:
    df: The pandas DataFrame to check.

  Returns:
    None
  """
  if df.isnull().any().any():
    print("The DataFrame contains null values.")
  else:
    print("The DataFrame does not contain null values.")

check_for_nulls(train)
check_for_nulls(test)
print(f'Train: {train.shape}, Test: {test.shape}')

set(train.columns.to_list())-set(test.columns.to_list())
func_fea = [sutun for sutun in func_fea if sutun not in Cikarilacak_sutunlar]
if use_pca:
    #pca = PCA(n_components=50)
    pca = PCA(n_components=n_components) 
    pca.fit(pd.concat([train.loc[:,func_fea].copy(), test.loc[:,func_fea].copy()], axis=0))
    pca_func_test = pca.transform(test.loc[:,func_fea])
    pca_func_train = pca.transform(train.loc[:,func_fea])
    print("###", pca_func_train.shape)
    print("###", pca_func_test.shape)
    pca_df_train = pd.DataFrame(pca_func_train, columns=[f'PC_{i+1}' for i in range(pca_func_train.shape[1])])
    pca_df_test = pd.DataFrame(pca_func_test, columns=[f'PC_{i+1}' for i in range(pca_func_test.shape[1])])
    pca_df_train.index=train.index
    pca_df_test.index=test.index
    train.drop(func_fea,axis=1,inplace=True)
    test.drop(func_fea,axis=1,inplace=True)
    train = pd.concat([train, pca_df_train], axis=1)
    test = pd.concat([test, pca_df_test], axis=1)
    print(train.shape)
    print(test.shape)
    train.head(1)

test.head(1)
def fea_eng(df):
    df["fea_1"] = df['ColorVision_CV_Score'] / (df['EHQ_EHQ_Total'] + 1e-9) 

    
    return df

train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
FEATURES=test.columns.to_list()
def Train_Model(model, train,test,y,folds=5,random_state=42):
    # 5 katlı çapraz doğrulama ile validation tahminlerini alalım
    cv = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
    val_preds = np.zeros(train.shape[0], dtype=float)
    test_preds = np.zeros(test.shape[0], dtype=float)
    #'ADHD_Outcome', 'Sex_F' 
    for fold, (train_idx, valid_idx) in enumerate(cv.split(train,y)):
        X_train, X_valid = train.iloc[train_idx,:], train.iloc[valid_idx,:]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        # Stacking modelini eğit
        model.fit(X_train, y_train)
        # Validation seti için tahmin yap
        val_pre = model.predict(X_valid)
        val_pre = np.where(val_pre < 0.5, 0, 1)
        val_preds[valid_idx] = val_pre
        test_preds +=model.predict(test)
        train_preds= model.predict(X_train)
        train_preds = np.where(train_preds < 0.5, 0, 1)
        f1 = f1_score(train_preds,y_train)
        print(f"{fold} - Train F1 Skoru:", f1)
        f1 = f1_score(val_pre,y_valid)
        print(f"{fold} - Val F1 Skoru:", f1)
    test_preds /= float(folds)
    test_preds = np.where(test_preds < 0.5, 0, 1)
    #test_preds = test_preds[test_preds < 0.5] = 0
    # Validation set tahminlerini değerlendirelim
    val_accuracy = accuracy_score(val_preds,y)
    print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
    f1 = f1_score(val_preds,y)
    print("ALL Val F1 Skoru:", f1)
    print(classification_report(val_preds,y))
    return test_preds

def Train_Model_WithEvalSet(model, train,test,y,folds=5,random_state=42):
    # 5 katlı çapraz doğrulama ile validation tahminlerini alalım
    cv = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
    val_preds = np.zeros(train.shape[0], dtype=float)
    test_preds = np.zeros(test.shape[0], dtype=float)
    #'ADHD_Outcome', 'Sex_F' 
    for fold, (train_idx, valid_idx) in enumerate(cv.split(train,y)):
        X_train, X_valid = train.iloc[train_idx,:], train.iloc[valid_idx,:]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        # Stacking modelini eğit
        model.fit(X_train, y_train,
                 eval_set = [(X_valid, y_valid)])
        # Validation seti için tahmin yap
        val_pre = model.predict(X_valid)
        val_pre = np.where(val_pre < 0.5, 0, 1)
        val_preds[valid_idx] = val_pre
        test_preds +=model.predict(test)
        train_preds= model.predict(X_train)
        train_preds = np.where(train_preds < 0.5, 0, 1)
        f1 = f1_score(train_preds,y_train)
        print(f"{fold} - Train F1 Skoru:", f1)
        f1 = f1_score(val_pre,y_valid)
        print(f"{fold} - Val F1 Skoru:", f1)
    test_preds /= float(folds)
    test_preds = np.where(test_preds < 0.5, 0, 1)
    #test_preds = test_preds[test_preds < 0.5] = 0
    # Validation set tahminlerini değerlendirelim
    val_accuracy = accuracy_score(val_preds,y)
    print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
    f1 = f1_score(val_preds,y)
    print("ALL Val F1 Skoru:", f1)
    print(classification_report(val_preds,y))
    return test_preds

SEX_FEATURES = [col for col in FEATURES if col.startswith("PC_")]
ADHD_FEATURES = [col for col in FEATURES if not col.startswith("PC_")]

# One-hot encode ADHD_FEATURES
adhd_train = train[ADHD_FEATURES]
adhd_test = test[ADHD_FEATURES]

# Use pandas get_dummies for simplicity
adhd_train_encoded = pd.get_dummies(adhd_train, drop_first=True)
adhd_test_encoded = pd.get_dummies(adhd_test, drop_first=True)

# Align columns in case train/test have different categories
adhd_train_encoded, adhd_test_encoded = adhd_train_encoded.align(adhd_test_encoded, join='left', axis=1, fill_value=0)

ADHD_FEATURES

lgbm_params = {
    "objective": "regression",
    "metric": "binary_logloss",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "n_estimators": 1253,
    "learning_rate": 0.039856150643362405,
    "num_leaves": 108,
    "max_depth": 5,
    "min_data_in_leaf": 62,
    "lambda_l1": 0.007248547448790944,
    "lambda_l2":  0.0010950522341775359,
    "colsample_bytree": 0.9583664658179115,
    "subsample": 0.9368720752732888,
    "subsample_freq": 1,
    "feature_fraction": 0.929215486907181,
    "bagging_fraction": 0.726036842383016,
    "bagging_freq": 21,
    #"min_gain_to_split": 0.01,
    "random_state": 42 
}
model_lgbm = LGBMClassifier(**lgbm_params)

cat_params = {
    'iterations': 1105,               # Modelin eğitim iterasyon sayısı
    'learning_rate': 0.05607127003608688,           # Öğrenme oranı
    'depth': 6,                      # Ağaç derinliği
    'l2_leaf_reg': 4.533796421065149,                # L2 regularization parametresi
    'border_count': 81,              # Ağaç sınırları sayısı
    'loss_function': 'Logloss',      # Binary classification için kayıp fonksiyonu
    'cat_features': [],              # Kategorik özelliklerin indeksleri
    'thread_count': -1,              # Kullanılacak iş parçacığı sayısı (-1 tüm çekirdekleri kullanır)
    'random_seed': 42,               # Rastgelelikeyi kontrol etmek için başlangıç tohumu
    'verbose': 0,                  # Eğitim sırasında her 100 iterasyonda bilgi yazdırılır
    'task_type': 'CPU',              # CPU ya da GPU kullanımı ('GPU' olarak da ayarlanabilir)
    'grow_policy': 'Depthwise',  # Ağaç büyütme politikası, alternatif: 'Depthwise' veya 'Lossguide'
    #'max_bin': 254,                  # Bin sayısı (sürekli değişkenlerin kategorize edilmesinde kullanılır)
    #'early_stopping_rounds': 50,     # Erken durdurma sayısı, doğrulama seti üzerinde iyileşme gözlemlenmezse durur
    'scale_pos_weight': 1,           # Dengesiz veri setlerinde pozitif sınıfı ağırlıklı hale getirmek için kullanılır
}
model_cat = CatBoostClassifier(**cat_params)

xgb_params = {
    'objective': 'binary:logistic',   # Kayıp fonksiyonu (binary classification için logloss kullanılır)
    'eval_metric': 'logloss',         # Değerlendirme metriği (logloss, AUC vs.)
    'learning_rate': 0.1,             # Öğrenme oranı (genellikle 0.01 ile 0.3 arasında değerler kullanılır)
    'max_depth': 6,                   # Ağaç derinliği (modelin karmaşıklığı)
    'n_estimators': 3000,              # Modeldeki toplam ağaç sayısı
    'subsample': 0.8,                 # Veri alt örnekleme oranı (1.0 tam veri kullanımı, <1.0 veri alt kümesi kullanır)
    'colsample_bytree': 0.8,          # Her bir ağaç için rastgele seçilen özelliklerin oranı
    'colsample_bylevel': 0.8,         # Her seviyede rastgele seçilen özelliklerin oranı
    'gamma': 0,                       # Ağaç büyütme sırasında minimum kayıp azaltımı (daha yüksek gamma, daha az ağaç büyümesine neden olur)
    'scale_pos_weight': 1,            # Dengesiz sınıflar için pozitif sınıfın ağırlığı
    'lambda': 1,                      # L2 regularizasyon parametresi
    'alpha': 0,                       # L1 regularizasyon parametresi
    'random_state': 42,               # Rastgelelikeyi kontrol etmek için başlangıç tohumu
    'n_jobs': -1,                     # Paralel işlem için kullanılacak çekirdek sayısı (-1, tüm çekirdekleri kullanır)
    'tree_method': 'auto',            # Ağaç büyütme metodu ('auto', 'exact', 'approx', 'hist', 'gpu_hist')
    'booster': 'gbtree',              # Ağaç tabanlı model ('gbtree' ya da 'dart' seçilebilir)
    'verbosity': 1                    # Eğitim sırasında bilgi seviyesini kontrol eder
}
model_xgb = XGBClassifier(**xgb_params)
models = [('lgbm', model_lgbm),
          ('catboost', model_cat),
          ('xgboost', model_xgb) ]
logreg = LogisticRegression(max_iter=5000,random_state=227,class_weight={0:0.4,1:0.6})
# StackingClassifier oluştur
stacking_model = StackingClassifier(
    estimators=models,
    final_estimator=logreg,
    passthrough=False  # True yaparsan, temel tahminlerin yanı sıra özellikleri de kullanır
)
voting_clf = VotingClassifier(estimators=models, voting='soft')
for i in range(1,11,1):
    print(i)
    logreg = LogisticRegression(max_iter=5000,random_state=227,class_weight={0:i/10,1:1-i/10})
    # StackingClassifier oluştur
    stacking_model = StackingClassifier(
        estimators=models,
        final_estimator=logreg,
        passthrough=False  # True yaparsan, temel tahminlerin yanı sıra özellikleri de kullanır
    )

i=4
print(i)
logreg = LogisticRegression(max_iter=5000,random_state=227,class_weight={0:i/10,1:1-i/10})
    # StackingClassifier oluştur
stacking_model = StackingClassifier(
        estimators=models,
        final_estimator=logreg,
        passthrough=False  # True yaparsan, temel tahminlerin yanı sıra özellikleri de kullanır
)
    #{'ADHD_Outcome', 'Sex_F'}
test_pred_y1 = Train_Model(stacking_model, adhd_train_encoded, adhd_test_encoded, train.loc[:, 'ADHD_Outcome'], folds=5, random_state=42)

sample_sub['ADHD_Outcome'] = test_pred_y1
sample_sub.to_csv('submission.csv', index=False)

APQ_P_APQ_P_ID                      15
SDQ_SDQ_Generating_Impact           30
SDQ_SDQ_Hyperactivity               30
PreInt_Demos_Fam_Child_Ethnicity     3
PreInt_Demos_Fam_Child_Race          6
dtype: int64
APQ_P_APQ_P_ID                      12
SDQ_SDQ_Generating_Impact            9
SDQ_SDQ_Hyperactivity                9
PreInt_Demos_Fam_Child_Ethnicity    43
PreInt_Demos_Fam_Child_Race         54
dtype: int64
Toplam NaN Sayısı: 0
The DataFrame does not contain null values.
The DataFrame does not contain null values.
Train: (1213, 16833), Test: (304, 16831)
### (1213, 825)
### (304, 825)
(1213, 832)
(304, 830)
1
2
3
4
5
6
7
8
9
10
4
0 - Train F1 Skoru: 0.8870967741935484
0 - Val F1 Skoru: 0.8663101604278075
1 - Train F1 Skoru: 0.8887381275440976
1 - Val F1 Skoru: 0.8432432432432433
2 - Train F1 Skoru: 0.8799463447350772
2 - Val F1 Skoru: 0.837696335078534
3 - Train F1 Skoru: 0.8706666666666667
3 - Val F1 Skoru: 0.8633879781420765
4 - Train F1 Skoru: 0.8847736625514403
4 - Val F1 Skor